# Financial Model Main Code (Final Version- Sept 14, 2025)

Contributors: Simon Bruno for Cornell University

The code is divided into the following sections (slightly differ from fig 2. of the paper "Integrating farm and solar business practices for agrivoltaics financial modeling" which had been simplified for the sake of clarity):

0 - Importing the necessary libraries and data

1 - Selecting the project location

2 - Selecting the project lifetime

3 - Selecting the APV partnership type

4 - Selecting the financial parameters

5 - Defining the crop schedule

6 - Designing the conventional systems (both PV and farm businesses)

7 - Designing the APV systems (both PV and farm businesses)

8 - Computing the financial results


Classes are used to encapsulate the data and methods related to each section, allowing for better organization and reusability of code. 

After all classes are defined, the required calculations are performed and the results are printed.

# A - Defining modules (code core functionality)

## 0 - Importing the necessary libraries and data

In [ ]:
# 0.1 - Importing necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.font_manager as fm
from ipywidgets import widgets, Layout
from IPython.display import display
from ipywidgets import Output
from scipy import stats
import ast  # Importing ast for safe conversion of string representations of lists to actual lists
import os  
from openpyxl import load_workbook



In [ ]:
# 0.2 - Creating a DataFrame with the activities available in different states(i.e., the available crops to grow in a given state) 

# Importing the cleaned and combined yield and price data from the CSV file

# Folder where this notebook is being run
BASE_DIR = os.getcwd()
# Path to the "data" folder in the repo
DATA_FOLDER = os.path.join(BASE_DIR, "data")
# Full path to your CSV file
PATH = os.path.join(DATA_FOLDER, "combined_yield_price_common_data.csv")


# Creating a DataFrame to represent the crops available in different states (will be used for crops selection)
crops_df = pd.read_csv(PATH)

def convert_columns_to_lists(df):
    """Converts object columns in the DataFrame to lists."""
    for column in df.columns[1:]:  # Skip the first column ('State')
        # Check if the column contains object data that should be lists
        if df[column].dtype == 'object':
            df[column] = df[column].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    return df

crops_df = convert_columns_to_lists(crops_df)



## 1 - Selecting the project location

In [ ]:
class Location:
    """ 
    Class to select a project location (for this version only states level).

    Returns:
    - Location: An instance of the Location class with the selected state. 
    """

    def __init__(self):
        self.output = Output()  # Output widget to capture print
        self.state = None  # This will hold the selected state
        


        # Create dropdown for states
        self.state_dropdown = widgets.Dropdown(
            options=crops_df['State'].tolist(),  # Use states from the DataFrame directly
            description='Select State:',
            disabled=False,
        )

        # Button to confirm selection
        self.confirm_button = widgets.Button(description="Confirm Location")
        self.confirm_button.on_click(self.confirm_selection)

        # Display the dropdown and button
        display(self.state_dropdown, self.confirm_button, self.output)

    def confirm_selection(self, b):
        """ Confirm the selected state and print the result. """
        self.state = self.state_dropdown.value
        with self.output:
            self.output.clear_output()  # Clear previous output
            print(f'Selected Location: {self.state}')
    

## 2 - Selecting the project lifetime

In [ ]:
class Lifetime: 
    """ 
    Class to select the project lifetime in years.

    Returns:
    - Project lifetime (yrs).  
    """
    
    def __init__(self):
        self.output = Output()
        self.years = None  
    
        # Create text box for years
        self.years_widget = widgets.IntText(
                value=25,
                description='Enter Project Lifetime (years):',
                disabled=False
            )
    
        self.confirm_button = widgets.Button(description="Confirm Year")
        self.confirm_button.on_click(self.confirm_selection)
    
        
        display(self.years_widget, self.confirm_button, self.output)
    
    def confirm_selection(self, b):
        """ Confirm the selected year and print the result. """
        self.years = self.years_widget.value
        with self.output:
            self.output.clear_output()
            print(f'Project Lifetime: {self.years} years')



## 3 - Selecting the APV partnership type

In [ ]:
class Partnership:
    """
    Class to select the partnership type between the Solar Developer and the Farmer in the APV case.

    Partnership Types:
    - Partnership A: a third-party landowner leases land to a solar developer, and the developer hires a farmer to maintain crops in the APV facility
    - Partnership B: the landowner and farmer are the same entity

    Returns:
    - Partnership Type: Selected partnership type
    - Annual Lease Payment from Solar Developer (SD) to Landowner during Partnership A ($/acre)
    - Annual Lease Payment from SD to farmer during Partnership B ($/acre)
    - Extra Payment from SD to farmer during Partnership A ($/acre)
    """

    def __init__(self):
        self.output = Output()
        # Initialize parameters
        self.partnership_type = None
        self.land_lease_sd_to_landowner = None # Partnership A
        self.land_lease_sd_to_farmer = None  # Partnership B
        self.extra_payment_sd_to_farmer = None # Partnership A
        
        # Create widgets for inputs

        common_layout = widgets.Layout(width='700px')  

        self.partnership_dropdown = widgets.Dropdown(
            options=[
                'Partnership A: Solar Developer, Farmer and Landowner',
                'Partnership B: Solar Developer and Farmer/Landowner',
            ],
            description='Partnership Type:',
            value='Partnership A: Solar Developer, Farmer and Landowner',
            style={'description_width': 'initial'},
            layout=common_layout
        )

        self.land_lease_sd_to_landowner_input = widgets.FloatText(
            description="Annual Lease Payment from Solar Developer (SD) to Landowner during Partnership A ($/acre):",
            value=0,
            style={'description_width': 'initial'},
            layout=common_layout
        )

        self.land_lease_sd_to_farmer_input = widgets.FloatText(
            description="Annual Lease Payment from SD to Farmer during Partnership B ($/acre):",
            value=0,
            style={'description_width': 'initial'},
            layout=common_layout
        )

        self.extra_payment_sd_to_farmer_input = widgets.FloatText(
            description="Extra Payment from SD to farmer during Partnership A ($/acre)",
            value=0,
            style={'description_width': 'initial'},
            layout=common_layout
        )

        self.confirm_button = widgets.Button(description="Confirm Selection")
        self.confirm_button.on_click(self.confirm_selection)

        display(self.partnership_dropdown, self.land_lease_sd_to_landowner_input, self.land_lease_sd_to_farmer_input, self.extra_payment_sd_to_farmer_input, self.confirm_button, self.output)

    def confirm_selection(self, b):
        """ Store the values from the widgets and compute farmer payment if needed. """
        self.partnership_type = self.partnership_dropdown.value
        self.extra_payment_sd_to_farmer = self.extra_payment_sd_to_farmer_input.value
        
        if self.partnership_type.startswith("Partnership B"):
            # For Partnership B, 
            self.land_lease_sd_to_landowner = 0
            self.land_lease_sd_to_farmer = self.land_lease_sd_to_farmer_input.value
            with self.output:
                self.output.clear_output()
                print(f"Partnership Type: {self.partnership_type}")
                print(f"Lease Payment from Solar Developer (SD) to Landowner (Partnership A): ${self.land_lease_sd_to_landowner:.2f}/acre")
                print(f"Lease Payment from SD to Farmer (Partnership B): ${self.land_lease_sd_to_farmer:.2f}/acre")
                print(f"Extra Payment from SD to Farmer (Partnership A): ${self.extra_payment_sd_to_farmer:.2f}/acre")
        else:
            # For Partnership A, there is no lease payment to farmer from solar developer
            self.land_lease_sd_to_landowner = self.land_lease_sd_to_landowner_input.value
            self.land_lease_sd_to_farmer = 0
            with self.output:
                self.output.clear_output()
                print(f"Partnership Type: {self.partnership_type}")
                print(f"Lease Payment from Solar Developer (SD) to Landowner (Partnership A): ${self.land_lease_sd_to_landowner:.2f}/acre")
                print(f"Lease Payment from SD to Farmer (Partnership B): ${self.land_lease_sd_to_farmer:.2f}/acre")
                print(f"Extra Payment from SD to Farmer (Partnership A): ${self.extra_payment_sd_to_farmer:.2f}/acre")
        
    def outputs(self):
        """ Output the partnership details as a dictionary. """
        return {
            "Partnership Type": self.partnership_type,
            "Lease Payment from SD to Landowner (Partnership A) ($/acre)": self.land_lease_sd_to_landowner,
            "Lease Payment from SD to Farmer (Partnership B) ($/acre)": self.land_lease_sd_to_farmer,
            "Extra Payment from SD to Farmer (Partnership A) ($/acre)": self.extra_payment_sd_to_farmer
        }



## 4 - Selecting the financial parameters

### 4.1 - Defining Unit CAPEX and Unit OPEX for PV businesses of conventional and APV systems (not farm businesses).

In [ ]:
class CapexOpex:
    """
    Class to define the Unit CAPEX and OPEX for PV businesses of conventional and APV systems (not farm businesses).

    Returns:
    - Unit CAPEX ($/W) for a given project type (conventional or APV) and system scale (MW) and energy system type (1Axis Tracking, 2 Axis Tracking, Vertical, etc.)
    - Unit OPEX ($/kW) for a given project type (conventional or APV) and system scale (MW) and energy system type (1Axis Tracking, 2 Axis Tracking, Vertical, etc.)
    """

    def __init__(self, system_type="conventional"):
        self.output = Output() 
        # Initialize CAPEX and OPEX components with default values
        self.capex_components_conventional = {
            "Module": 0.35,
            "Inverter": 0.1,
            "BOS": 0.49,
            "Install Labor & Equipment": 0.26,
            "EPC Overhead": 0.17,
            "Sales Tax": 0.0,
            "Permitting Fee": 0.02,
            "Interconnection Fee": 0.04,
            "Contingency": 0.06,
            "Developer Overhead": 0.2,
            "EPC Developer Net Profit": 0.15,
        }

        self.capex_components_apv_options = {
            "1AxisTracking/Bifacial/NormalClearance": {
                "Module": 0.35,
                "Inverter": 0.10,
                "BOS": 0.52,
                "Install Labor & Equipment": 0.27,
                "EPC Overhead": 0.18,
                "Sales Tax": 0.0,
                "Permitting Fee": 0.02,
                "Interconnection Fee": 0.04,
                "Contingency": 0.07,
                "Developer Overhead": 0.21,
                "EPC Developer Net Profit": 0.16,
            },
            "2AxisTracking/Bifacial/HighClearance": {
                "Module": 0.35,
                "Inverter": 0.10,
                "BOS": 0.82,
                "Install Labor & Equipment": 0.27,
                "EPC Overhead": 0.23,
                "Sales Tax": 0.0,
                "Permitting Fee": 0.02,
                "Interconnection Fee": 0.04,
                "Contingency": 0.09,
                "Developer Overhead": 0.24,
                "EPC Developer Net Profit": 0.20,
            },
            "Vertical/Bifacial": {
                "Module": 0.35,
                "Inverter": 0.10,
                "BOS": 0.56,
                "Install Labor & Equipment": 0.31,
                "EPC Overhead": 0.21,
                "Sales Tax": 0.0,
                "Permitting Fee": 0.02,
                "Interconnection Fee": 0.04,
                "Contingency": 0.07,
                "Developer Overhead": 0.22,
                "EPC Developer Net Profit": 0.17,
            },
        }

        self.opex_components_conventional = {
            "Cleaning": 2.7248,
            "Inspection":3.2344,
            "New BOS": 1.4456,
            "New Module": 0.2496,
            "New Inverter": 2.2672,
            "Land Lease": 4.16,
            "Property Tax": 2.0488,
            "Insurance": 2.5584,
            "Management": 1.8720,
        }

        self.opex_components_apv = {
            "Inspection": 2.7248,
            "New BOS": 1.6848,
            "New Module": 0.2496,
            "New Inverter": 3.1512,
            "Property Tax": 2.7872,
            "Insurance": 3.4944,
            "Management": 3.0680,
        }

        # Set the system type (Conventional or APV)
        self.system_type = system_type
        self.current_apv_capex = {}

        # Widgets for CAPEX inputs
        if system_type == "conventional":
            self.capex_widgets = {
                key: widgets.FloatText(value=value, description=f"{key} [$/W]:", style={"description_width": "initial"})
                for key, value in self.capex_components_conventional.items()
            }
            self.opex_widgets = {
                key: widgets.FloatText(
                    value=value, description=f"{key} [$/kW]:", style={"description_width": "initial"}
                )
                for key, value in self.opex_components_conventional.items()
            }
        elif system_type == "APV":
            self.module_type_dropdown = widgets.Dropdown(
                options=list(self.capex_components_apv_options.keys()),
                value="1AxisTracking/Bifacial/NormalClearance",
                description="Module Type:",
                style={"description_width": "initial"},
            )
            self.module_type_dropdown.observe(self.update_apv_capex, names="value")

            # Initialize CAPEX widgets for APV
            self.current_apv_capex = self.capex_components_apv_options[self.module_type_dropdown.value]
            self.capex_widgets = {
                key: widgets.FloatText(value=value, description=f"{key} ($/W):", style={"description_width": "initial"})
                for key, value in self.current_apv_capex.items()
            }
            self.opex_widgets = {
                key: widgets.FloatText(
                    value=value, description=f"{key} ($/W):", style={"description_width": "initial"}
                )
                for key, value in self.opex_components_apv.items()
            }

        
        # Widget for project scale
        self.project_scales = [0.2, 0.5, 1, 5, 10, 15, 20]
        self.project_scale_widget = widgets.Dropdown(
            options=self.project_scales,
            value=0.5,
            description="Project Scale (MW):",
            style={"description_width": "initial"},
        )

        self.confirm_button = widgets.Button(description="Confirm Selection")
        self.confirm_button.on_click(self.confirm_selection)

        print("Additional Inputs:")
        display(self.project_scale_widget)
        if system_type == "APV":
            print("Select Module Type:")
            display(self.module_type_dropdown)
        print("CAPEX Components:")
        display(*self.capex_widgets.values())
        print("OPEX Components:")
        display(*self.opex_widgets.values())
        display(self.confirm_button, self.output)

    def update_apv_capex(self, change):
        """Update CAPEX widgets based on the selected APV module type."""
        self.current_apv_capex = self.capex_components_apv_options[change["new"]]
        for key, widget in self.capex_widgets.items():
            widget.value = self.current_apv_capex[key]

    def confirm_selection(self, b):
        """Store the selected CAPEX and OPEX values."""

        self.project_scale = self.project_scale_widget.value  # Store the selected project scale

        if self.system_type == "conventional":
            self.capex_components_conventional = {
                key: widget.value for key, widget in self.capex_widgets.items()
            }
            self.opex_components_conventional = {
                key: widget.value for key, widget in self.opex_widgets.items()
            }
        elif self.system_type == "APV":
            self.current_apv_capex = {
                key: widget.value for key, widget in self.capex_widgets.items()
            }
            self.opex_components_apv = {
                key: widget.value for key, widget in self.opex_widgets.items()
            }
        with self.output:
            self.output.clear_output() 
            print("Selection Confirmed")
            print(f"For a Project Scale: {self.project_scale} MW")

    def compute_total_unit_capex(self):
        """Compute the total Unit CAPEX."""
        if self.system_type == "conventional":
            total_unit_capex = sum(self.capex_components_conventional.values())
        elif self.system_type == "APV":
            total_unit_capex = sum(self.current_apv_capex.values())
        return total_unit_capex

    def compute_total_unit_opex(self):
        """Compute the total OPEX."""
        if self.system_type == "conventional":
            total_unit_opex = sum(self.opex_components_conventional.values())
        elif self.system_type == "APV":
            total_unit_opex = sum(self.opex_components_apv.values())
        return total_unit_opex
    
    def compute_total_unit_capex_for_scales(self):
        """
        Compute total CAPEX for all project scales based on the entered project scale.
        If the missing scale is below the entered scale, divide by the correct ratio.
        If the missing scale is above the entered scale, multiply by the correct ratio.
        """
        # Get the entered scale and total unit CAPEX for the entered scale
        entered_scale = self.project_scale_widget.value  # Scale for which the user entered CAPEX
        capex_values = (
            sum(self.capex_components_conventional.values())
            if self.system_type == "conventional"
            else sum(self.current_apv_capex.values())
        )

        # Define scale relationships
        scale_relationships = {
            (0.2, 0.5): 0.8454,
            (0.5, 1): 0.9143,
            (1, 5): 0.91875,
            (5, 10): 0.9455,
            (10, 15): 0.9671,
            (15, 20): 0.968,
        }

        # Sort scales for easier computation
        sorted_scales = sorted(self.project_scales)

        # Compute total unit CAPEX for each scale
        unit_capex_by_scale = {}
        for scale in sorted_scales:
            if scale == entered_scale:
                unit_capex_by_scale[scale] = capex_values
            elif scale < entered_scale:
                # Divide iteratively for scales below the entered scale
                capex = capex_values
                for (low, high), factor in scale_relationships.items():
                    if high <= entered_scale and low >= scale:
                        capex /= factor
                unit_capex_by_scale[scale] = capex
            elif scale > entered_scale:
                # Multiply iteratively for scales above the entered scale
                capex = capex_values
                for (low, high), factor in scale_relationships.items():
                    if low >= entered_scale and high <= scale:
                        capex *= factor
                unit_capex_by_scale[scale] = capex

        # Debugging: Print results
        print(f"Unit CAPEX values extrapolated for all scales based on {entered_scale} MW input:")
        for scale, total_capex in unit_capex_by_scale.items():
            print(f"  {scale} MW: ${total_capex:,.2f}/W")
        return unit_capex_by_scale
    
    
    def find_best_trendline(self):
        """
        Find the best trendline (linear, logarithmic, or power) for the CAPEX data across scales.

        Parameters:
        - total_capex_by_scale: Dictionary with project scales as keys and total CAPEX as values.

        Returns:
        - Best trendline type (linear, log, or power) and its coefficients.
        """
        # Extract scales and CAPEX values
        unit_capex_by_scale = self.compute_total_unit_capex_for_scales()
        scales = np.array(list(unit_capex_by_scale.keys()))
        capex_values = np.array(list(unit_capex_by_scale.values()))

        # Initialize best regression
        best_r2 = -np.inf
        best_type = None
        best_coefficients = None

        # 1. Linear regression
        slope, intercept, r_value, _, _ = stats.linregress(scales, capex_values)
        r2_linear = r_value**2
        if r2_linear > best_r2:
            best_r2 = r2_linear
            best_type = "Linear"
            best_coefficients = (slope, intercept)

        # 2. Logarithmic regression (Excel-style)
        if np.all(scales > 0):  # Logarithmic regression requires positive x-values
            log_scales = np.log(scales)
            slope, intercept, r_value, _, _ = stats.linregress(log_scales, capex_values)
            r2_log = r_value**2
            if r2_log > best_r2:
                best_r2 = r2_log
                best_type = "Logarithmic"
                best_coefficients = (slope, intercept)

        # 3. Power regression (Excel-style)
        if np.all(scales > 0) and np.all(capex_values > 0):  # Power regression requires positive x and y
            log_scales = np.log(scales)
            log_capex = np.log(capex_values)
            slope, intercept, r_value, _, _ = stats.linregress(log_scales, log_capex)
            r2_power = r_value**2
            if r2_power > best_r2:
                best_r2 = r2_power
                best_type = "Power"
                best_coefficients = (slope, np.exp(intercept)) # --> the regression is log(y) = slope * log(x) + intercept but we will have equation y = exp(intercept) * x^slope

        # Print the best regression results
        print(f"Best Trendline: {best_type}")
        print(f"R² Value: {best_r2:.4f}")
        print(f"Coefficients: {best_coefficients}")

        return best_type, best_coefficients, best_r2



### 4.2 - Defining the financial rates for both conventional and APV systems

In [ ]:
class FinancialRates:
    """
    Class to define Financial Rates for both conventional and APV systems.

    Requires: 
    - Lifetime (from the Lifetime class)

    Returns:
    - Real discount rate (%)
    - Inflation rate (%)
    - Investment Tax Credit (ITC) rate (%)
    - Federal tax rate for both PV and farm businesses (%)
    - State tax rate for both PV and farm businesses (%)
    - Performance-Based Incentive (PBI) in $/kWh
    - Solar Renewable Energy Credit (SREC) in $/kWh
    - Duration of SREC program (years)
    - Energy price escalation rate (%)
    - Debt share (%)
    - Debt interest rate (%)
    - Equity cost (%)
    - Farm costs change (aka gamma in literature) (i.e., change of farm costs when going from conventional to APV) (%)
    - Yield change (when going from conventional to APV) (%)

    Computes and returns:
    - Nominal discount rate (%)
    - Equity share (%)
    - Effective tax rate (%)
    - Weighted Average Cost of Capital (WACC) (%)
    """

    def __init__(self, lifetime):
        self.output = Output()
        self.lifetime = lifetime
        
        # Input widgets for user-defined parameters

        common_layout = widgets.Layout(width='400px')  # Adjust width of input boxes

        self.real_discount_rate_widget = widgets.FloatText(description='Real Discount Rate (%):', value=0, style={'description_width': 'initial'}, layout=common_layout)
        self.inflation_rate_widget = widgets.FloatText(description='Inflation Rate (%):', value=0, style={'description_width': 'initial'}, layout=common_layout)
        self.itc_rate_widget = widgets.IntText(description='Investment Tax Credit (ITC):', value=0, style={'description_width': 'initial'}, layout=common_layout)
        self.fed_tax_rate_pv_widget = widgets.FloatText(description='Federal Tax Rate PV(%):', value=0, style={'description_width': 'initial'}, layout=common_layout)
        self.fed_tax_rate_farm_widget = widgets.FloatText(description='Federal Tax Rate farm(%):', value=0, style={'description_width': 'initial'}, layout=common_layout)
        self.state_tax_rate_pv_widget = widgets.FloatText(description='State Tax Rate PV(%):', value=0, style={'description_width': 'initial'}, layout=common_layout)
        self.state_tax_rate_farm_widget = widgets.FloatText(description='State Tax Rate farm(%):', value=0, style={'description_width': 'initial'}, layout=common_layout)
        self.pbi_widget = widgets.FloatText(description='PBI ($/kWh):', value=0, style={'description_width': 'initial'}, layout=common_layout)
        self.srec_widget = widgets.FloatText(description='SREC ($/kWh):', value=0, style={'description_width': 'initial'}, layout=common_layout)
        self.srec_duration_widget = widgets.IntText(description='SREC Program Duration (years):', value=0, style={'description_width': 'initial'}, layout=common_layout)
        self.energy_price_escalation_widget = widgets.FloatText(description='Energy Price Escalation Rate (%):', value=0, style={'description_width': 'initial'}, layout=common_layout)
        self.debt_share_widget = widgets.FloatText(description='Debt Share (%):', value=0, style={'description_width': 'initial'}, layout=common_layout)
        self.cost_debt_widget = widgets.FloatText(description='Cost of Debt (%):', value=0, style={'description_width': 'initial'}, layout=common_layout)
        self.cost_equity_widget = widgets.FloatText(description='Cost of Equity(%):', value=0, style={'description_width': 'initial'}, layout=common_layout)
        self.farm_costs_change_widget = widgets.FloatText(description='Farm Costs Change (conventional to APV) (%):', value=0, style={'description_width': 'initial'}, layout=common_layout)
        self.yield_change_widget = widgets.FloatText(description='Yield Change (conventional to APV) (%):', value=0, style={'description_width': 'initial'}, layout=common_layout)
        
        # Button to confirm the values
        self.confirm_button = widgets.Button(description="Confirm Financial Rates")
        self.confirm_button.on_click(self.confirm_selection)
        
        # Display widgets
        display(self.real_discount_rate_widget, self.inflation_rate_widget, 
                self.itc_rate_widget, self.fed_tax_rate_pv_widget, self.fed_tax_rate_farm_widget, 
                self.state_tax_rate_pv_widget, self.state_tax_rate_farm_widget, 
                self.pbi_widget, self.srec_widget, self.srec_duration_widget, self.energy_price_escalation_widget, 
                self.debt_share_widget, self.cost_debt_widget, self.cost_equity_widget, self.farm_costs_change_widget, self.yield_change_widget,
                self.confirm_button, self.output)
        
        # Initialize attributes
        self.real_discount_rate = None
        self.inflation_rate = None
        self.itc_rate = None
        self.fed_tax_rate_pv = None
        self.fed_tax_rate_farm = None
        self.state_tax_rate_pv = None
        self.state_tax_rate_farm = None
        self.pbi = None
        self.srec = None
        self.srec_duration = None
        self.energy_price_escalation_rate = None
        self.nominal_discount_rate = None
        self.discount_factors = None
        self.debt_share = None
        self.cost_debt = None
        self.cost_equity = None
        self.wacc = None
        self.farm_costs_change = None
        self.yield_change = None
    

    def confirm_selection(self, b):
        """ Store the values and fetch rates if necessary """
        with self.output:
            self.output.clear_output()
            # Set real discount rate
            self.real_discount_rate = self.real_discount_rate_widget.value / 100
            # Set inflation rate
            self.inflation_rate = self.inflation_rate_widget.value / 100

            # Set the nominal discount rate from the two above rates (don't use get method since you will use the self.nominal_discount_rate later for sensitivity analysis)
            self.get_nominal_discount_rate()

            # Set itc rate
            self.itc_rate = self.itc_rate_widget.value / 100
            
            # Federal tax rate (PV and farm)
            self.fed_tax_rate_pv = self.fed_tax_rate_pv_widget.value / 100
            self.fed_tax_rate_farm = self.fed_tax_rate_farm_widget.value / 100

            # State tax rate (PV and farm)
            self.state_tax_rate_pv = self.state_tax_rate_pv_widget.value / 100 
            self.state_tax_rate_farm = self.state_tax_rate_farm_widget.value / 100
            
            # Set PBI and SREC values
            self.pbi = self.pbi_widget.value
            self.srec = self.srec_widget.value
            self.srec_duration = self.srec_duration_widget.value

            # Escalation Rate 
            self.energy_price_escalation_rate = self.energy_price_escalation_widget.value / 100

            # Set debt share
            self.debt_share = self.debt_share_widget.value / 100
            self.cost_debt = self.cost_debt_widget.value / 100

            # Set equity cost
            self.cost_equity = self.cost_equity_widget.value / 100

            # Set WACC
            self.get_wacc()

            # Set farm costs change
            self.farm_costs_change = 1 + (self.farm_costs_change_widget.value / 100)

            # Set Yield Change
            self.yield_change = 1 + (self.yield_change_widget.value / 100)
            
            # Print confirmed values
            print(f"Financial Rates:\nReal Discount Rate: {self.real_discount_rate * 100}%\n"
                f"Inflation Rate: {self.inflation_rate * 100}%\nNominal Discount Rate: {self.nominal_discount_rate * 100}%\n"
                f"Federal Tax Rate PV: {self.fed_tax_rate_pv * 100}%\nFederal Tax Rate farm: {self.fed_tax_rate_farm * 100}%\n"
                f"State Tax Rate PV: {self.state_tax_rate_pv * 100}%\nState Tax Rate farm: {self.state_tax_rate_farm * 100}%\nITC Rate: {self.itc_rate * 100}%\n"
                f"PBI: {self.pbi} $/kWh\nSREC: {self.srec} $/kWh\nSREC Duration: {self.srec_duration} years\n"
                f"Energy Price Escalation Rate: {self.energy_price_escalation_rate * 100}%\n"
                f"Debt Share: {self.debt_share * 100}%\nCost of Debt: {self.cost_debt * 100}%\nCost of Equity: {self.cost_equity * 100}%\nWACC: {self.wacc * 100}%\n"
                f"Farm Costs Change (conventional to APV): {(self.farm_costs_change * 100) - 100}%\nYield Change (conventional to APV): {(self.yield_change * 100) - 100}%")

    def get_nominal_discount_rate(self):
        """ Compute the nominal discount rate """
        self.nominal_discount_rate = (1 + self.real_discount_rate) * (1 + self.inflation_rate) - 1
        return self.nominal_discount_rate
    
    def get_equity_share(self):
        """ Calculate the equity share """
        return 1 - self.debt_share
    
    def get_effective_tax_rate(self):
        """ 
        Calculate the effective tax rate
        Assume to be the same for both energy and farm businesses 
        """
        return self.state_tax_rate_pv + self.fed_tax_rate_pv * (1-self.state_tax_rate_pv)
    
    def get_wacc(self):
        """ 
        Calculate the Weighted Average Cost of Capital (WACC) 
        """
        self.wacc = self.debt_share * self.cost_debt * (1 - self.get_effective_tax_rate()) + self.get_equity_share() * self.cost_equity
        return self.wacc

    def outputs(self):
        """ Output the financial rates as a dictionary. """
        return {
            "Real Discount Rate": self.real_discount_rate,
            "Inflation Rate": self.inflation_rate,
            "ITC Rate": self.itc_rate,
            "Federal Tax Rate PV": self.fed_tax_rate_pv,
            "Federal Tax Rate farm": self.fed_tax_rate_farm,
            "State Tax Rate PV": self.state_tax_rate_pv,
            "State Tax Rate farm": self.state_tax_rate_farm,
            "PBI ($/kWh)": self.pbi,
            "SREC ($/kWh)": self.srec,
            "SREC Duration (years)": self.srec_duration,
            "Energy Price Escalation Rate": self.energy_price_escalation_rate,
            "Debt Share": self.debt_share,
            "Cost of Debt": self.cost_debt,
            "Cost of Equity": self.cost_equity,
            "Farm Costs Change from Conventional to APV": self.farm_costs_change,
            "Yield Change": self.yield_change
        }

## 5 - Defining the crop schedule

In [ ]:
class CropSchedule:
    """ 
    Class to define the crop schedule for farm business of conventional and APV systems.

    Requires:
    - Location (from the Location class)
    - Lifetime (from the Lifetime class)
    - Crops DataFrame
    
    Returns:
    - Selected crops and their area percentages
    - Yearly allocation of crops (which crops are grown in which years)
    - Establishment and production years for each crop
    """

    def __init__(self, location_instance, lifetime, crops_df):
        self.output = Output()
        self.location = location_instance
        self.lifetime = lifetime
        self.crops_df = crops_df
        self.selected_crops = []  # To store the selected crops
        self.crop_area_percentages = {}  # Store area percentages for each crop
        self.num_crops = 0  # Number of crops specified by the user

        # Widgets to define the number of crops
        self.num_crops_selector = widgets.IntText(
            value=1,
            description="Number of Crops:",
            min=1,
            style={'description_width': 'initial'}
        )
        self.confirm_num_crops_button = widgets.Button(description="Confirm Number of Crops", layout=widgets.Layout(width='400px'))
        self.confirm_num_crops_button.on_click(self.confirm_num_crops)

        # Containers for dynamic widgets
        self.crop_dropdowns = {}
        self.crop_area_sliders = {}
        self.years_checkboxes = {}
        self.tick_all_checkboxes = {}
        self.establishment_inputs = {}
        self.production_inputs = {}
        self.year_allocation = {}

        # Button to confirm selections
        self.confirm_button = widgets.Button(description="Confirm Selection")
        self.confirm_button.on_click(self.confirm_selection)

        # Display number of crops selector
        display(self.num_crops_selector, self.confirm_num_crops_button, self.output)

    def confirm_num_crops(self, b):
        """ Setup widgets dynamically based on the selected number of crops """
        self.num_crops = self.num_crops_selector.value
        print(f"Setting up for {self.num_crops} crops.")
        
        # Clear previous widgets if any
        self.crop_dropdowns.clear()
        self.crop_area_sliders.clear()
        self.years_checkboxes.clear()
        self.tick_all_checkboxes.clear()
        self.establishment_inputs.clear()
        self.production_inputs.clear()

        # Generate widgets for the specified number of crops
        for i in range(1, self.num_crops + 1):
            # Dropdowns
            self.crop_dropdowns[i] = widgets.Dropdown(
                options=[],  # Will be populated based on selected state
                description=f'Select Crop {i}:',
                style={'description_width': 'initial'},
                disabled=False,
            )
            
            # Area sliders
            self.crop_area_sliders[i] = widgets.FloatSlider(
                value=1.0,
                min=0.0,
                max=1.0,
                step=0.01,
                description=f"Crop {i} Area %",
                style={'description_width': 'initial'},
                layout=widgets.Layout(display='none')  # Initially hidden
            )
            
            # Year checkboxes and tick-all checkbox
            self.years_checkboxes[i] = [
                widgets.Checkbox(value=False, description=f'Year {j+1}') 
                for j in range(self.lifetime.years)
            ]
            self.tick_all_checkboxes[i] = widgets.Checkbox(
                value=False,
                description="Tick All Years", 
                indent=False
            )
            self.tick_all_checkboxes[i].observe(self.update_all_years, names='value', type='change')
            
            # Establishment and production inputs
            self.establishment_inputs[i] = widgets.IntText(
                value=0,
                description=f"Crop {i} Establishment Years:",
                style={'description_width': 'initial'},
                layout=widgets.Layout(width="300px")
            )
            self.production_inputs[i] = widgets.IntText(
                value=0,
                description=f"Crop {i} Production Years:",
                style={'description_width': 'initial'},
                layout=widgets.Layout(width="300px")
            )

            # Display widgets
            display(self.crop_dropdowns[i])
            display(widgets.Label(value=f"Crop {i} Year Selection:"))
            display(self.tick_all_checkboxes[i])
            display(widgets.VBox(self.years_checkboxes[i]))
            display(self.crop_area_sliders[i])
            display(self.establishment_inputs[i])
            display(self.production_inputs[i])

        # Display the confirm button
        display(self.confirm_button)

        # Update crop dropdowns with options
        self.update_crop_dropdown()

        # Add logic to dynamically show sliders when crops are selected
        for i in range(1, self.num_crops + 1):
            self.crop_dropdowns[i].observe(self.update_area_sliders, names='value')

    def update_crop_dropdown(self):
        """ Update the crop dropdowns based on the selected state """
        selected_state = self.location.state

        if selected_state is not None:
            # Get the row corresponding to the selected state
            state_data = self.crops_df.loc[self.crops_df['State'] == selected_state]

            if not state_data.empty:
                non_zero_crops = [
                    column for column in state_data.columns[1:]  # Skip the first column (State)
                    if state_data[column].values[0][0] > 0  # Check if yield > 0
                ]

                # Update the crop dropdown options
                for dropdown in self.crop_dropdowns.values():
                    dropdown.options = non_zero_crops
                    # Reset selections
                    dropdown.value = None
            else:
                # Clear options if state data is empty
                for dropdown in self.crop_dropdowns.values():
                    dropdown.options = []

    def update_all_years(self, change):
        """ Automatically tick or untick all year checkboxes for a crop based on the "Tick All Years" checkbox. """
        for crop, tick_all_checkbox in self.tick_all_checkboxes.items():
            if change['owner'] == tick_all_checkbox:
                for year_checkbox in self.years_checkboxes[crop]:
                    year_checkbox.value = tick_all_checkbox.value

    def update_area_sliders(self, change):
        """ Show or hide area sliders based on selected crops """
        for i in range(1, self.num_crops + 1):
            slider = self.crop_area_sliders[i]
            if self.crop_dropdowns[i].value:  # If a crop is selected
                slider.layout.display = ''  # Show the slider
            else:
                slider.layout.display = 'none'  # Hide the slider

    def confirm_selection(self, b):
        """ Confirm the selected state and crops """

        with self.output:
            self.output.clear_output()
            selected_state = self.location.state
            self.selected_crops = [
                self.crop_dropdowns[i].value for i in range(1, self.num_crops + 1)
            ]
            self.selected_crops = [crop for crop in self.selected_crops if crop]  # Filter out None

            if self.selected_crops:
                print(f'Selected State: {selected_state}')
                print(f'Selected Crops: {" / ".join(self.selected_crops)}')

                for i, crop in enumerate(self.selected_crops, 1):
                    years_selected = [j+1 for j, checkbox in enumerate(self.years_checkboxes[i]) if checkbox.value]
                    print(f"{crop} will be grown in years: {years_selected}")
            else:
                print("Please select at least one crop.")

            # Collect area percentages
            self.crop_area_percentages = {
                crop: self.crop_area_sliders[i].value
                for i, crop in enumerate(self.selected_crops, 1)
            }

            # Add logic to process establishment and production years
            self.establishment_years_dict = {
                crop: self.establishment_inputs[i].value for i, crop in enumerate(self.selected_crops, 1)
            }
            self.production_years_dict = {
                crop: self.production_inputs[i].value for i, crop in enumerate(self.selected_crops, 1)
            }

            # Allocate establishment and production years
            self.allocate_years()
                    
            # Print year allocation for each selected crop
            for crop in self.selected_crops:
                allocation = self.year_allocation.get(crop, {})
                if allocation:
                    print(f"{crop} Establishment Years: {allocation['establishment_years']}, " 
                        f"Production Years: {allocation['production_years']}")
                    

    def allocate_years(self):
        """ Allocate establishment and production years for each crop """
        
        for i, crop in enumerate(self.selected_crops, 1):
            selected_years = [j+1 for j, checkbox in enumerate(self.years_checkboxes[i]) if checkbox.value]
            establishment_duration = self.establishment_years_dict.get(crop, 0)
            production_duration = self.production_years_dict.get(crop, 0)

            if len(selected_years) != (establishment_duration + production_duration):
                with self.output:
                    print(f"Error: Total selected years for {crop} do not match the establishment and production durations.")
                continue

            self.year_allocation[crop] = {
                "establishment_years": selected_years[:establishment_duration],
                "production_years": selected_years[establishment_duration:establishment_duration + production_duration]
            }

    def outputs(self):
        """ 
        Returns the selected state, crops, crop-year allocations, and crop area percentages as a structured dictionary.
        """
        crop_years = {
            crop: {
                "selected_years": [j+1 for j, checkbox in enumerate(self.years_checkboxes[i]) if checkbox.value],
                "establishment_years": self.year_allocation.get(crop, {}).get("establishment_years", []),
                "production_years": self.year_allocation.get(crop, {}).get("production_years", [])
            }
            for i, crop in enumerate(self.selected_crops, 1)
        }

        return {
            "state": self.location.state,
            "selected_crops": self.selected_crops,
            "crop_years": crop_years,
            "crop_area_percentages": {crop: self.crop_area_percentages.get(crop, 1.0) for crop in self.selected_crops}
        }



## 6 - Designing the conventional systems (both PV and farm businesses)

In [ ]:
class ConventionalEconomics:
    """
    Class to define the economics for conventional systems (both PV and farm).

    Requires:
    - Location (from the Location class)
    - Cropschedule (from the CropSchedule class)
    - Lifetime (from the Lifetime class)
    - Merged DataFrame (Crops DataFrame)
    - FinancialRates (from the FinancialRates class)
    - Unit CAPEX and OPEX for conventional systems (from the CapexOpex class)

    Returns:
    - Detailed variable costs for establishment and production phases
    - Detailed fixed costs for establishment and production phases
    - Toal crop costs for each year ($/yr)
    - Crop yield and price for each selected crop (t/acres and $/t)
    - PV business specifications and economics 
        - Panel Nameplate Power (Wdc)
        - PV total power (kW)
        - Degradation Rate (%)
        - Total Area/power (acre/kW)
        - Row Spacing (ft)
        - Ground Clearance (ft)
        - Length Panel (ft)
        - Width Panel (ft)
        - Number of Panels per Array (-)
        - PV Yield (kWh/kWp)
        - PV Price ($/kWh)

    Computes and returns:
    - Total site area (acres)
    - Array length and width (ft)
    - Total CAPEX and OPEX for conventional systems (both PV and farm businesses) ($ and $/yr)
    - Conventional annual energy production (kWh/yr)
    - Conventional annual escalated energy price ($/yr)
    - Conventional annual total PV revenue [PBI + SREC + base (i.e., PPA)] ($/yr)    
    """

    def __init__(self, location, cropschedule, lifetime, crops_df, financial_rates, capex_opex_conventional):
        self.output = Output()
        self.location = location
        self.cropschedule = cropschedule  
        self.lifetime = lifetime
        self.crops_df = crops_df
        self.financial_rates = financial_rates
        self.capex_opex = capex_opex_conventional

        # Constant
        self.unitopex_gmpv = self.capex_opex.compute_total_unit_opex()  # $/kWdc

        # Initialize Widgets for crops
        self.detailed_variable_costs_widgets = {
            "establishment": {},
            "production": {}
        }
        self.detailed_fixed_costs_widgets = {
            "establishment": {},
            "production": {}
        }
        self.conventionalcropyield_widgets = {}
        self.conventionalcropprice_widgets = {}
        self.conventionalcropcapex_widgets = {}

        self.conventionalPVpanelpower_widget = self.create_widget('Panel Nameplate Power Capacity (Wdc):')
        self.conventionalPVpower_widget = self.create_widget('PV Business Total Power (kW):')
        self.conventionalPVdegrate_widget = self.create_widget('Degradation Rate (%):')
        self.conventionalareapower_widget = self.create_widget('Total Area/power (acre/kW):')
        self.conventionalPVrowspacing_widget = self.create_widget('Row Spacing (ft):')
        self.conventionalPVgroundclearance_widget = self.create_widget('Ground Clearance (ft):')
        self.conventionalpanellength_widget = self.create_widget('Length Panel (ft):')
        self.conventionalpanelwidth_widget = self.create_widget('Width Panel (ft):')
        self.conventionalPVpanelarray_widget = self.create_widget('Number of Panels per Array (-):')
        self.conventionalPVyield_widget = self.create_widget('PV Energy Yield (kWh/kWp):')
        self.conventionalPVprice_widget = self.create_widget('PV Electricity Selling Price (PPA)($/kWh):')

        # Placeholder variables to store values
        self.detailed_variable_costs = {
            "establishment": {},
            "production": {}
        }
        self.detailed_fixed_costs = {
            "establishment": {},
            "production": {}
        }
        self.conventionalcropyield = {}
        self.conventionalcropprice = {}
        self.conventionalcropcapex = {}
        self.conventionalPVpanelpower = None
        self.conventionalPVpower = None
        self.conventionalPVdegrate = None
        self.conventionalareapower = None
        self.conventionalarea = None
        self.conventionalPVrowspacing = None
        self.conventionalPVgroundclearance = None
        self.conventionalpanellength = None
        self.conventionalpanelwidth = None
        self.conventionalarraylength = None
        self.conventionalarraywidth = None
        self.conventionalPVpanelarray = None
        self.conventionalPVyield = None
        self.conventionalPVprice = None
        self.conventionalPVcapex = None
        self.conventionalPVopex = None

        # Dynamically create crop-specific widgets
        self.create_crop_widgets()

        # Confirm button
        self.confirm_button = widgets.Button(description="Confirm Conventional Economics")
        self.confirm_button.on_click(self.confirm_selection)

        # Display widgets
        print(f"Conventional PV Settings")
        display(self.conventionalPVpower_widget, self.conventionalPVpanelpower_widget, self.conventionalPVdegrate_widget, self.conventionalareapower_widget, 
                self.conventionalPVrowspacing_widget, self.conventionalPVgroundclearance_widget, 
                self.conventionalpanellength_widget, self.conventionalpanelwidth_widget, self.conventionalPVpanelarray_widget,
                self.conventionalPVyield_widget, self.conventionalPVprice_widget, self.confirm_button,self.output)
        
    def create_widget(self, description):
        """Helper to create a styled widget with a specified description."""
        return widgets.FloatText(
            description=description,
            value=0,
            style={'description_width': 'initial'},  # Ensures the description fits without truncation
            layout=Layout(width='700px')  # Adjust width as needed
        )
    def create_detailed_costs_widgets(self, crop, phase):
        """
        Create widgets for detailed costs (variable or fixed) for a specific crop, phase.
        Used for the production phase where costs are the same each year.

        Parameters:
        - crop: Name of the crop (e.g., 'PEPPERS, BELL')
        - phase: 'establishment' or 'production' --> here only production will be used 

        Returns:
        - A dictionary of widgets for detailed costs
        """
        variable_costs = {
            "Establishment & Soil Preparation": self.create_widget(f"{crop} {phase.capitalize()} Establishment & Soil Preparation Costs ($/acre):"),
            "Field Activities": self.create_widget(f"{crop} {phase.capitalize()} Field Activities Costs ($/acre):"),
            "Harvest Activities": self.create_widget(f"{crop} {phase.capitalize()} Harvest Activities Costs ($/acre):"),
            "Packing & Handling Charges": self.create_widget(f"{crop} {phase.capitalize()} Packing & Handling Charges Costs ($/acre):"),
            "Maintenance & Repairs": self.create_widget(f"{crop} {phase.capitalize()} Maintenance & Repairs Costs ($/acre):"),
            "Other Variable Costs": self.create_widget(f"{crop} {phase.capitalize()} Other Variable Costs ($/acre):"),
        }

        fixed_costs = {
            "Machine Equipment & Building Depreciation": self.create_widget(f"{crop} {phase.capitalize()} Machine Equipment & Building Depreciation Costs ($/acre):"),
            "Land Depreciation": self.create_widget(f"{crop} {phase.capitalize()} Land Depreciation Costs ($/acre):"),
            "Machine Equipment & Building Interests": self.create_widget(f"{crop} {phase.capitalize()} Machine Equipment & Building Interests Costs ($/acre):"),
            "Land Interests": self.create_widget(f"{crop} {phase.capitalize()} Land Interests Costs ($/acre):"),
            "Land & Property Taxes": self.create_widget(f"{crop} {phase.capitalize()} Land & Property Taxes ($/acre):"),
            "Miscellaneous Supplies Costs": self.create_widget(f"{crop} {phase.capitalize()} Miscellaneous Supplies Costs ($/acre):"),
            "Insurance Costs": self.create_widget(f"{crop} {phase.capitalize()} Insurance Costs ($/acre):"),
            "Management Costs": self.create_widget(f"{crop} {phase.capitalize()} Management Costs ($/acre):"),
        }

        return variable_costs, fixed_costs

    def create_crop_widgets(self):
        """Dynamically create widgets for each selected crop."""
        for crop in self.cropschedule.selected_crops:
            print(f"Creating widgets for crop: {crop}")

            # Create detailed costs widgets for both establishment and production phases
            # Establishement Costs Widgets (for establishement years costs are different each year)

            cropschedule_outputs = self.cropschedule.outputs()
            establishment_years = cropschedule_outputs.get('crop_years', {}).get(crop, {}).get("establishment_years", [])

            self.detailed_variable_costs_widgets["establishment"][crop] = {
                year: self.create_detailed_costs_widgets(crop, "establishment")[0]
                for year in establishment_years
            }
            self.detailed_fixed_costs_widgets["establishment"][crop] = {
                year: self.create_detailed_costs_widgets(crop, "establishment")[1]
                for year in establishment_years
            }
            # Production Costs Widgets (for production years costs are the same each year)
            self.detailed_variable_costs_widgets["production"][crop], self.detailed_fixed_costs_widgets["production"][crop] = self.create_detailed_costs_widgets(crop, "production")      


            # Yield widgets for establishment years
            self.conventionalcropyield_widgets.setdefault("establishment", {}).setdefault(crop, {})
            for year in establishment_years:
                self.conventionalcropyield_widgets["establishment"][crop][year] = self.create_widget(f'{crop} Yield (t/acre) for Year {year}:')
                
            # Yield widget for production years
            self.conventionalcropyield_widgets.setdefault("production", {}).setdefault(crop, {})
            self.conventionalcropyield_widgets["production"][crop] = self.create_widget(f'{crop} Yield (t/acre) for Production Years:')
            
            # Other crop-specific Economics Widgets
            price_widget = self.create_widget(f'{crop} Price ($/t):')
            self.conventionalcropprice_widgets[crop] = price_widget
            capex_widget = self.create_widget(f'{crop} Unit Capex ($/acre):')
            self.conventionalcropcapex_widgets[crop] = capex_widget

            # Display widgets for the crop
            for year, widgets_dict in self.detailed_variable_costs_widgets["establishment"][crop].items():
                print(f"Detailed Variable Costs for {crop} (Establishment, Year {year})")
                for component, widget in widgets_dict.items():
                    display(widget)
            
            for year, widgets_dict in self.detailed_fixed_costs_widgets["establishment"][crop].items():
                print(f"Detailed Fixed Costs for {crop} (Establishment, Year {year})")
                for component, widget in widgets_dict.items():
                    display(widget)

            print(f"Detailed Variable Costs for {crop} (Production)")
            for widget in self.detailed_variable_costs_widgets["production"][crop].values():
                display(widget)

            print(f"Detailed Fixed Costs for {crop} (Production)")
            for widget in self.detailed_fixed_costs_widgets["production"][crop].values():
                display(widget)
    
            print(f"Rest of crop economics for {crop}")
            for year in establishment_years:
                display(self.conventionalcropyield_widgets["establishment"][crop][year])
            display(self.conventionalcropyield_widgets["production"][crop])
            display(price_widget, capex_widget)
            
    
    def derived_variables(self):
        self.conventionalarea = self.conventionalareapower * self.conventionalPVpower
        print(f"Total Site Area: {self.conventionalarea} acres")
        self.conventionalarraylength = self.conventionalpanellength 
        print(f"Array Length: {self.conventionalarraylength} ft")
        self.conventionalarraywidth = self.conventionalpanelwidth * self.conventionalPVpanelarray
        print(f"Array Width: {self.conventionalarraywidth} ft")

    def conventional_capex_opex(self):
        # CAPEX and OPEX are to be computed via internal formula (values modified in capex_opex class)
        
        # Use the best trendline from the CapexOpex class
        best_trendline, coefficients, _ = self.capex_opex.find_best_trendline()
            
        # Calculate CAPEX based on the selected trendline
        if best_trendline == "Linear":
            slope, intercept = coefficients
            self.conventionalPVcapex = (slope * (self.conventionalPVpower*1e-3) + intercept) * (self.conventionalPVpower * 1e3)
        elif best_trendline == "Logarithmic":
            slope, intercept = coefficients
            self.conventionalPVcapex = (slope * np.log(self.conventionalPVpower*1e-3) + intercept) * (self.conventionalPVpower * 1e3)
        elif best_trendline == "Power":
            slope, intercept = coefficients  # intercept is already exponentiated in the best_trendline method
            self.conventionalPVcapex = (intercept * ((self.conventionalPVpower*1e-3) ** slope)) * (self.conventionalPVpower * 1e3)

        print(f"PV Capex from best trendline ({best_trendline}): {self.conventionalPVcapex} $")
        
        # Calculate OPEX based on the unit OPEX from the capex_opex class
        self.conventionalPVopex = self.unitopex_gmpv*self.conventionalPVpower
        print(f"PV Opex from internal formula: {self.conventionalPVopex} $/yr")

    def confirm_selection(self, b):
        """ Store the values from the widgets and fetch defaults if necessary. """
        with self.output:
            self.output.clear_output()
            # 1- Conventional PV parameters
            self.conventionalPVpower = self.conventionalPVpower_widget.value 
            self.conventionalPVpanelpower = self.conventionalPVpanelpower_widget.value*1e-3
            self.conventionalPVdegrate = self.conventionalPVdegrate_widget.value/100
            self.conventionalareapower = self.conventionalareapower_widget.value
            self.conventionalPVrowspacing = self.conventionalPVrowspacing_widget.value
            self.conventionalPVgroundclearance = self.conventionalPVgroundclearance_widget.value
            self.conventionalpanellength = self.conventionalpanellength_widget.value
            self.conventionalpanelwidth = self.conventionalpanelwidth_widget.value 
            self.conventionalPVpanelarray = self.conventionalPVpanelarray_widget.value
            self.conventionalPVyield = self.conventionalPVyield_widget.value
            self.conventionalPVprice = self.conventionalPVprice_widget.value

            
            print(f"PV Business Total Power: {self.conventionalPVpower} kW")
            print(f"Panel Nameplate Power Capacity: {self.conventionalPVpanelpower*1000} Wdc")
            print(f"Degradation Rate: {self.conventionalPVdegrate*100}%")
            print(f"Total Area/Power: {self.conventionalareapower} acre/kW")
            print(f"Row Spacing: {self.conventionalPVrowspacing} ft")
            print(f"Ground Clearance: {self.conventionalPVgroundclearance} ft")
            print(f"Lenght Panel: {self.conventionalpanellength} ft")
            print(f"Width Panel: {self.conventionalpanelwidth} ft")
            print(f"Number of Panels per Array: {self.conventionalPVpanelarray}")
            print(f"PV Energy Yield: {self.conventionalPVyield} kWh/kWp")
            print(f"PV Electricity Selling Price (PPA): {self.conventionalPVprice} $/kWh")

            # Calculate derived variables   
            self.derived_variables()

            # CAPEX and OPEX are to be computed via internal formula (values modified in capex_opex class)
            self.conventional_capex_opex()

            # 2- Conventional Crop Parameters 
            for crop in self.cropschedule.selected_crops:
            
                # 2a- Handle Establishment Costs 
                cropschedule_outputs = self.cropschedule.outputs()
                establishment_years = cropschedule_outputs.get('crop_years', {}).get(crop, {}).get("establishment_years", [])

                for year in establishment_years:
                    # Detailled Variable Costs (retrieve + store)
                    detailed_var_est_costs = self.detailed_variable_costs_widgets["establishment"][crop][year]
                    self.detailed_variable_costs.setdefault("establishment", {}).setdefault(crop, {}).setdefault(year, {}).update(
                        {component: widget.value for component, widget in detailed_var_est_costs.items()}
                    )

                    # Detailed Fixed Costs
                    detailed_fix_est_costs = self.detailed_fixed_costs_widgets["establishment"][crop][year]
                    self.detailed_fixed_costs.setdefault("establishment", {}).setdefault(crop, {}).setdefault(year, {}).update(
                        {component: widget.value for component, widget in detailed_fix_est_costs.items()}
                    )   


                # 2b- Handle Production Costs
                # For variable costs (same every year during production)
                detailed_var_prod_costs = self.detailed_variable_costs_widgets["production"][crop]
                for component, widget in detailed_var_prod_costs.items():
                    value = widget.value
                    self.detailed_variable_costs.setdefault("production", {}).setdefault(crop, {})[component] = widget.value

                detailed_fix_prod_costs = self.detailed_fixed_costs_widgets["production"][crop]
                for component, widget in detailed_fix_prod_costs.items():
                    value = widget.value
                    self.detailed_fixed_costs.setdefault("production", {}).setdefault(crop, {})[component] = widget.value

                # 2c- Handle Crop Yield 
                # Store yield for establishment years
                self.conventionalcropyield.setdefault("establishment", {}).setdefault(crop, {})
                for year in establishment_years:
                    yield_value = self.conventionalcropyield_widgets["establishment"][crop][year].value
                    self.conventionalcropyield["establishment"][crop][year] = yield_value
                    print(f"Stored Yield for {crop}, Year {year}: {yield_value} t/acre")

                # Store yield for production years
                self.conventionalcropyield.setdefault("production", {}).setdefault(crop, {})
                yield_value = self.conventionalcropyield_widgets["production"][crop].value
                self.conventionalcropyield["production"][crop] = yield_value
                print(f"Stored Yield for {crop}, Production Years: {yield_value} t/acre")

        
                # 2d- Handle crop price
                price = self.conventionalcropprice_widgets[crop].value or \
                        self.crops_df.loc[self.crops_df['State'] == self.location.state, crop].values[0][1]
                self.conventionalcropprice[crop] = price 
                print(f"Price for {crop}: {price} $/t")
            
                # 2e- Handle crop capex
                capex = self.conventionalcropcapex_widgets[crop].value
                self.conventionalcropcapex[crop] = capex 
                print(f"Capex for {crop}: {capex} $/acre")

            print("All values confirmed.")
    
    def get_conventional_crop_yield(self, t):
        """
        Retrieve yield values for each crop for year t.
        """
        cropschedule_outputs = self.cropschedule.outputs()
        crop_years = cropschedule_outputs.get('crop_years', {})
            
        establishment_crops = [
            crop for crop, years in crop_years.items() if t in years.get("establishment_years", [])
        ]
        production_crops = [
            crop for crop, years in crop_years.items() if t in years.get("production_years", [])
        ]

        yields = {}
        for crop in establishment_crops:
            yields[crop] = self.conventionalcropyield["establishment"][crop].get(t, 0)  # Default to 0 if not found

        for crop in production_crops:
            yields[crop] = self.conventionalcropyield["production"][crop]  # Production yield is the same for all years

        return yields
    
    def get_conventional_crop_revenues(self, t):
        """
        Calculate revenues for crops for a given year t.

        This method dynamically handles both establishment and production years by relying on
        the `get_conventional_crop_yield` method, which already accounts for the correct yield.
        """
        cropschedule_outputs = self.cropschedule.outputs()
        crop_years = cropschedule_outputs.get('crop_years', {}) 

        # Retrieve dynamic crop yields for the given year
        crop_yields = self.get_conventional_crop_yield(t)
    
        # Calculate total revenues
        if t > 0:
            total_revenues = sum(
                crop_yields.get(crop, 0) * self.conventionalcropprice.get(crop, 0) * self.conventionalarea *
                self.cropschedule.crop_area_percentages.get(crop, 1.0)
                for crop in crop_years
            )
        else:
            total_revenues = 0
        return total_revenues

    # Detailed Variable and Fixed Costs in $/yr and $/yr
    
    def get_conventional_crop_detailed_variable_costs(self, t):
        """
        Calculate detailed variable costs for each crops for year t, handling either establishment or production.

        Parameters:
        - t: Year for which the costs are calculated.

        Returns:
        - Dictionary where each crop has a sub-dictionary of detailed variable costs per component for year t.
        """
        cropschedule_outputs = self.cropschedule.outputs()
        crop_years = cropschedule_outputs.get('crop_years', {})

        # Determine phase based on year
        establishment_crops = [
            crop for crop, years in crop_years.items() if t in years.get("establishment_years", [])
        ]
        production_crops = [
            crop for crop, years in crop_years.items() if t in years.get("production_years", [])
        ]

        # Fetch detailed costs for the relevant phase
        if establishment_crops:
            phase = "establishment"
            relevant_crops = establishment_crops
        elif production_crops:
            phase = "production"
            relevant_crops = production_crops
        else:
            return {}  # No costs if the year is not part of establishment or production

        detailed_costs = self.detailed_variable_costs[phase]

        result = {}
        for crop in relevant_crops:
            if phase == "production":
                # Production costs might not be year-specific
                crop_costs = detailed_costs.get(crop, {})  # Retrieve costs directly
                if not isinstance(crop_costs, dict) or not crop_costs:  # Handle empty or invalid entries
                    print(f"DEBUG: No valid production costs found for crop {crop} in Year {t}")
                    continue
            else:
                # Establishment costs are year-specific
                crop_costs = detailed_costs.get(crop, {}).get(t, {})

            # Calculate component costs
            if t>0:
                result[crop] = {
                    component: cost * self.conventionalarea * self.cropschedule.crop_area_percentages.get(crop, 1.0)
                    for component, cost in crop_costs.items()
                }
            else:
                result[crop] = {
                    component: 0 * self.conventionalarea * self.cropschedule.crop_area_percentages.get(crop, 1.0)
                    for component, cost in crop_costs.items()
                }

        return result
    
    def get_conventional_crop_detailed_fixed_costs(self, t):
        """
        Calculate detailed fixed costs for each crops for year t, handling either establishment or production.

        Parameters:
        - t: Year for which the costs are calculated.

        Returns:
        - Dictionary where each crop has a sub-dictionary of detailed fixed costs per component for year t.
        """
        cropschedule_outputs = self.cropschedule.outputs()
        crop_years = cropschedule_outputs.get('crop_years', {})

        # Determine phase based on year
        establishment_crops = [
            crop for crop, years in crop_years.items() if t in years.get("establishment_years", [])
        ]
        production_crops = [
            crop for crop, years in crop_years.items() if t in years.get("production_years", [])
        ]

        # Fetch detailed costs for the relevant phase
        if establishment_crops:
            phase = "establishment"
            relevant_crops = establishment_crops
        elif production_crops:
            phase = "production"
            relevant_crops = production_crops
        else:
            return {}  # No costs if the year is not part of establishment or production

        detailed_costs = self.detailed_fixed_costs[phase]

        result = {}
        for crop in relevant_crops:
            if phase == "production":
                crop_costs = detailed_costs.get(crop, {}) 
                if not isinstance(crop_costs, dict) or not crop_costs: 
                    print(f"DEBUG: No valid production costs found for crop {crop} in Year {t}")
                    continue
            else:
                crop_costs = detailed_costs.get(crop, {}).get(t, {})
            
            if t>0:
                result[crop] = {
                    component: cost * self.conventionalarea * self.cropschedule.crop_area_percentages.get(crop, 1.0)
                    for component, cost in crop_costs.items()
                }
            else:
                result[crop] = {
                    component: 0 * self.conventionalarea * self.cropschedule.crop_area_percentages.get(crop, 1.0)
                    for component, cost in crop_costs.items()
                }

        return result


    # Total Variable or Fixed Costs in $/yr

    def get_conventional_crop_total_variable_costs(self, t):
        """
        Calculate total variable costs for all crops for year t.

        Parameters:
        - t: Year for which the costs are calculated.

        Returns:
        - Total variable costs ($/yr) for all crops for year t.
        """
        detailed_variable_costs = self.get_conventional_crop_detailed_variable_costs(t)
        return sum(
            sum(component_costs.values()) for component_costs in detailed_variable_costs.values()
        )
    
    def get_conventional_crop_total_fixed_costs(self, t):
        """
        Calculate total fixed costs for all crops for year t.

        Parameters:
        - t: Year for which the costs are calculated.

        Returns:
        - Total fixed costs ($/yr) for all crops for year t.
        """
        detailed_fixed_costs = self.get_conventional_crop_detailed_fixed_costs(t)
        return sum(
            sum(component_costs.values()) for component_costs in detailed_fixed_costs.values()
        )
    
    # Total Costs  in $/yr
    
    def get_conventional_crop_total_costs(self, t):
        """
        Calculate total costs (variable + fixed) for all crops for year t.

        Parameters:
        - t: Year for which the costs are calculated.

        Returns:
        - Total costs ($/yr) for all crops for year t.
        """
        total_variable_costs = self.get_conventional_crop_total_variable_costs(t)
        total_fixed_costs = self.get_conventional_crop_total_fixed_costs(t)
        return total_variable_costs + total_fixed_costs
    
   
    def get_conventional_crop_capex_individual(self):
        """ Calculate total capex for all selected crops in $ 
            Return a dictionary with crop names as keys and unit capex values multiply by each crop area as values.
        """
        total_capex = {
            crop: self.conventionalcropcapex.get(crop, 0) * self.conventionalarea * self.cropschedule.crop_area_percentages.get(crop, 1.0)
            for crop in self.cropschedule.selected_crops
        }

        #print(f"Debug: Total Crop Capex: {total_capex} $")

        return total_capex

    def get_conventional_crop_capex_total(self):
        """Calculate and return the total CAPEX for all selected crops in $."""
        return sum(self.get_conventional_crop_capex_individual().values())


    def get_conventional_energy_prod(self, t):
        """
        Calculates the energy production (kWh) for year t, considering annual degradation.
        """
        energy_prod_0 = self.conventionalPVpower * self.conventionalPVyield
        # Apply degradation for each year
        if t < 2:
            return energy_prod_0 
        else:
            degraded_energy_prod = energy_prod_0 * ((1 - self.conventionalPVdegrate) ** (t - 1))
            return degraded_energy_prod
    
    def get_conventional_escalated_energy_price(self, t):
        """
        Calculates the escalated energy price for year t based on the escalation rate.
        """
        base_energy_price = self.conventionalPVprice
        escalation_rate = self.financial_rates.energy_price_escalation_rate
        if t < 2:
            escalated_price = base_energy_price
        else:
            escalated_price = base_energy_price * ((1 + escalation_rate) ** (t-1))
        return escalated_price

    def get_conventional_pbi_revenues(self, t):
        # Add code to calculate revenues from the PBI incentive in $ for the APV system
        # Apply PBI only if t >= 1
        pbi = self.financial_rates.pbi if t >= 1 else 0
        pbi_revenues = pbi * self.get_conventional_energy_prod(t)
        return pbi_revenues

    def get_conventional_srec_revenues(self, t):
        # Add code to calculate revenues from the SREC incentive in $ for the APV system
        # Apply SREC only if t >= 1 and t <= SREC duration
        srec = self.financial_rates.srec if t >= 1 and t <= self.financial_rates.srec_duration else 0
        srec_revenues = srec * self.get_conventional_energy_prod(t)
        return srec_revenues
    
    def get_conventional_base_PV_revenues(self, t):
        # Add code to calculate base revenues with PPA from the PV system in $
        baseenergyprice = self.get_conventional_escalated_energy_price(t) if t >= 1 else 0
        basePVrevenues = baseenergyprice*self.get_conventional_energy_prod(t)
        return basePVrevenues

    def get_conventional_PV_revenues(self, t):
        """ Calculate conventional PV total revenues in $ """
        return self.get_conventional_base_PV_revenues(t) + self.get_conventional_pbi_revenues(t) + self.get_conventional_srec_revenues(t)
    
    def outputs(self):
        """
        Returns a detailed breakdown of economic metrics for the conventional system.
        Includes separate revenues, costs for establishment and production years.
        """

        # Calculate revenues for each year
        revenues = {
            f"Year {t}": self.get_conventional_crop_revenues(t)
            for t in range(self.lifetime.years + 1)
        }

        # Calculate costs for each year

        costs = {
            f"Year {t}": {
                "Establishment": self.get_conventional_crop_total_variable_costs(t) + self.get_conventional_crop_total_fixed_costs(t) if self.cropschedule.outputs().get('crop_years', {}).get("establishment_years", []) else 0,
                "Production": self.get_conventional_crop_total_variable_costs(t) + self.get_conventional_crop_total_fixed_costs(t) if self.cropschedule.outputs().get('crop_years', {}).get("production_years", []) else 0,
                "Total": self.get_conventional_crop_total_costs(t),
            }
            for t in range(self.lifetime.years + 1)
        }

        # Calculate detailed costs for each crop and year
        detailed_costs = {
            crop: {
                f"Year {t}": {
                    "Variable Costs ($)": self.get_conventional_crop_detailed_variable_costs(t).get(crop, {}),
                    "Fixed Costs ($)": self.get_conventional_crop_detailed_fixed_costs(t).get(crop, {}),
                    "Total Costs ($)": sum(self.get_conventional_crop_detailed_variable_costs(t).get(crop, {}).values()) +
                                    sum(self.get_conventional_crop_detailed_fixed_costs(t).get(crop, {}).values()),
                }
                for t in range(self.lifetime.years + 1)
            }
            for crop in self.cropschedule.selected_crops
        }

        # Return full breakdown
        return {
            "Panel Power STC (Wdc)": self.conventionalPVpanelpower * 1000,
            "Power (kW)": self.conventionalPVpower,
            "Degradation Rate (%)": self.conventionalPVdegrate,
            "Area/Power (acre/MW)": self.conventionalareapower,
            "Area project (total) (acres)": self.conventionalarea,
            "Rowspacing (ft)": self.conventionalPVrowspacing,
            "Ground Clearance (ft)": self.conventionalPVgroundclearance,
            "Panel Length (ft)": self.conventionalpanellength,
            "Panel Width (ft)": self.conventionalpanelwidth,
            "Number of Panels per Array": self.conventionalPVpanelarray,
            "Array Length (ft)": self.conventionalarraylength,
            "Array Width (ft)": self.conventionalarraywidth,
            "Revenues ($/yr)": revenues,
            "Costs ($/yr)": costs,
            "Detailed Costs ($)": detailed_costs,
            "Total Crop Capex ($)": self.get_conventional_crop_capex_total(),
            "PV Capex ($)": self.conventionalPVcapex,
            "PV Opex ($/yr)": self.conventionalPVopex,
            "PV Energy Yield (kWh/yr)": {
                f"Year {t}": self.get_conventional_energy_prod(t) for t in range(self.lifetime.years + 1)
            },
            "PV Revenues ($/yr)": {
                f"Year {t}": self.get_conventional_PV_revenues(t) for t in range(self.lifetime.years + 1)
            },
        }




## 7 - Designing the APV systems (both PV and farm businesses)

### 7.1 - Designing the PV business of the APV system (like rowspacing changes...)

In [ ]:
class PVDesign:
    """
    Class to design the PV business of the APV system (fewer input compared to 6 to control the design changes in PV business between conventional and APV systems).

    Requires:
    - FinancialRates (from the FinancialRates class)
    - Lifetime (from the Lifetime class)
    - ConventionalEconomics (from the ConventionalEconomics class)
    - Unit CAPEX and OPEX for APV system(from the CapexOpex class)

    Returns:
    - Energy Yield (kWh/kWp) (normally different from the conventional system but here we assume the same yield since no model to determine energy yield under APV system)
    - Vegetation Maintenance Costs ($/acre) 
    - Electricity selling price (here PPA) ($/kWh) (again could be different from the conventional system but here we assume the same price)
    - Ground Clearance Change (ft) (to be used to adjust the ground clearance of the APV system)
    - Row Spacing Change (ft) (to be used to adjust the row spacing of the APV system)

    Computes and returns:
    - New PV business specifications if the user changes the ground clearance or row spacing (indeed if APV row spacing is different from the conventional one, it induces changes on power produced over a fixed area and all the derived variables)
        - New Area used per array 
        - New number of arrays
        - New number of panels
        - New power produced by the PV business in the APV system
        - New CAPEX and OPEX based on the new PV business specifications
    - APV annual energy production (kWh/yr)
    - Number of operating hours per years (hrs/yr) - just useful to compute the capacity factor later
    - Area of the APV system used by the farm business (acres)
    """
    
    def __init__(self, financial_rates, lifetime ,conventional_economics, capex_opex_agrivoltaics):
        self.output = Output()
        # Initialize all parameters to None until the user inputs values
        self.name = None
        self.power = None
        self.panelpower = None
        self.systemtype = None
        self.rowspacing = None
        self.ground_clearance = None 
        self.extra_capex = None
        self.gcr = None
        self.apv_areapower = None
        self.array_width = None
        self.array_length = None
        self.panel_array = None 
        self.energy_yield = None
        self.capacityfactor = None  #not entered as input but calculated internally from energy_yield
        self.capex = None
        self.opex = None
        self.maintenance_costs = None
        self.energyprice = None
        self.financial_rates = financial_rates
        self.lifetime = lifetime
        self.conventional_economics = conventional_economics
        self.capex_opex = capex_opex_agrivoltaics
        # Internall Computed values
        self.arrayarea = None
        self.totalpanels = None
        self.totalarrays = None
        self.areaused = None
        self.totalarea = None
        self.arearatio = None
        
        # PV Design Changes (normally used after at least one test if NPV<0) for ground clearance, GCR, and inter-row spacing
        self.ground_clearance_change = 0
        self.row_spacing_change = 0
        # Constants
        self.unitopex_apvpv = self.capex_opex.compute_total_unit_opex() # $/kWdc without land costs which are accounted after regadring the case and w/o cleaning costs since management of vegetation by farmer
        self.conv_ft2_acre = 1/43560 #1ft2 to acres
        
        
        # Create widgets for all inputs
        self.name_input = widgets.Text(
            description='Name:',
            placeholder='Enter system name',
            style={'description_width': 'initial'}
        )

        self.energy_yield_input = widgets.FloatText(
            description="Energy Yield (kWh/kWp):",
            value=0,
            style={'description_width': 'initial'}
        )

        self.maintenance_costs_widget = widgets.FloatText( 
            description="Vegetation Maintenance Costs ($/acre):",
            value=0,
            style={'description_width': 'initial'}
        )

        self.energyprice_input = widgets.FloatText(
            description="Electricity selling price (PPA) ($/kWh):",
            value=0,
            style={'description_width': 'initial'}
        )

        self.ground_clearance_change_input = widgets.FloatText(
            description="Ground Clearance Change (ft):",
            value=0,
            style={'description_width': 'initial'}
        )
        
        self.row_spacing_change_input = widgets.FloatText(
            description="Inter-row Spacing Change (ft):",
            value=0,
            style={'description_width': 'initial'}
        )

        # Button to confirm selections
        self.confirm_button = widgets.Button(description="Confirm Selection")
        self.confirm_button.on_click(self.confirm_selection)

        # Display the widgets
        display(self.name_input, #self.systemtype_dropdown,
                self.energy_yield_input, self.maintenance_costs_widget,
                self.energyprice_input, self.ground_clearance_change_input, self.row_spacing_change_input, self.confirm_button, self.output)
    
    


    def derived_variables(self, rowspacing):
        """ Compute all derived variables based on the user inputs 
            NB1: when power is to be changed input power as self.power or self.pv_design.power
            but when power is to be fixed input power as self.conventional_economics.conventionalPVpower
            NB2: We have to first compute area used by the PV system and total area available for APV system from the conventionaltional row spacing
            and then we can adjust the row spacing based on the user input for a fixed total and used area in the method adjust_row_spacing

        """
        self.capacityfactor = self.energy_yield / (365*24)
        self.ground_clearance = self.conventional_economics.conventionalPVgroundclearance + self.ground_clearance_change
        # When SA on powers, we need to use directly the rowspacing with the changes included 
        # In all other cases (SA on rwosapcing or normal use of model) we need to use the rowspacing without the changes 
        # and then add the chages and adjust all variables and power with the method adjust_row_spacing 
        rowspacing_intermediate = rowspacing
        print(f"Intermediate row spacing: {rowspacing_intermediate:.2f} ft.")

        # Calculate the area used by the PV system and total area available for APV system from the intermediate row spacing 
        
        self.arrayarea = self.array_width * rowspacing * self.conv_ft2_acre #in acres
        self.totalpanels = self.power/self.panelpower
        self.totalarrays = self.totalpanels/self.panel_array
        self.areaused = self.arrayarea * self.totalarrays #in acres
        self.totalarea = self.power * self.conventional_economics.conventionalareapower
        self.arearatio = self.areaused/self.totalarea

        # Now the used and total area are computed we can adjust the row spacing based on the user input
        self.rowspacing = self.conventional_economics.conventionalPVrowspacing + self.row_spacing_change

        # Calculate GCR once we have the rowspacing with the changes
        self.gcr = self.array_length/self.rowspacing
            

    def adjust_row_spacing(self):   
        """ Adjust row spacing based on the user input for a fixed total and used area and recompute array area, total arrays, total panels, and power
            Thus never use this module when changing the power in your SA because SA on power induces change in total and used area 
            so not compatible with this module which requires fixed areas.
        """
        # Adjust row spacing
        if self.row_spacing_change != 0:
            #self.rowspacing += self.row_spacing_change
            print(f"Updated row spacing to {self.rowspacing:.2f} ft compared to conventional {self.conventional_economics.conventionalPVrowspacing} ft.")
            
            # Calculate all values impacted by rowspacing changes 
            # Array area
            self.arrayarea = self.array_width * self.rowspacing * self.conv_ft2_acre
            # Total Arrays based on the fixed area used that's why we can't use this module when doing SA on power
            self.totalarrays = self.areaused/self.arrayarea
            # Total Panels
            self.totalpanels = self.totalarrays*self.panel_array
            # Power 
            self.power = self.totalpanels * self.panelpower
            print(f"New power calculated as: {self.power:.2f} kW based on row spacing changes.")
            # Get the new area power density based on the fixed total area and new power (again that's why we can't use this module when doing SA on power because total area would not be fixed)
            self.apv_areapower = self.totalarea/self.power
            

    def get_capex_opex(self):
        # Calculate CAPEX 
        # Use CapexOpex class to determine CAPEX
        best_trendline, coefficients, _ = self.capex_opex.find_best_trendline()
            
        # Calculate CAPEX based on the selected trendline
        if best_trendline == "Linear":
            slope, intercept = coefficients
            self.capex = (slope * (self.power * 1e-3) + intercept) * (self.power * 1e3)
        elif best_trendline == "Logarithmic":
            slope, intercept = coefficients
            self.capex = (slope * np.log(self.power * 1e-3) + intercept) * (self.power * 1e3)
        elif best_trendline == "Power":
            slope, intercept = coefficients  # intercepct is already exponentiated in the best_trendline method
            self.capex = (intercept * ((self.power * 1e-3) ** slope)) * (self.power * 1e3)
            
        print(f"CAPEX calculated using {best_trendline} trendline: ${self.capex:,.2f}")


        # Adjust CAPEX based on ground clearance change
        if self.ground_clearance_change != 0:
            print(f"Updated ground clearance to {self.ground_clearance} ft compared to conventional {self.conventional_economics.conventionalPVgroundclearance} ft")
            STEELPRICE = 0.51  # Example steel price in $/lb
            self.extra_capex = STEELPRICE * (0.0116 * self.ground_clearance_change) * (self.power * 1e3) #only the extra capex, not the full capex recompute
            original_capex = self.capex
            self.capex += self.extra_capex

            if self.extra_capex > 0:
                print(f"Original CAPEX: ${original_capex:,.2f}. Increased by ${self.extra_capex:,.2f} due to ground clearance change.")
            else:
                print(f"Original CAPEX: ${original_capex:,.2f}. Decreased by ${self.extra_capex:,.2f} due to ground clearance change.")

            print(f"New CAPEX: ${self.capex:,.2f}")

        # Compute OPEX based on the unit OPEX and power
        
        self.opex = self.unitopex_apvpv*self.power 


    def confirm_selection(self, b):
        """ Get the selected values from the widgets and store them as class attributes """
        with self.output:
            self.output.clear_output()
            self.name = self.name_input.value
            self.power = self.conventional_economics.conventionalPVpower  #at first take same power than the one entered in conventional system, then if adjustements needed --> only done on the power of APV case not conventional one (cf. changes below)
            self.panelpower = self.conventional_economics.conventionalPVpanelpower
            self.systemtype = self.capex_opex.module_type_dropdown.value #self.systemtype_dropdown.value 
            self.array_width = self.conventional_economics.conventionalarraywidth
            self.array_length = self.conventional_economics.conventionalarraylength
            self.panel_array = self.conventional_economics.conventionalPVpanelarray
            self.energy_yield = self.energy_yield_input.value
            self.apv_areapower = self.conventional_economics.conventionalareapower #in acres/kW
            self.maintenance_costs = self.maintenance_costs_widget.value 
            self.energyprice = self.energyprice_input.value
            self.ground_clearance_change = self.ground_clearance_change_input.value
            self.row_spacing_change = self.row_spacing_change_input.value
            

            # Compute all derived variables based on the user inputs
            self.derived_variables(self.conventional_economics.conventionalPVrowspacing) # Compute all derived variables based on the user inputs
            # Adjust row spacing derived variables if the row spacing change is not 0
            self.adjust_row_spacing() # Adjust row spacing based on the user input
            # Compute CAPEX and OPEX based on the user inputs and after adjusting the row spacing if needed
            self.get_capex_opex() # Compute CAPEX and OPEX based on the user inputs    

            # Debugging: Display the selected values
            print(f"Selected Values: \nName: {self.name}\nPower: {self.power} kW\nPanel Power: {self.panelpower}kWdc\n"
                f"System Type: {self.systemtype}\nRow Spacing: {self.rowspacing} ft\nArray Width: {self.array_width} ft\n"
                f"Area per Array: {self.arrayarea} acres\nArea Used: {self.areaused} acres\nArea Power Density: {self.apv_areapower} acres/kW\nTotal Area: {self.totalarea} acres\n"
                f"Ground Clearance: {self.ground_clearance} ft\nArray Length: {self.array_length} ft\nGCR: {self.gcr*100} %\n"
                f"Number of Arrays: {self.totalarrays}\nPanel per Array: {self.panel_array}\nNumber of Panels: {self.totalpanels}\n" 
                f"Energy Yield: {self.energy_yield} kWh/kWp\n"
                f"Capacity Factor: {self.capacityfactor*100} %\nCAPEX: ${self.capex}\nOPEX w/o land costs: ${self.opex}/year\nMaintenance Costs: ${self.maintenance_costs}/acre\n"
                f"Electricity Selling Price (PPA): ${self.energyprice}/kWh")
                
    def update_pv_design(self):
        """
        Recompute all dependent variables when rowspacing or other parameters change.
        Should be called after modifying rowspacing in the sensitivity analysis.
        """
        # Update power based on new rowspacing
        if self.rowspacing is not None:
            
            # Calculate all values impacted by rowspacing changes 
            # Array area
            self.arrayarea = self.array_width * self.rowspacing * self.conv_ft2_acre
            # Total Arrays
            self.totalarrays = self.areaused/self.arrayarea
            # Total Panels
            self.totalpanels = self.totalarrays*self.panel_array
            # Power 
            self.power = self.totalpanels * self.panelpower
        else:
            print("Error: Row spacing not defined.")
            return

        # Calculate GCR
        self.gcr = self.array_length/self.rowspacing  

        # Recalculate CAPEX if rowspacing influences power
        best_trendline, coefficients, _ = self.capex_opex.find_best_trendline()
        if best_trendline == "Linear":
            slope, intercept = coefficients
            self.capex = (slope * (self.power * 1e-3) + intercept) * (self.power * 1e3)
        elif best_trendline == "Logarithmic":
            slope, intercept = coefficients
            self.capex = (slope * np.log(self.power * 1e-3) + intercept) * (self.power * 1e3)
        elif best_trendline == "Power":
            slope, intercept = coefficients
            self.capex = (intercept * ((self.power * 1e-3) ** slope)) * (self.power * 1e3)

        print(f"Updated CAPEX using {best_trendline} trendline: ${self.capex:,.2f}")


        # Readjust CAPEX based on ground clearance change
        if self.ground_clearance_change != 0:
            self.ground_clearance += self.ground_clearance_change
            STEELPRICE = 0.51  # Example steel price in $/lb
            extra_capex = STEELPRICE * (0.0116 * self.ground_clearance_change) * (self.power * 1e3) #only the extra capex, not the full capex recompute
            original_capex = self.capex
            self.capex += extra_capex

            if extra_capex > 0:
                print(f"Original CAPEX: ${original_capex:,.2f}. Increased by ${extra_capex:,.2f} due to ground clearance change.")
            else:
                print(f"Original CAPEX: ${original_capex:,.2f}. Decreased by ${extra_capex:,.2f} due to ground clearance change.")
        
        # Compute OPEX w/o land costs (added later in the financial tool)
        self.opex = self.unitopex_apvpv*self.power
        
        # Print the updated values for verification especially after the sensitivity is done and rowspacing value reset to the original value
        print(f"Updated(Restored) Rowspacing: {self.rowspacing:.2f} ft")
        print(f"Updated(Restored) Power: {self.power:.2f} kW")
        print(f"Updated(Restored) Number Panels: {self.totalpanels:.2f}")
        print(f"Updated(Restored) Number Arrays: {self.totalarrays:.2f}")
        print(f"Updated(Restored) Area per Array: {self.arrayarea:.2f} acres")
        print(f"Updated(Restored) GCR: {self.gcr:.2f}")
        print(f"Updated(Restored) CAPEX: ${self.capex:.2f}")
        print(f"Updated(Restored) OPEX w/o land costs: ${self.opex:.2f}/yr")


    def get_energy_prod(self, t):
        """
        Calculates the energy production (kWh) for year t, considering annual degradation.
        
        Parameters:
        - t (int): The year for which energy production is being calculated.
        
        Returns:
        - energy_prod (float): The energy production for year t.
        """
        if self.power is not None and self.energy_yield is not None:
            energy_prod_0 = self.power * self.energy_yield
            # Apply degradation for each year
            if t < 2:
                return energy_prod_0 
            else:
                degraded_energy_prod = energy_prod_0 * ((1 - self.conventional_economics.conventionalPVdegrate) ** (t - 1))
                return degraded_energy_prod

    def get_operationhours(self):
        """ Calculate the total operation hours per year """
        if self.capacityfactor is not None:
            return self.capacityfactor * 365 * 24
        return None
    
    def get_area_farm(self):
        """ Calculate the area of the APV system used by the farm business (acres) """
        area_pv = self.array_length * self.array_width * self.totalarrays * 0.0000229568 #in acre so 0.0000229568 is the conversion factor from ft2 to acre
        area_farm = self.totalarea - area_pv
        return area_farm

    def outputs(self):
        """
        Returns a detailed breakdown of the PV design for the APV system.
        
        Returns:
        - Dictionary containing all relevant parameters and computed values.
        """
        return {
            "Name": self.name,
            "Power (kW)": self.power,
            "Panel Power (kWdc)": self.panelpower,
            "System Type": self.systemtype,
            "Row Spacing (ft)": self.rowspacing,
            "Ground Clearance (ft)": self.ground_clearance,
            "Array Width (ft)": self.array_width,
            "Array Length (ft)": self.array_length,
            "Area per Array (acres)": self.arrayarea,
            "Area Used by PV System (acres)": self.areaused,
            "Total Area Available for APV System (acres)": self.totalarea,
            "Area Power Density (acres/kW)": self.apv_areapower,
            "GCR (%)": self.gcr * 100,  # Convert to percentage
            "Number of Arrays": self.totalarrays,
            "Panels per Array": self.panel_array,
            "Total Panels": self.totalpanels,
            "Energy Yield (kWh/kWp)": self.energy_yield,
            "Capacity Factor (%)": self.capacityfactor * 100 if self.capacityfactor is not None else None,  # Convert to percentage
            "CAPEX ($)": self.capex,
            "OPEX w/o land costs ($/yr)": self.opex,
            "Vegetation Maintenance Costs ($/acre)": self.maintenance_costs,
            "Energy Price ($/kWh)": self.energyprice,
            "Annual Energy Production (kWh/yr)": {f"Year {t}": self.get_energy_prod(t) for t in range(self.lifetime.years + 1)},
            "Operation Hours per Year (hrs/yr)": {f"Year {t}": self.get_operationhours() for t in range(self.lifetime.years + 1)},
            "Area of APV System used by Farm Business (acres)": {f"Year {t}": self.get_area_farm() for t in range(self.lifetime.years + 1)}
        }

### 7.2 - Designing the farm business of the APV system (like crop yield changes...)

In [ ]:
class CropYield:
    """
    Class to design the farm business of the APV system (like crop yields in APV system).
    
    Requires:
    - FinancialRates (from the FinancialRates class)
    - ConventionalEconomics (from the ConventionalEconomics class)
    - CropSchedule (from the CropSchedule class)


    Returns:
    - crop yield in APV system (t/acre) for each selected crop (NB: this module could be replaced by a more complex crop yield model)
    """

    def __init__(self, cropschedule, financial_rates, conventional_economics):
        self.cropschedule = cropschedule 
        self.financial_rates = financial_rates
        self.conventional_economics = conventional_economics 
        self.yield_change = self.financial_rates.yield_change  # Yield change due to AV system
        self.override_yields = {}  # Add this to store temporary yield overrides

    def get_APV_crop_yield(self, crop):
        """ Calculate and return the yield for a given crop """
        # Check if there is an override value (used for the sensitivity analysis around crop yields)
        if crop in self.override_yields:
            return self.override_yields[crop]
        
        # Get the production year crop yield from ConventionalEconomics
        conventional_production_yield = self.conventional_economics.conventionalcropyield.get("production", {}).get(crop, 0)

        # Adjust yield for APV conditions using self.yield_change
        apv_yield = conventional_production_yield * self.yield_change

        return apv_yield  # Default to 0 if not found
    
    def set_temporary_yield(self, crop, value):
        """ Set a temporary yield for a crop. Used in sensitivity analysis. """
        self.override_yields[crop] = value

    def reset_yields(self):
        """ Reset temporary yield overrides. Used in sensitivity analysis. """
        self.override_yields = {}

    def outputs(self):
        """ 
        Return the yields for the selected crops.
        The number of yields returned will match the number of selected crops in the CropSchedule instance.
        """
        selected_crops = self.cropschedule.selected_crops  # Get selected crops from CropSchedule instance
        
        # Calculate yields for each selected crop
        yields = [self.get_APV_crop_yield(crop) for crop in selected_crops]
        
        return yields


## 8 - Computing the financial results

In [ ]:
class FinancialTool:
    def __init__(self, pv_design, cropschedule, crop_yield, financial_rates, partnership, conventional_economics, lifetime, system_type):
        """
        Class to perform financial analysis for PV and farm business of both conventional and APV systems.

        Requires:
        - PVDesign (from the PVDesign class)
        - CropSchedule (from the CropSchedule class)
        - CropYield (from the CropYield class)
        - FinancialRates (from the FinancialRates class)
        - Partnership (from the Partnership class)
        - ConventionalEconomics (from the ConventionalEconomics class)
        - Lifetime (from the Lifetime class)
        - system_type (str): Type of system considered ('APV PV' or 'APV farm' or 'conventional PV' or 'conventional farm')

        Computes and returns: 
        - Financial metrics for the PV and farm business of the APV and conventional systems, most importantly NPV, IRR, and LCOE.
        """
        self.pv_design = pv_design
        self.cropschedule = cropschedule
        self.crop_yield = crop_yield
        self.financial_rates = financial_rates
        self.partnership = partnership
        self.conventional_economics = conventional_economics
        self.lifetime = lifetime
        self.system_type = system_type

        # Initializing some metrics each time the class is instantiated
        self.carryover = [0] * (self.lifetime.years + 1)  # Pre-fill for all years
        self.remaining_debt = [0] * (self.lifetime.years + 1)  # Pre-fill for all years
        self.farm_costs_change = self.financial_rates.farm_costs_change #1.075  # Adjustment factor for variable costs
    
#DCF analysis and NPV calculation

    def get_discount_factors(self):
        """ Calculate discount factors to avoid recalculation """
        years = self.lifetime.years
        if self.system_type in ('APV PV', 'conventional PV'):
            self.discount_factors = [1 / (1 + self.financial_rates.wacc) ** t for t in range(years + 1)]
        elif self.system_type in ('APV farm', 'conventional farm'):
            self.discount_factors = [1 / (1 + self.financial_rates.nominal_discount_rate) ** t for t in range(years + 1)]
        return self.discount_factors

    def get_escalated_energy_price(self, t):
        """Calculates the escalated energy price for year t based on the escalation rate."""
        base_energy_price = self.pv_design.energyprice
        escalation_rate = self.financial_rates.energy_price_escalation_rate
        if t < 2:
            escalated_price = base_energy_price
        else:
            escalated_price = base_energy_price * ((1 + escalation_rate) ** (t-1))
        return escalated_price
    
    def get_pbi_revenues(self, t):
        """Calculate revenues from the PBI incentive in $ for the PV business of the APV system"""
        # Apply PBI only if t >= 1
        pbi = self.financial_rates.pbi if t >= 1 else 0
        pbi_revenues = pbi * self.pv_design.get_energy_prod(t)
        return pbi_revenues

    def get_srec_revenues(self, t):
        """Calculate revenues from the SREC incentive in $ for PV business of the APV system"""
        # Apply SREC only if t >= 1 and t <= SREC duration
        srec = self.financial_rates.srec if t >= 1 and t <= self.financial_rates.srec_duration else 0
        srec_revenues = srec * self.pv_design.get_energy_prod(t)
        return srec_revenues
    
    def get_base_PV_revenues(self, t):
        """ Calculate base revenues from the PV business of the APV system in $"""
        baseenergyprod = self.pv_design.get_energy_prod(t)
        baseenergyprice = self.get_escalated_energy_price(t) if t >= 1 else 0

        basePVrevenues = baseenergyprod * baseenergyprice

        return basePVrevenues

    def get_APV_PV_revenues(self, t):
        # Add code to calculate total revenues from the PV system in $
        APVPVrevenues = self.get_base_PV_revenues(t) + self.get_pbi_revenues(t) + self.get_srec_revenues(t)

        return APVPVrevenues
    
    def get_yearly_extra_farmer_payment(self, t):
        """
        Calculate the yearly payments in $ to the farmer for the APV system.
        t in case we need later to introduce a escalation rate for the payments.
        """
        if self.system_type in ('APV PV', 'APV farm'):
            
            if t == 0:
                yearly_extra_farmer_payment = 0
            elif t == 1:
                yearly_extra_farmer_payment = self.partnership.extra_payment_sd_to_farmer * self.pv_design.totalarea
            else:
                #inflate the payment by the inflation rate for each year starting from year 2 since year 1 is first payment and year 0 = 0
                yearly_extra_farmer_payment = (self.partnership.extra_payment_sd_to_farmer * self.pv_design.totalarea) * ((1 + self.financial_rates.inflation_rate) ** (t-1))
            
            return yearly_extra_farmer_payment
        
        else:
            
            return 0 
       
    def get_APV_crop_revenues(self, t):
        """
        Calculate APV crop revenues for each selected crop in $ for year t.
        Uses conventional yields for establishment years and APV yields for production years.

        Parameters:
        - t: Year for which to calculate revenues.

        Returns:
        - Dictionary with crop names as keys and their respective revenues as values.
        """

        cropschedule_outputs = self.cropschedule.outputs()
        crop_years = cropschedule_outputs.get('crop_years', {})  # Safely access the crop_years dictionary

        # Placeholder for reduction factor and adjustment
        percedible = 1  # Placeholder for reduction factor in case a part of production is not edible
        area = self.pv_design.get_area_farm()  # Area used for farm in APV system

        # Retrieve conventional yields for the year
        conventional_yields = self.conventional_economics.get_conventional_crop_yield(t)

        # Calculate total revenues
        if t == 0:
            # No revenues in Year 0
            total_revenues = 0
        else:
            total_revenues = sum(
                (
                    (conventional_yields[crop]  # Yield from conventional economics
                    if t in crop_years[crop].get("establishment_years", [])
                    else self.crop_yield.get_APV_crop_yield(crop))  # APV yield for production (time-independent)
                    * self.conventional_economics.conventionalcropprice[crop]
                    * percedible * area
                    * self.cropschedule.crop_area_percentages.get(crop, 1.0)
                )
                for crop in self.cropschedule.selected_crops
                if t in crop_years[crop].get("establishment_years", []) or t in crop_years[crop].get("production_years", [])
            )
        return total_revenues
    
    # Detailed Variable and Fixed Costs in $/yr and $/yr

    def get_APV_crop_detailed_variable_costs(self, t):
        """
        Calculate detailed variable costs for each crops in the APV system for year t, automatically handling establishment or production phases.

        Parameters:
        - t: Year for which the costs are calculated.

        Returns:
        - Dictionary where each crop has a sub-dictionary of detailed variable costs per component for year t.
        """
        cropschedule_outputs = self.cropschedule.outputs()
        crop_years = cropschedule_outputs.get('crop_years', {})
        area = self.pv_design.get_area_farm()  # Area used for farm in APV system

        # Determine if year t is part of establishment or production
        establishment_crops = [
            crop for crop, years in crop_years.items() if t in years.get("establishment_years", [])
        ]
        production_crops = [
            crop for crop, years in crop_years.items() if t in years.get("production_years", [])
        ]

        if establishment_crops:
            phase = "establishment"
            relevant_crops = establishment_crops
        elif production_crops:
            phase = "production"
            relevant_crops = production_crops
        else:
            return {}  # No costs if the year is not part of establishment or production

        # Fetch the detailed costs for the relevant phase
        detailed_costs = self.conventional_economics.detailed_variable_costs[phase]

        result = {}
        for crop in relevant_crops:
            if phase == "production":
                # Production costs might not be year-specific
                crop_costs = detailed_costs.get(crop, {})  # Retrieve costs directly
                if not isinstance(crop_costs, dict) or not crop_costs:  # Handle empty or invalid entries
                    print(f"DEBUG: No valid production costs found for crop {crop} in Year {t}")
                    continue
            else:
                # Establishment costs are year-specific
                crop_costs = detailed_costs.get(crop, {}).get(t, {})
                
            # Calculate component costs
            if t == 0:
                # No costs in Year 0
                result[crop] = {
                    component: 0 * self.farm_costs_change * area * self.cropschedule.crop_area_percentages.get(crop, 1.0)
                    for component, cost in crop_costs.items()
                }
            else:
                result[crop] = {
                    component: cost * self.farm_costs_change * area * self.cropschedule.crop_area_percentages.get(crop, 1.0)
                    for component, cost in crop_costs.items()
                }
            
        return result
    
    def get_APV_crop_detailed_fixed_costs(self, t):
        """
        Calculate detailed fixed costs for crops in the APV system for year t, automatically handling establishment or production phases.

        Parameters:
        - t: Year for which the costs are calculated.

        Returns:
        - Dictionary where each crop has a sub-dictionary of detailed fixed costs per component for year t.
        """
        
        cropschedule_outputs = self.cropschedule.outputs()
        crop_years = cropschedule_outputs.get('crop_years', {})
        area = self.pv_design.get_area_farm()  # Area used for farm in APV system

        # Determine if year t is part of establishment or production
        establishment_crops = [
            crop for crop, years in crop_years.items() if t in years.get("establishment_years", [])
        ]
        production_crops = [
            crop for crop, years in crop_years.items() if t in years.get("production_years", [])
        ]

        if establishment_crops:
            phase = "establishment"
            relevant_crops = establishment_crops
        elif production_crops:
            phase = "production"
            relevant_crops = production_crops
        else:
            return {}  # No costs if the year is not part of establishment or production

        # Fetch the detailed costs for the relevant phase
        detailed_costs = self.conventional_economics.detailed_fixed_costs[phase]

        result = {}
        for crop in relevant_crops:
            if phase == "production":
                # Production costs might not be year-specific
                crop_costs = detailed_costs.get(crop, {})  # Retrieve costs directly
                if not isinstance(crop_costs, dict) or not crop_costs:  # Handle empty or invalid entries
                    print(f"DEBUG: No valid production costs found for crop {crop} in Year {t}")
                    continue
            else:
                # Establishment costs are year-specific
                crop_costs = detailed_costs.get(crop, {}).get(t, {})
                
            # Calculate component costs
            if t == 0:
                result[crop] = {
                    component: 0 * self.farm_costs_change * area * self.cropschedule.crop_area_percentages.get(crop, 1.0)
                    for component, cost in crop_costs.items()
                }
            else:
                if self.partnership.partnership_type.startswith("Partnership A"):
                    result[crop] = {
                        component: cost * self.farm_costs_change * area * self.cropschedule.crop_area_percentages.get(crop, 1.0)
                        for component, cost in crop_costs.items()
                        if component not in ["Land Depreciation","Land & Property Taxes", "Land Interests", ]  # Exclude Land Costs in case Partnership A
                    }
                else:
                    result[crop] = {
                        component: cost * self.farm_costs_change * area * self.cropschedule.crop_area_percentages.get(crop, 1.0)
                        for component, cost in crop_costs.items()
                        # Don't exclude Land Costs in case Partnership is Partnership B
                    }
            
        return result
        

    # Total Variable or Fixed Costs in $/yr

    def get_APV_crop_total_variable_costs(self, t):
        """
        Calculate the total variable costs for all crops in the APV system for year t.

        Parameters:
        - t: Year for which the costs are calculated.

        Returns:
        - Total variable costs ($/yr) for all crops in the APV system for year t.
        """
        detailed_variable_costs = self.get_APV_crop_detailed_variable_costs(t)

        # Sum variable costs for all crops
        return sum(
            sum(component_costs.values()) for component_costs in detailed_variable_costs.values()
        )

    def get_APV_crop_total_fixed_costs(self, t):
        """
        Calculate the total fixed costs for all crops in the APV system for year t.

        Parameters:
        - t: Year for which the costs are calculated.

        Returns:
        - Total fixed costs ($/yr) for all crops in the APV system for year t.
        """
        detailed_fixed_costs = self.get_APV_crop_detailed_fixed_costs(t)

        # Sum fixed costs for all crops
        return sum(
            sum(component_costs.values()) for component_costs in detailed_fixed_costs.values()
        )
    
    def get_APV_crop_total_costs(self, t):
        """
        Calculate the total costs (variable + fixed) for all crops in the APV system for year t.

        Parameters:
        - t: Year for which the costs are calculated.

        Returns:
        - Total costs ($/yr) for all crops in the APV system for year t.
        """
        total_variable_costs = self.get_APV_crop_total_variable_costs(t)  
        total_fixed_costs = self.get_APV_crop_total_fixed_costs(t)
        return total_variable_costs + total_fixed_costs


    def get_gross_profit(self, t):
        """
        Calculates the Gross Profit per year for the conventional and APV systems.
        For the APV system case, computed for the farm and PV businesses separately and then add them up.

        Returns:
        - gross profit ($/yr)
        """
        if self.system_type == 'conventional PV':
            gross_profit = self.conventional_economics.get_conventional_PV_revenues(t) # no COGS for PV business so gross profit = revenues

        elif self.system_type == 'conventional farm':
            revenues = self.conventional_economics.get_conventional_crop_revenues(t)
            # COGS are only variable costs for farm 
            costs = self.conventional_economics.get_conventional_crop_total_variable_costs(t)
            gross_profit = revenues - costs            
        
        elif self.system_type == 'APV PV':
            gross_profit = self.get_APV_PV_revenues(t) #no costs of goods sold for PV business so gross profit = revenues
            
        elif self.system_type == 'APV farm':
            crop_revenues = self.get_APV_crop_revenues(t)
            # COGS are only variable costs for farm 
            crop_costs = self.get_APV_crop_total_variable_costs(t)
            gross_profit = crop_revenues - crop_costs

        return gross_profit
    
    def get_yearly_land_lease_revenues(self, t):
        """
        Calculate the yearly payments in $ for land lease from solar developer to farmer in case farmer is landowner (Partnership B). 
        Only used in APV case for a farmer, solar developer never receive a land revenue
        t to inflate the payment by the inflation rate for each year starting from year 2 since year 1 is first payment and year 0 = 0
        """
        if self.partnership.partnership_type.startswith("Partnership B"):
            if t == 0:
                yearly_land_lease_revenues = 0
            elif t == 1:
                yearly_land_lease_revenues = self.partnership.land_lease_sd_to_farmer * self.pv_design.totalarea
            else:
                yearly_land_lease_revenues = (self.partnership.land_lease_sd_to_farmer * self.pv_design.totalarea) * ((1 + self.financial_rates.inflation_rate) ** (t-1))

            return yearly_land_lease_revenues
        else:
            return 0
        
    def get_yearly_land_lease_payments(self, t):
        """
        Calculate the yearly payments in $ for land from the Solar Dev perspective
        This payment is introduced as:
        - Land lease payment to a landowner in case of Partnership A 
        - Land lease payment to a farmer in case of Partnership B 

        Land Costs for farmer in case partnership is B are considered in FIXED COSTS as for conventional case
        Like this it is only a function of the input in the fixed costs and can be adapted with the type of crops
        i.e. usually land FC will be interests, taxes, and depre in case the farmer owns a land (small farm business), 
        but will be land lease in case farmer leases the land (big commodity farm)
        For partnership A, there is never land costs for farmer since farmer comes on the land of solar dev to maintain it
        """
        if self.system_type == 'APV PV':
            if self.partnership.partnership_type.startswith("Partnership A"):
                if t == 0:
                    yearly_land_lease_payments = 0
                elif t == 1:
                    yearly_land_lease_payments = self.partnership.land_lease_sd_to_landowner * self.pv_design.totalarea
                else:
                    yearly_land_lease_payments = (self.partnership.land_lease_sd_to_landowner * self.pv_design.totalarea) * ((1 + self.financial_rates.inflation_rate) ** (t-1))
            else:
                if t == 0:
                    yearly_land_lease_payments = 0
                elif t == 1:
                    yearly_land_lease_payments = self.partnership.land_lease_sd_to_farmer * self.pv_design.totalarea
                else:
                    yearly_land_lease_payments = (self.partnership.land_lease_sd_to_farmer * self.pv_design.totalarea) * ((1 + self.financial_rates.inflation_rate) ** (t-1))
        elif self.system_type == 'APV farm':
                yearly_land_lease_payments = 0
                
        return yearly_land_lease_payments
    
    def get_total_initial_investment(self):
        """
        Calculates the Total Initial Investment for the system.
        
        Returns:
        - total initial investment ($)
        """
        if self.system_type == 'conventional PV':
            tot_ini_inv = self.conventional_economics.conventionalPVcapex

            
        elif self.system_type == 'conventional farm':
            tot_ini_inv = self.conventional_economics.get_conventional_crop_capex_total()  # Total capex for conventional farm
            
        elif self.system_type == 'APV PV':
            tot_ini_inv = self.pv_design.capex 
            
        elif self.system_type == 'APV farm':
            # Calculate the total APV capex for each crop 
            # Partnership A: no land owned or lease by farmer so removed it from conventional capex which means the APV capex is only 30% of conventional one
            if self.partnership.partnership_type.startswith("Partnership A"):
                crop_capex = {
                    crop: self.conventional_economics.conventionalcropcapex.get(crop, 0) * self.pv_design.get_area_farm() * self.cropschedule.crop_area_percentages.get(crop, 1.0) * 0.3
                    for crop in self.cropschedule.selected_crops
                }
            # Partnership B: land owned or leased by farmer but in any case the APV capex is the same as conventional capex (except area which differ) 
            else:
                crop_capex = {
                    crop: self.conventional_economics.conventionalcropcapex.get(crop, 0) * self.pv_design.get_area_farm() * self.cropschedule.crop_area_percentages.get(crop, 1.0)
                    for crop in self.cropschedule.selected_crops
                }
            # Sum each individual crop APV capex to get the total one
            tot_ini_inv = sum(crop_capex.values())
        return tot_ini_inv
    
    def get_capital_expenditure(self, t):
        """
        Calculate the capital expenditure for year t.
        Also called PPE (Property, Plant, and Equipment).
        Based on the CAPEX and the depreciation rate.

        Returns:
        - capital expenditure ($/yr)
        """
        if t == 0:
            if self.system_type in ('APV PV', 'conventional PV'):
                capex =  self.get_total_initial_investment() * self.financial_rates.get_equity_share()
            elif self.system_type in ('APV farm', 'conventional farm'):
                capex = self.get_total_initial_investment() 
        else:
            capex = 0

        return capex

    
    def get_opex(self, t):
        """
        Calculates the Operation Expenditure (OPEX) for the system.

        'Conventional' cases: Use the OPEX from conventional economics and apply inflation rate to it(only for PV system since no OPEX for conventional farm)
        'PV' case: Use the OPEX w/o land costs and add to it land costs and potential extra farmer payments (all adjusted by inflation rate)
        'farm' case: No OPEX for farm, every thing is considered as revenues or COGS and thus is handled in gross profit method

        Returns:
        - opex: Operational Expenditure ($/yr) 
        """
        # Conventional systems
        if self.system_type == 'conventional PV':
            if t == 0:
                opex = 0
            elif t == 1:
                opex = self.conventional_economics.conventionalPVopex
            else:   
                opex = self.conventional_economics.conventionalPVopex * ((1 + self.financial_rates.inflation_rate) ** (t-1))
            
        elif self.system_type == 'conventional farm':
            if t == 0:
                opex = 0
            else:
                # Sum specific fixed costs across all crops
                relevant_fixed_costs = [
                    "Land & Property Taxes",
                    "Miscellaneous Supplies Costs",
                    "Insurance Costs",
                    "Management Costs",
                    "Machine Equipment & Building Interests",
                    "Land Interests"
                ]
                detailed_fixed_costs = self.conventional_economics.get_conventional_crop_detailed_fixed_costs(t)
                opex = sum(
                    sum(costs[component] for component in relevant_fixed_costs if component in costs)
                    for costs in detailed_fixed_costs.values()
                )
        # APV system
        elif self.system_type == 'APV PV':
            yearly_extra_farmer_payment = self.get_yearly_extra_farmer_payment(t)
            land_lease_sd_to_landowner = self.get_yearly_land_lease_payments(t)
            if t == 0:
                opex_base = 0
            elif t == 1:
                opex_base = self.pv_design.opex 
            else:
                opex_base = self.pv_design.opex * ((1 + self.financial_rates.inflation_rate) ** (t-1))

            opex = opex_base + yearly_extra_farmer_payment + land_lease_sd_to_landowner
            
        elif self.system_type == 'APV farm':
            if t == 0: 
                opex = 0
            else:
                if self.partnership.partnership_type.startswith("Partnership A"):
                    relevant_fixed_costs = [
                        "Miscellaneous Supplies Costs",
                        "Insurance Costs",
                        "Management Costs",
                        "Machine Equipment & Building Interests"
                    ]
                else:
                    relevant_fixed_costs = [
                        "Land & Property Taxes",
                        "Miscellaneous Supplies Costs",
                        "Insurance Costs",
                        "Management Costs",
                        "Machine Equipment & Building Interests",
                        "Land Interests" 
                    ]
                detailed_fixed_costs = self.get_APV_crop_detailed_fixed_costs(t)
                fixed_costs = sum(
                    sum(costs[component] for component in relevant_fixed_costs if component in costs)
                    for costs in detailed_fixed_costs.values()
                )
                opex = fixed_costs

        return opex

    def get_operating_income(self,t):
        """
        Calculate the Operating Income for year t.
        Based on the Gross Profit, the Operational Expenditure and Depreciation (for all systems and businesses).

        Returns:
        - operating_income: Operating Income ($/yr)
        where operating income = gross profit - opex - depreciation
        """
        operating_income = self.get_gross_profit(t) - self.get_opex(t) - self.get_depreciation(t) 

        return operating_income

    def get_other_income(self, t):
        """
        Calculate the other income for year t.
        In conventional systems no other income
        In APV system, other income is
         - no other income for PV case
         - sum of land lease revenues and extra farmer payments of the solar developer to the farmer for farm case.
        
        Returns:
        - other_income: Other income ($/yr)
        """
        if self.system_type in ('conventional PV', 'conventional farm', 'APV PV'):
            other_income = 0
        elif self.system_type == 'APV farm':
            revenues_land = self.get_yearly_land_lease_revenues(t)
            yearly_farmer_payment = self.get_yearly_extra_farmer_payment(t)
            other_income = revenues_land + yearly_farmer_payment

        return other_income

    def get_annual_debt_payment(self, t):
        """
        Calculate the annual debt payment for year t.
        Based on an annuity formula with a fixed interest rate being the cost of debt.
        Note: annual debt payment = annual principal + annual interests
        """
        if t > 0:
            debt_share = self.financial_rates.debt_share
            debt_term = self.lifetime.years
            cost_of_debt = self.financial_rates.cost_debt
            if self.system_type in ('conventional PV', 'APV PV'):
                annuity = self.get_total_initial_investment() * debt_share #pv of the total amount of loan (without interests)
            elif self.system_type in ('conventional farm', 'APV farm'):
                annuity = 0 # no debt for farm business

            annual_debt_payment = annuity * cost_of_debt / (1 - (1 + cost_of_debt) ** -debt_term)
            return annual_debt_payment
        else:
            return 0
        
    def get_remaining_debt(self, t):
        """
        Calculate the remaining total principal debt (without the interests) at the beginning of year t.
        Interests will be calculated based on this amount.
        Actually this get method is a bit useless since we could only use the self.remaining_debt variable directly.
        But I kept it for the sake of consistency with the other get methods.
        """
        if t == 0:
            #self.remaining_debt[t] = self.get_total_initial_investment() * self.financial_rates.debt_share
            return self.remaining_debt[t]
        else:
            #return self.remaining_debt[t-1] - self.get_annual_principal_payment(t-1)
            return self.remaining_debt[t]
        
    def get_annual_interests_payment(self, t):
        """
        Calculate the annual interests payment for year t.
        Based on the remaining debt and the cost of debt.
        """
        if t == 0:
            return 0
        if t > 0:
            if self.system_type in ('APV PV', 'conventional PV'):
                remaining_debt = self.remaining_debt[t-1]
                cost_of_debt = self.financial_rates.cost_debt
                annual_interests_payment = remaining_debt * cost_of_debt
            elif self.system_type in ('APV farm', 'conventional farm'):
                annual_interests_payment = 0

            return annual_interests_payment
        
    def get_annual_principal_payment(self, t):
        """
        Calculate the annual principal payment for year t.
        Based on the annual debt payment and the annual interests payment.
        Note: principal payment is not tax deductible contrary to interests payment.
        Note: each time after the calculation of the principal payment, the remaining debt is updated.
        """
        if t == 0:
            if  self.system_type == 'conventional PV':
                self.remaining_debt[t] = self.get_total_initial_investment() * self.financial_rates.debt_share
            elif self.system_type == 'APV PV':
                self.remaining_debt[t] = self.get_total_initial_investment() * self.financial_rates.debt_share
            return 0
        if t > 0:
            annual_debt_payment = self.get_annual_debt_payment(t)
            annual_interests_payment = self.get_annual_interests_payment(t)
            annual_principal_payment = annual_debt_payment - annual_interests_payment
            self.remaining_debt[t] = self.remaining_debt[t-1] - annual_principal_payment
            return annual_principal_payment   
    
    def get_equity_financing(self, t):
        """
        Calculate the equity financing for year t=0.
        """
        if t == 0:
            equity_share = self.financial_rates.get_equity_share()
            if self.system_type in ('APV PV', 'conventional PV'):
                equity_financing = self.get_total_initial_investment() * equity_share
        else:
            equity_financing = 0
        return equity_financing
    
    def get_depreciation(self, t):
        """
        Calculates the Depreciation for a given year for the conventional and APV systems.
        Computed for the PV businesses only.
        Depreciation type:
            - Modified Accelerated Cost Recovery System (MACRS),
            - GDS,
            - 5-year recovery period (solar energy property),
            - Declining balance method at 200% DB,
            - Half-year convention.
        Also calculates the basis of depreciation after removing the impact from ITC.
        
        Parameters:
        - t (int): The year for which to calculate depreciation (starting from 0).
        
        Returns:
        - depreciation (float): The depreciation amount for the given year ($).
        """
        # Define MACRS 5-year property schedule percentages (200% DB with half-year convention)
        macrs_schedule = [0.2, 0.32, 0.192, 0.1152, 0.1152, 0.0576]  # Percentages for each year
        
        # Adjust for 1-based start (skip year 0)
        if t == 0:
            return 0  # No depreciation for year 0 (investment year)
        
        # Adjust year index to match MACRS schedule (year 1 corresponds to index 0)
        macrs_index = t - 1
    
        # Check if the year exceeds the MACRS schedule period
        if macrs_index >= len(macrs_schedule):
            return 0  # No depreciation after the 5-year period

        # Calculate the basis for depreciation
        if self.system_type == 'conventional PV':
            capex_pv = self.conventional_economics.conventionalPVcapex
            itc_rate = self.financial_rates.itc_rate
            depreciation_basis = capex_pv * (1 - itc_rate / 2)
            # Calculate depreciation for year t based on the MACRS schedule
            depreciation = depreciation_basis * macrs_schedule[macrs_index]

        elif self.system_type == 'conventional farm':
            # Sum the fixed costs "Machine Equipment & Building Depreciation" and "Land Depreciation"
            relevant_fixed_costs = ["Machine Equipment & Building Depreciation", "Land Depreciation"]
            detailed_fixed_costs = self.conventional_economics.get_conventional_crop_detailed_fixed_costs(t)
            depreciation = sum(
                sum(costs[component] for component in relevant_fixed_costs if component in costs)
                for costs in detailed_fixed_costs.values()
            )

        elif self.system_type == 'APV PV':
            capex_pv = self.get_total_initial_investment()
            itc_rate = self.financial_rates.itc_rate
            depreciation_basis = capex_pv * (1 - itc_rate / 2) #depreciation is calculated on the full CAPEX not just the equity part (cf.lazard methodology p36)
            # Calculate depreciation for year t based on the MACRS schedule
            depreciation = depreciation_basis * macrs_schedule[macrs_index]
            
        elif self.system_type == 'APV farm':
            # Sum the fixed costs "Machine Equipment & Building Depreciation" and "Land Depreciation" if partnership B)
            if self.partnership.partnership_type.startswith("Partnership A"):
                relevant_fixed_costs = ["Machine Equipment & Building Depreciation"] 
            else:
                relevant_fixed_costs = ["Machine Equipment & Building Depreciation", "Land Depreciation"]

            detailed_fixed_costs = self.get_APV_crop_detailed_fixed_costs(t)
            depreciation = sum(
                sum(costs[component] for component in relevant_fixed_costs if component in costs)
                for costs in detailed_fixed_costs.values()
            )

        return depreciation

    def get_ebit(self, t):
        """
        Calculates the Earning Before Interests and Taxes (EBIT) per year for the APV system or conventional system.
        For the APV system case, we can calculate the EBIT for the farm and PV businesses separately or for the whole APV system.
        For the conventional system, we can calculate the EBIT

        Returns:
        - ebit ($/year) 
        where ebit = operating income + other income = gross profit - opex - depreciation + other income
        """
        ebit = self.get_operating_income(t) + self.get_other_income(t)
        return ebit

    def get_ebt(self, t):
        """
        Calculates the Earning Before Taxes (EBT) per year for the APV system or conventional system.
        Also called the pre-tax income or taxable income.
        For the APV system case, we can calculate the EBT for the farm and PV businesses separately or for the whole APV system.
        For the conventional system, we can calculate the EBT
        Computed for the farm and PV businesses separately and then add them up.

        Returns:
        - ebt ($/year) 
        where ebt = ebit - interests payment = operating income + other income - interests payment = gross profit - opex - depreciation + other income - interests payment
        """
        if self.system_type in ('conventional PV', 'APV PV'):
            ebt = self.get_ebit(t) - self.get_annual_interests_payment(t)
        elif self.system_type in ('conventional farm', 'APV farm'):
            ebt = self.get_ebit(t)
        return ebt
    
    
    def get_itc(self, t):
        """
        Calculates the Investment Tax Credit (ITC-Section 48e) for the PV and APV systems (not for farm systems since it does not exist).
        This ITC is only applicable for the first year (t=1) of the project, but can be carried over if not fully used.

        Parameters:
        - t (int): The year for which the ITC is being calculated.

        Returns:
        - investment_tax_credit (float): The ITC value for the specified year ($).
        """
        if t == 1:
            # Apply ITC in the first year only
            if self.system_type == 'conventional PV':
                investment_tax_credit = self.conventional_economics.conventionalPVcapex * self.financial_rates.itc_rate
            elif self.system_type == 'conventional farm':
                investment_tax_credit = 0
            else:
                if self.system_type == 'APV PV':
                    # ITC for the PV business only in the APV system
                    investment_tax_credit = self.get_total_initial_investment() * self.financial_rates.itc_rate
                elif self.system_type == 'APV farm':
                    investment_tax_credit = 0
        else:
            # No ITC after the first year no matter the case
            investment_tax_credit = 0

        return investment_tax_credit 
    
    def get_carryover(self, t):
        """
        Returns carryover from the previous year.
        Includes ITC in Year 1 if applicable.
        """
        if t == 1:
            # Add ITC for the first year only
            if self.system_type in ('APV PV', 'conventional PV'):
                return self.get_itc(t) 
        elif t > 1:
            # For subsequent years, use the carryover value from the previous year (same for all cases)
            return self.carryover[t - 1]
        else:
            # No carryover in Year 0 for all cases
            return 0

    def update_carryover(self, t, carryover):
        """
        Updates the carryover for year t.
        This method is the same no matter the system type (conventional or APV) or business (PV or farm).
        """
        if t < len(self.carryover):
            #print(f"Carryover for year {t} updated to {carryover}")
            self.carryover[t] = carryover
            #print(f"Carryover for year {t} is now {self.carryover[t]}")
        else:
            self.carryover.append(carryover)  # Extend the carryover list if necessary
    
    def get_state_income_tax(self, t):
        """
        Calculates the State Income Tax per year for the conventional and APV systems.
        If EBT is negative, income tax is set to 0. We assume no tax credit, nor NOLs carryforwards for state income tax.
        """
        ebt = self.get_ebt(t)
        if ebt <= 0:
            state_income_tax = 0
        else:
            if self.system_type in ('APV PV', 'conventional PV'):
                state_income_tax = self.financial_rates.state_tax_rate_pv * ebt
            elif self.system_type in ('APV farm', 'conventional farm'):
                state_income_tax = self.financial_rates.state_tax_rate_farm * ebt

        return state_income_tax

    def get_fed_income_tax(self, t):
        """
        Calculates the Federal Income Tax per year for the conventional and APV systems.
        If EBT (after removing state income tax) is negative, income tax is set to 0, 
        But any available tax credits are carried over to the next year.
        NOLs are not carried over for the moment.
        """
        # Remove state income tax from EBT to get federal taxable income 
        # (https://www.mercatus.org/students/research/policy-briefs/deduction-state-and-local-taxes-federal-income-taxes)
        if self.system_type in ('APV PV', 'conventional PV'):
            ebt = self.get_ebt(t)
            carryover = self.get_carryover(t)
            ebt_after_state_tax = ebt - self.get_state_income_tax(t)   
        else: 
            ebt = self.get_ebt(t)
            ebt_after_state_tax = ebt - self.get_state_income_tax(t)

        if ebt_after_state_tax <= 0:
            fed_income_tax = 0
            if self.system_type in ('APV PV', 'conventional PV'):
                self.update_carryover(t, carryover)  # Preserve carryover if EBT after state tax is negative
        else:
            if self.system_type in ('APV PV', 'conventional PV'):
                fed_income_tax = self.financial_rates.fed_tax_rate_pv * ebt_after_state_tax - carryover
            elif self.system_type in ('APV farm', 'conventional farm'):
                fed_income_tax = self.financial_rates.fed_tax_rate_farm * ebt_after_state_tax 

            if fed_income_tax < 0:
                # Carryover unused tax credit
                if self.system_type in ('APV PV', 'conventional PV'):
                    self.update_carryover(t, -fed_income_tax)
                fed_income_tax = 0  # No negative federal income tax
            else:
                # No carryover remains
                if self.system_type in ('APV PV', 'conventional PV'):
                    self.update_carryover(t, 0)

        return fed_income_tax

    def get_net_income(self, t):
        """
        Calculates the Net Income per year for the conventional and APV systems.
        Also called the after-tax income or net earnings.
        For the APV system case, we can calculate the Net Income for the farm and PV businesses separately or for the whole APV system.
        For the conventional system, we can calculate the Net Income.

        Returns:
        - net_income ($/year) 
        where net income = ebt - federal income tax - state income tax
        """

        net_income = self.get_ebt(t) - self.get_fed_income_tax(t) - self.get_state_income_tax(t)

        return net_income
    
    def get_cash_generated_operating_activities(self, t):
        """
        Calculates the Cash Generated from Operating Activities per year for the conventional and APV systems.
        For the APV system case, we can calculate the Cash Generated from Operating Activities for the farm and PV businesses separately or for the whole APV system.
        For the conventional system, we can calculate the Cash Generated from Operating Activities.

        Returns:
        - cash_generated_operating_activities ($/year) 
        where cash generated from operating activities = net income + depreciation

        TBD later: deferred taxes (when NOLs considered), changes in working capital, etc....
        """

        cash_generated_operating_activities = self.get_net_income(t) + self.get_depreciation(t)

        return cash_generated_operating_activities
    
    def get_cash_generated_investing_activities(self, t):
        """
        Calculates the Cash Generated from Investing Activities per year for the conventional and APV systems.
        For the APV system case, we can calculate the Cash Generated from Investing Activities for the farm and PV businesses separately or for the whole APV system.
        For the conventional system, we can calculate the Cash Generated from Investing Activities.

        Returns:
        - cash_generated_investing_activities ($/year) 
        where cash generated from investing activities = -capex
        """

        cash_generated_investing_activities = self.get_capital_expenditure(t)

        return cash_generated_investing_activities

    def get_cash_generated_financing_activities(self, t):
        """
        Calculates the Cash Generated from Financing Activities per year for the conventional and APV systems.
        For the APV system case, we can calculate the Cash Generated from Financing Activities for the farm and PV businesses separately or for the whole APV system.
        For the conventional system, we can calculate the Cash Generated from Financing Activities.

        Returns:
        - cash_generated_financing_activities ($/year) 
        where cash generated from financing activities = - annual principal payment
        """
        if self.system_type in ('APV PV', 'conventional PV'):
            cash_generated_financing_activities = self.get_annual_principal_payment(t)
        elif self.system_type in ('APV farm', 'conventional farm'):
            cash_generated_financing_activities = 0 #self.get_annual_principal_payment(t)

        return cash_generated_financing_activities
    
    def get_cashflows(self, t):
        """
        Calculates the Cash Flows per year for the APV and conventional systems.
        For the APV system case, we can calculate the cash flows for the farm and PV businesses separately or for the whole APV system.

        Returns:
        - cashflows ($/year)
        where cashflows = cash generated from operating activities - cash generated from investing activities - cash generated from financing activities
        """
        
        cashflows = self.get_cash_generated_operating_activities(t) - self.get_cash_generated_investing_activities(t) - self.get_cash_generated_financing_activities(t)
        
        return cashflows
    
    def get_npv(self):
        """
        Calculates the Net Present Value (NPV) of the APV system.

        Returns:
        - npv ($)
        """
    
        npv = 0        
        years = self.lifetime.years
        discount_factors = self.get_discount_factors()

        for t in range(years + 1):    
            cashflows_t = self.get_cashflows(t)

            # Net Present Value Calculation (will give the final NPV and not a list of NPV for each year)
            npv += (cashflows_t) * discount_factors[t]
        return npv
    
    def plot_cumulated_discounted_cash_flows(self, color='skyblue', y_min=None, y_max=None):
        """Plot the cumulated discounted cash flows for each year as a bar graph."""

        # Calculate cash flows for each year
        years = self.lifetime.years
        
        # Initialize cumulative discounted cash flows list
        cumulative_cash_flows = []
        cumulative_total = 0
        
        # Fetch discount factors

        discount_factors = self.get_discount_factors()
        
        # Calculate discounted cash flows and their cumulative total
        for t in range(years + 1):
            cash_flow_t = self.get_cashflows(t)    
            discounted_cash_flow = cash_flow_t*discount_factors[t]
            cumulative_total += discounted_cash_flow
            cumulative_cash_flows.append(cumulative_total)
        
        # Plotting
        plt.figure(figsize=(10, 6))
        plt.bar(range(years + 1), cumulative_cash_flows, color=color)
        # Set font properties
        font_properties = {'fontname': 'Arial', 'fontsize': 20}

        plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '${:.1f}'.format(x / 1e6)))
        plt.xlabel('Year', **font_properties)
        plt.ylabel('Cumulated Discounted Cash Flow ($M)', **font_properties)

        # Set y-axis limits if specified
        if y_min is not None or y_max is not None:
            plt.ylim(y_min, y_max)

        # Set tick labels font
        plt.xticks(fontname='Arial', fontsize=20)
        plt.yticks(fontname='Arial', fontsize=20)

        plt.show()
        
    
    def generate_dcf_table(self):
        """
        Generate a table of the DCF analysis with columns as years and rows for metrics.
        Will only do a printing of each rows from the data collected in the previous methods.
        So don't expect to see formulae in the table but only the results of the calculations since formulae are used in the methods
        
        Metrics include:
        - PV Revenues (incentives and base) in $/yr
        - Farm Revenues in $/yr
        - PV COGS in $/yr
        - Farm COGS in $/yr
        - Gross Profit in $/yr
        - PV Operational Costs (OPEX) in $/yr
        - Farm Operational Costs (OPEX) in $/yr
        - Depreciation in $/yr
        - EBT in $/yr
        - Investment Tax Credit (ITC) only for year 1 in $/yr
        - Income Tax in $/yr
        - Net Income in $/yr
        - PV Capital Costs in $ for yr=0
        - Farm Capital Costs in $ for yr=0
        - Nominal Cashflows in $/yr
        - Discounted Cashflows in $/yr

        Returns:
        - DataFrame: A table displaying the DCF analysis.
        """

        years = self.lifetime.years
        discount_factors = self.get_discount_factors()

        # Initialize lists for each metric
        pv_pbi_revenues = []
        pv_srec_revenues = []
        pv_base_revenues = []
        pv_total_revenues = []
        crop_revenues = []
        farm_revenues_payment = []
        farm_land_revenues = []
        farm_land_payments = []
        pv_cogs = []    
        farm_cogs = []
        pv_gross_profit = []
        farm_gross_profit = []
        gross_profit = []
        pv_opecosts = []
        farm_opecosts = []

        pv_ope_income = []
        farm_ope_income = []

        pv_other_income = []
        farm_other_income = []

        pv_depreciation = []
        farm_depreciation = []
        depreciation = []
        pv_ebit = []
        farm_ebit = []
        pv_ebt = []
        farm_ebt = []
        ebt = []
        remaining_itc = []
        carry_over = []
        pv_state_income_tax = []
        farm_state_income_tax = []
        pv_fed_income_tax = []
        farm_fed_income_tax = []
        state_income_tax = []
        fed_income_tax = []

        pv_net_income = []
        farm_net_income = []
        
        annual_debt_payment = []
        annual_interests_payment = []
        annual_principal_payment = []
        remaining_debt = []
        pv_annual_debt_payment = []
        pv_annual_interests_payment = []
        pv_annual_principal_payment = []
        pv_remaining_debt = []
        pv_capcosts = []
        farm_capcosts = []
        pv_cash_ope_activities = []
        farm_cash_ope_activities = []
        pv_capcosts = []
        farm_capcosts = []
        pv_cash_inv_activities = []
        farm_cash_inv_activities = []
        pv_cash_fin_activities = []
        farm_cash_fin_activities = []
        pv_nominal_cashflows = []
        farm_nominal_cashflows = []
        nominal_cashflows = []
        pv_discounted_cashflows = []
        farm_discounted_cashflows = []
        discounted_cashflows = []

        for t in range(years + 1):
            
            # Set revenues and costs only from year 1 onwards

            if self.system_type == 'conventional PV':
                pv_pbi_revenues_t = self.conventional_economics.get_conventional_pbi_revenues(t)  if t >= 1 else 0
                pv_srec_revenues_t = self.conventional_economics.get_conventional_srec_revenues(t) if t >= 1 else 0
                pv_base_revenues_t = self.conventional_economics.get_conventional_base_PV_revenues(t) if t >= 1 else 0
                pv_total_revenues_t = self.conventional_economics.get_conventional_PV_revenues(t) if t >= 1 else 0
                pv_cogs_t = 0
                annual_debt_payment_t = self.get_annual_debt_payment(t) 
                annual_interests_payment_t = self.get_annual_interests_payment(t) 
                annual_principal_payment_t = self.get_annual_principal_payment(t) 
                remaining_debt_t = self.get_remaining_debt(t) 

            elif self.system_type == 'conventional farm':
              
                crop_revenues_t = self.conventional_economics.get_conventional_crop_revenues(t) if t >= 1 else 0
                farm_cogs_t = self.conventional_economics.get_conventional_crop_total_variable_costs(t) if t >= 1 else 0

            else:
                
                crop_revenues_t = self.get_APV_crop_revenues(t) 
                farm_revenues_payment_t = self.get_yearly_extra_farmer_payment(t) 
                farm_land_revenues_t = self.get_yearly_land_lease_revenues(t)
                farm_land_payments_t = self.get_yearly_land_lease_payments(t)

                pv_pbi_revenues_t = self.get_pbi_revenues(t)
                pv_srec_revenues_t = self.get_srec_revenues(t) 
                pv_base_revenues_t = self.get_base_PV_revenues(t) 
                pv_total_revenues_t = self.get_APV_PV_revenues(t) 
                
                pv_cogs_t = 0
                farm_cogs_t = self.get_APV_crop_total_variable_costs(t) 

                pv_annual_debt_payment_t = self.get_annual_debt_payment(t)
                pv_annual_interests_payment_t = self.get_annual_interests_payment(t) 
                pv_annual_principal_payment_t = self.get_annual_principal_payment(t) 
                pv_remaining_debt_t = self.get_remaining_debt(t)


            # Shared metrics

            gross_profit_t = self.get_gross_profit(t) if t >= 1 else 0
            depreciation_t = self.get_depreciation(t) if t >= 1 else 0
            opecosts_t = self.get_opex(t) 
            ope_income_t = self.get_operating_income(t) if t >= 1 else 0
            other_income_t = self.get_other_income(t) 
            ebit_t = self.get_ebit(t) 
            ebt_t = self.get_ebt(t) if t >= 1 else 0
            remaining_itc_t = self.get_itc(t) if t == 1 else 0
            carry_over_t = self.get_carryover(t)
            state_income_tax_t = self.get_state_income_tax(t) if t >= 1 else 0
            fed_income_tax_t = self.get_fed_income_tax(t) if t >= 1 else 0
            net_income_t = self.get_net_income(t) if t >= 1 else 0
            cash_ope_activities_t = self.get_cash_generated_operating_activities(t) if t >= 1 else 0
            capcosts_t = self.get_capital_expenditure(t)
            cash_inv_activities_t = self.get_cash_generated_investing_activities(t) if t >= 1 else 0
            cash_fin_activities_t = self.get_cash_generated_financing_activities(t) if t >= 1 else 0


            cashflow_t = self.get_cashflows(t)
            discounted_cashflow_t = cashflow_t * discount_factors[t]

            # Append values to corresponding lists for each system type (won't have same table length if system type is conventional)
            if self.system_type == 'conventional PV':
                pv_pbi_revenues.append(pv_pbi_revenues_t)
                pv_srec_revenues.append(pv_srec_revenues_t)
                pv_base_revenues.append(pv_base_revenues_t)
                pv_total_revenues.append(pv_total_revenues_t)
                pv_cogs.append(pv_cogs_t)
                gross_profit.append(gross_profit_t)

                pv_opecosts.append(opecosts_t)
                depreciation.append(depreciation_t)
                pv_ope_income.append(ope_income_t)
                pv_other_income.append(other_income_t)
                pv_ebit.append(ebit_t)
                
                ebt.append(ebt_t)

                remaining_itc.append(remaining_itc_t)
                carry_over.append(carry_over_t)
                state_income_tax.append(state_income_tax_t)
                fed_income_tax.append(fed_income_tax_t)
                pv_net_income.append(net_income_t)
                annual_debt_payment.append(annual_debt_payment_t)
                annual_interests_payment.append(annual_interests_payment_t)
                annual_principal_payment.append(annual_principal_payment_t)
                remaining_debt.append(remaining_debt_t)
                
                pv_cash_ope_activities.append(cash_ope_activities_t)
                pv_capcosts.append(capcosts_t)
                pv_cash_inv_activities.append(cash_inv_activities_t)
                pv_cash_fin_activities.append(cash_fin_activities_t)

                nominal_cashflows.append(cashflow_t)

                discounted_cashflows.append(discounted_cashflow_t)

                # Create a DataFrame with each list as a row
                dcf_data = {
                    'PV PBI Revenues ($/yr)': pv_pbi_revenues,
                    'PV SREC Revenues ($/yr)': pv_srec_revenues,
                    'PV base Revenues ($/yr)': pv_base_revenues,
                    'PV total Revenues ($/yr)': pv_total_revenues,
                    'PV COGS ($/yr)': pv_cogs,
                    'Gross Profit ($/yr)': gross_profit,
                    'PV Operational Costs OPEX ($/yr)': pv_opecosts,
                    'Depreciation ($/yr)': depreciation,
                    'PV Operating Income ($/yr)': pv_ope_income,
                    'PV Other Income ($/yr)': pv_other_income,
                    'PV EBIT ($/yr)': pv_ebit,
                    'Annual Debt Payment ($/yr)': annual_debt_payment,
                    'Annual Interests Payment ($/yr)': annual_interests_payment,
                    'Annual Principal Payment ($/yr)': annual_principal_payment,
                    'Remaining Debt ($/yr)': remaining_debt,
                    'EBT ($/yr)': ebt,
                    'ITC ($/yr)': remaining_itc,
                    'Carryover ($/yr)': carry_over,
                    'Less: State Income Tax ($/yr)': state_income_tax,
                    'Less: Federal Income Tax ($/yr)': fed_income_tax,
                    'Net Income ($/yr)': pv_net_income,
                    'Plus: Depreciation ($/yr)': depreciation,
                    'Cash Generated from Operating Activities ($/yr)': pv_cash_ope_activities,
                    'Less: Capex ($/yr)': pv_capcosts,
                    'Cash Generated from Investing Activities ($/yr)': pv_cash_inv_activities,
                    'Less: Annual Principal Payment ($/yr)': annual_principal_payment,
                    'Cash Generated from Financing Activities ($/yr)': pv_cash_fin_activities,         
                    'Nominal Cashflows ($/yr)': nominal_cashflows,
                    'Discounted Cashflows ($/yr)': discounted_cashflows
                }


            elif self.system_type == 'conventional farm':
        
                crop_revenues.append(crop_revenues_t)
                farm_cogs.append(farm_cogs_t)
                gross_profit.append(gross_profit_t)
                farm_depreciation.append(depreciation_t)
                farm_opecosts.append(opecosts_t)
                farm_ope_income.append(ope_income_t)
                farm_other_income.append(other_income_t)
                farm_ebit.append(ebit_t)
                ebt.append(ebt_t)
                state_income_tax.append(state_income_tax_t)
                fed_income_tax.append(fed_income_tax_t)
                farm_net_income.append(net_income_t)
                farm_capcosts.append(capcosts_t)
                farm_cash_ope_activities.append(cash_ope_activities_t)
                farm_cash_inv_activities.append(cash_inv_activities_t)
                farm_cash_fin_activities.append(cash_fin_activities_t)
                nominal_cashflows.append(cashflow_t)
                discounted_cashflows.append(discounted_cashflow_t)
                
                # Create a DataFrame with each list as a row
                dcf_data = {
                    'Crop Revenues ($/yr)': crop_revenues,
                    'Costs of Goods Sold (VC) ($/yr)': farm_cogs,
                    'Gross Profit ($/yr)': gross_profit,
                    'Operational Costs OPEX ($/yr)': farm_opecosts,
                    'Depreciation ($/yr)': farm_depreciation,
                    'Operating Income ($/yr)': farm_ope_income,
                    'Other Income ($/yr)': farm_other_income,
                    'EBIT ($/yr)': farm_ebit,
                    'EBT ($/yr)': ebt,
                    'Less: State Income Tax ($/yr)': state_income_tax,
                    'Less: Federal Income Tax ($/yr)': fed_income_tax,
                    'Net Income ($/yr)': farm_net_income,
                    'Plus: Depreciation ($/yr)': farm_depreciation,        
                    'Cash Generated from Operating Activities ($/yr)': farm_cash_ope_activities,
                    'Less: Capex ($/yr)': farm_capcosts,
                    'Cash Generated from Investing Activities ($/yr)': farm_cash_inv_activities,
                    'Cash Generated from Financing Activities ($/yr)': farm_cash_fin_activities, 
                    'Nominal Cashflows ($/yr)': nominal_cashflows,
                    'Discounted Cashflows ($/yr)': discounted_cashflows
                }


            elif self.system_type == 'APV PV':

                pv_pbi_revenues.append(pv_pbi_revenues_t)
                pv_srec_revenues.append(pv_srec_revenues_t)
                pv_base_revenues.append(pv_base_revenues_t)
                pv_total_revenues.append(pv_total_revenues_t)
                pv_cogs.append(pv_cogs_t)
                pv_gross_profit.append(gross_profit_t)

                pv_opecosts.append(opecosts_t)
                pv_depreciation.append(depreciation_t)
                pv_ope_income.append(ope_income_t)

                pv_other_income.append(other_income_t)
                pv_ebit.append(ebit_t)

                pv_annual_debt_payment.append(pv_annual_debt_payment_t)
                pv_annual_interests_payment.append(pv_annual_interests_payment_t)
                pv_annual_principal_payment.append(pv_annual_principal_payment_t)
                pv_remaining_debt.append(pv_remaining_debt_t)

                pv_ebt.append(ebt_t)

                remaining_itc.append(remaining_itc_t)
                carry_over.append(carry_over_t)
                pv_state_income_tax.append(state_income_tax_t)
                pv_fed_income_tax.append(fed_income_tax_t)
  
                pv_net_income.append(net_income_t)

                pv_capcosts.append(capcosts_t)
                
                pv_cash_ope_activities.append(cash_ope_activities_t)
                pv_cash_inv_activities.append(cash_inv_activities_t)
                pv_cash_fin_activities.append(cash_fin_activities_t)

                pv_nominal_cashflows.append(cashflow_t)
                pv_discounted_cashflows.append(discounted_cashflow_t)
  

                # Create a DataFrame with each list as a row
                dcf_data = {
                    'PV PBI Revenues ($/yr)': pv_pbi_revenues,
                    'PV SREC Revenues ($/yr)': pv_srec_revenues,
                    'PV base Revenues ($/yr)': pv_base_revenues,
                    'PV total Revenues ($/yr)': pv_total_revenues,
                    'PV COGS ($/yr)': pv_cogs,
                    'PV Gross Profit ($/yr)': pv_gross_profit,
                    'PV Operational Costs  OPEX($/yr)': pv_opecosts,
                    'PV Depreciation ($/yr)': pv_depreciation,
                    'PV Operating Income ($/yr)': pv_ope_income,
                    'PV Other Income ($/yr)': pv_other_income,
                    'PV EBIT ($/yr)': pv_ebit,
                    'PV Annual Debt Payment ($/yr)': pv_annual_debt_payment,
                    'PV Annual Interests Payment ($/yr)': pv_annual_interests_payment,
                    'PV Annual Principal Payment ($/yr)': pv_annual_principal_payment,
                    'PV Remaining Debt ($/yr)': pv_remaining_debt,
                    'PV EBT ($/yr)': pv_ebt,
                    'ITC ($/yr)': remaining_itc,
                    'Carryover ($/yr)': carry_over,
                    'Less: PV State Income Tax ($/yr)': pv_state_income_tax,
                    'Less: PV Federal Income Tax ($/yr)': pv_fed_income_tax,
                    'PV Net Income ($/yr)': pv_net_income,
                    'Plus: Depreciation ($/yr)': pv_depreciation,
                    'Cash Generated from Operating Activities ($/yr)': pv_cash_ope_activities,
                    'Less: PV Equity Capital Costs ($/yr)': pv_capcosts,
                    'Cash Generated from Investing Activities ($/yr)': pv_cash_inv_activities,
                    'Less: PV Annual Principal Payment ($/yr)': pv_annual_principal_payment,
                    'Cash Generated from Financing Activities ($/yr)': pv_cash_fin_activities,
                    'PV Nominal Cashflows ($/yr)': pv_nominal_cashflows,
                    'PV Discounted Cashflows ($/yr)': pv_discounted_cashflows
                }

            elif self.system_type == 'APV farm':
                crop_revenues.append(crop_revenues_t)
                farm_land_revenues.append(farm_land_revenues_t)
                farm_cogs.append(farm_cogs_t)
                farm_land_payments.append(farm_land_payments_t)
                farm_revenues_payment.append(farm_revenues_payment_t)
                farm_gross_profit.append(gross_profit_t)
                farm_opecosts.append(opecosts_t)
                farm_depreciation.append(depreciation_t)
                farm_ope_income.append(ope_income_t)
                farm_other_income.append(other_income_t)
                farm_ebit.append(ebit_t)
                farm_ebt.append(ebt_t)

                farm_state_income_tax.append(state_income_tax_t)
                farm_fed_income_tax.append(fed_income_tax_t)
                farm_net_income.append(net_income_t)

                farm_capcosts.append(capcosts_t)

                farm_cash_ope_activities.append(cash_ope_activities_t)
                farm_cash_inv_activities.append(cash_inv_activities_t)
                farm_cash_fin_activities.append(cash_fin_activities_t)
                
                farm_nominal_cashflows.append(cashflow_t)
                farm_discounted_cashflows.append(discounted_cashflow_t)

                # Create a DataFrame with each list as a row
                dcf_data = {
                    'Crop Revenues ($/yr)': crop_revenues,
                    'Farm Costs of Goods Sold ($/yr)': farm_cogs,
                    'Farm Gross Profit ($/yr)': farm_gross_profit,
                    'Farm Land Payments ($/yr)': farm_land_payments,
                    'Farm Operational Costs OPEX ($/yr)': farm_opecosts,
                    'Farm Depreciation ($/yr)': farm_depreciation,
                    'Farm Operating Income ($/yr)': farm_ope_income,
                    'Farm Other Income ($/yr)': farm_other_income,
                    'Farm EBIT ($/yr)': farm_ebit,
                    'Farm EBT ($/yr)': farm_ebt,
                    'Less: Farm State Income Tax ($/yr)': farm_state_income_tax,
                    'Less: Farm Federal Income Tax ($/yr)': farm_fed_income_tax,
                    'Farm Net Income ($/yr)': farm_net_income,
                    'Plus: Depreciation ($/yr)': farm_depreciation,
                    'Cash Generated from Operating Activities ($/yr)': farm_cash_ope_activities,
                    'Less: Farm Capital Costs ($/yr)': farm_capcosts,
                    'Cash Generated from Investing Activities ($/yr)': farm_cash_inv_activities,
                    'Cash Generated from Financing Activities ($/yr)': farm_cash_fin_activities,
                    'Farm Nominal Cashflows ($/yr)': farm_nominal_cashflows,
                    'Farm Discounted Cashflows ($/yr)': farm_discounted_cashflows
                }

        # Create DataFrame with years as columns
        dcf_table = pd.DataFrame(dcf_data, index=range(years + 1)).T
        dcf_table.columns = [f"Year {t}" for t in range(years + 1)]

        return dcf_table

    def print_dcf_table(self):
        """Prints the DCF table."""
        # Reset tax credit carryover before generating the table
        self.tax_credit_carryover = 0

        dcf_table = self.generate_dcf_table()
        print(dcf_table)

# Revenues/Costs Table for crops (conventional farm and APV farm)
    
    def generate_farm_budget_table(self):
        """
        Generate a budget table for farm businesses (conventional and APV systems).
        Includes all establishment years as columns and a single column for production years.
        Rows include revenues, land lease revenues, and detailed variable/fixed costs.

        Returns:
        - A pandas DataFrame.
        """
        # Define row labels
        row_labels = ["Revenues", "Land Lease Revenues"] + [
            f"Variable Cost: {component}" for component in [
                "Establishment & Soil Preparation", "Field Activities", "Harvest Activities",
                "Packing & Handling Charges", "Maintenance & Repairs", "Other Variable Costs"
            ]
        ] + [
            f"Fixed Cost: {component}" for component in [
                "Machine Equipment & Building Depreciation", "Land Depreciation",
                "Machine Equipment & Building Interests", "Land Interests",
                "Land & Property Taxes", "Miscellaneous Supplies Costs",
                "Insurance Costs", "Management Costs"
            ]
        ]

        # Initialize table data dictionary
        table_data = {label: [] for label in row_labels}

        # Fetch cropschedule outputs and crop years
        cropschedule_outputs = self.cropschedule.outputs()
        crop_years = cropschedule_outputs.get("crop_years", {})

        # Gather establishment years and production years
        establishment_years = sorted(set(
            year for crop, years in crop_years.items() for year in years.get("establishment_years", [])
        ))
        production_years = sorted(set(
            year for crop, years in crop_years.items() for year in years.get("production_years", [])
        ))

        # Add columns for establishment years
        for t in establishment_years:
            # Revenues
            if self.system_type == "conventional farm":
                revenues = self.conventional_economics.get_conventional_crop_revenues(t)
            else:
                revenues = self.get_APV_crop_revenues(t)
            table_data["Revenues"].append(revenues)

            # Land Lease Revenues
            land_lease_revenues = self.get_yearly_land_lease_revenues(t)
            table_data["Land Lease Revenues"].append(land_lease_revenues)

            # Variable Costs
            if self.system_type == "conventional farm":
                variable_costs = self.conventional_economics.get_conventional_crop_detailed_variable_costs(t)
            else:
                variable_costs = self.get_APV_crop_detailed_variable_costs(t)
            for component in [
                "Establishment & Soil Preparation", "Field Activities", "Harvest Activities",
                "Packing & Handling Charges", "Maintenance & Repairs", "Other Variable Costs"
            ]:
                table_data[f"Variable Cost: {component}"].append(
                    sum(variable_costs.get(crop, {}).get(component, 0) for crop in self.cropschedule.selected_crops)
                )

            # Fixed Costs
            if self.system_type == "conventional farm":
                fixed_costs = self.conventional_economics.get_conventional_crop_detailed_fixed_costs(t)
            else:
                fixed_costs = self.get_APV_crop_detailed_fixed_costs(t)
            for component in [
                "Machine Equipment & Building Depreciation", "Land Depreciation",
                "Machine Equipment & Building Interests", "Land Interests",
                "Land & Property Taxes", "Miscellaneous Supplies Costs",
                "Insurance Costs", "Management Costs"
            ]:
                table_data[f"Fixed Cost: {component}"].append(
                    sum(fixed_costs.get(crop, {}).get(component, 0) for crop in self.cropschedule.selected_crops)
                )

        # Add a single column for production years
        if production_years:
            t = production_years[0]  # Use the first production year

            # Revenues
            if self.system_type == "conventional farm":
                revenues = self.conventional_economics.get_conventional_crop_revenues(t)
            else:
                revenues = self.get_APV_crop_revenues(t)
            table_data["Revenues"].append(revenues)

            # Land Lease Revenues
            land_lease_revenues = self.get_yearly_land_lease_revenues(t)
            table_data["Land Lease Revenues"].append(land_lease_revenues)

            # Variable Costs
            if self.system_type == "conventional farm":
                variable_costs = self.conventional_economics.get_conventional_crop_detailed_variable_costs(t)
            else:
                variable_costs = self.get_APV_crop_detailed_variable_costs(t)
            for component in [
                "Establishment & Soil Preparation", "Field Activities", "Harvest Activities",
                "Packing & Handling Charges", "Maintenance & Repairs", "Other Variable Costs"
            ]:
                table_data[f"Variable Cost: {component}"].append(
                    sum(variable_costs.get(crop, {}).get(component, 0) for crop in self.cropschedule.selected_crops)
                )

            # Fixed Costs
            if self.system_type == "conventional farm":
                fixed_costs = self.conventional_economics.get_conventional_crop_detailed_fixed_costs(t)
            else:
                fixed_costs = self.get_APV_crop_detailed_fixed_costs(t)
            for component in [
                "Machine Equipment & Building Depreciation", "Land Depreciation",
                "Machine Equipment & Building Interests", "Land Interests",
                "Land & Property Taxes", "Miscellaneous Supplies Costs",
                "Insurance Costs", "Management Costs"
            ]:
                table_data[f"Fixed Cost: {component}"].append(
                    sum(fixed_costs.get(crop, {}).get(component, 0) for crop in self.cropschedule.selected_crops)
                )

        # Convert to DataFrame
        table_df = pd.DataFrame.from_dict(table_data, orient="index")
        column_labels = [f"Year {t}" for t in establishment_years] + ["Production Years"]
        table_df.columns = column_labels

        return table_df

    def export_dcf_to_excel(self, filename, table = "cashflows"):
        """
        Export the DCF table to an Excel file, appending as a new sheet if the file exists.
        
        Parameters:
        - filename: str, file name or path for the Excel file (default: 'dcf_table.xlsx').
               If it's just a name (no directory), the file is saved to the repo's 'data/' folder.
               If it's a path (e.g., '~/Downloads/out.xlsx' or 'reports/out.xlsx'), that path is used.
        - table: str, which table to export ('cashflows' or 'budgets').
        """
        # --- Validate inputs ---
        if table not in {"cashflows", "budgets"}:
            raise ValueError("table must be 'cashflows' or 'budgets'")
        
        filename = os.path.expanduser(filename)
        if os.path.dirname(filename):  # user provided a directory or path
            filepath = filename
        else:
            BASE_DIR = os.getcwd()                   # works well for notebooks run from repo root
            DATA_FOLDER = os.path.join(BASE_DIR, "data")
            os.makedirs(DATA_FOLDER, exist_ok=True)
            filepath = os.path.join(DATA_FOLDER, filename)

        # Ensure the file has the correct extension
        if not filepath.endswith('.xlsx'):
            filepath += '.xlsx'

        # Generate the DCF table
        if table == "cashflows":
            dcf_table = self.generate_dcf_table()
        elif table == "budgets":
            dcf_table = self.generate_farm_budget_table()

        # Determine the sheet name
        if self.system_type == 'APV PV':
            sheet_name = 'DCF Analysis APV PV'
        elif self.system_type == 'APV farm':
            if table == "cashflows":
                sheet_name = 'DCF Analysis APV Farm'
            elif table == "budgets":
                sheet_name = 'Farm APV Budget'
        elif self.system_type == 'conventional PV':
            sheet_name = 'DCF Analysis Conventional PV'
        elif self.system_type == 'conventional farm':
            if table == "cashflows":
                sheet_name = 'DCF Analysis Conventional Farm'
            elif table == "budgets":
                sheet_name = 'Farm Conventional Budget'
        else:
            sheet_name = 'DCF Analysis'

        # --- Write/append, replacing sheet if it already exists ---
        # try:
        #     mode = "a" if os.path.exists(filepath) else "w"
        #     with pd.ExcelWriter(filepath, engine="openpyxl", mode=mode, if_sheet_exists="replace") as writer:
        #         dcf_table.to_excel(writer, sheet_name=sheet_name, index=True)

        #     print(f"Table successfully exported to {filepath} under sheet '{sheet_name}'.")
        # except Exception as e:
        #     print(f"An error occurred while exporting to Excel: {e}")

        try:
            if os.path.exists(filepath):
                # Load the existing workbook and append the sheet
                with pd.ExcelWriter(filepath, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                    # Check for duplicate sheet names
                    existing_workbook = load_workbook(filepath)
                    if sheet_name in existing_workbook.sheetnames:
                        del existing_workbook[sheet_name]  # Remove the existing sheet
                    dcf_table.to_excel(writer, sheet_name=sheet_name, index=True)
            else:
                # Create a new workbook if the file does not exist
                with pd.ExcelWriter(filepath, engine='openpyxl') as writer:
                    dcf_table.to_excel(writer, sheet_name=sheet_name, index=True)

            print(f"Table successfully exported to {filepath} under sheet '{sheet_name}'.")
        except Exception as e:
            print(f"An error occurred while exporting to Excel: {e}")

#IRR calculation and plotting of NPV vs discount rate

    def get_irr(self):
        """
        Manually calculates the Internal Rate of Return (IRR) by finding the discount rate 
        that makes the Net Present Value (NPV) of cash flows equal to zero.
        
        Returns:
        - irr: The approximate internal rate of return.
        """
        # Initial guess range for IRR
        low_rate = 0  # Starting from -50% for exploration of potential IRR
        high_rate = 3.0  # Ending at 300% as an upper bound for IRR
        tolerance = 1e-3  # Tolerance level for convergence
        max_iterations = 1000  # To prevent infinite loops
        years = self.lifetime.years
       
        def npv_with_irr(rate):
            if self.system_type in ('APV PV', 'conventional PV'):
                self.financial_rates.wacc = rate
            elif self.system_type in ('APV farm', 'conventional farm'):
                self.financial_rates.nominal_discount_rate = rate
            # Compute NPV with the current discount rate
            npv_value = self.get_npv()
            
            return npv_value

        # Perform a binary search to find the IRR
        original_wacc = self.financial_rates.wacc
        original_nominal_disc_rate = self.financial_rates.nominal_discount_rate
        for _ in range(max_iterations):

            mid_rate = (low_rate + high_rate) / 2  # Midpoint of the current rate range
            
            npv_mid = npv_with_irr(mid_rate)  # NPV at the midpoint 
        

            if abs(npv_mid) < tolerance:
                self.financial_rates.wacc = original_wacc
                self.financial_rates.nominal_discount_rate = original_nominal_disc_rate
                print(f"IRR found: {mid_rate:.4f} (or {mid_rate * 100:.2f}%)")
                if self.system_type in ('APV PV', 'conventional PV'):
                    print(f"Compared to wacc of {self.financial_rates.wacc:.4f} (or {self.financial_rates.wacc * 100:.2f}%)")
                elif self.system_type in ('APV farm', 'conventional farm'):
                    print(f"Compared to nominal discount rate of {self.financial_rates.nominal_discount_rate:.4f} (or {self.financial_rates.nominal_discount_rate * 100:.2f}%)")
                return mid_rate  # Return mid_rate (in decimal not %) as IRR when NPV is close to zero a
            elif npv_mid > 0:
                low_rate = mid_rate  # Adjust lower bound if NPV is positive (rate is too low)
            else:
                high_rate = mid_rate  # Adjust upper bound if NPV is negative (rate is too high)

        
        self.financial_rates.wacc = original_wacc
        self.financial_rates.nominal_discount_rate = original_nominal_disc_rate
        # If no solution is found within max_iterations, raise an exception
        return "IRR calculation did not converge"

    def plot_npv_vs_discount_rate(self, rate_range=(0, 1), step=0.01, capacities=[], srec_incentives_duration=[]):
        """
        Plots NPV for a range of discount rates to visually show the point where NPV = 0 (IRR).

        Parameters:
        - rate_range: tuple, the range of discount rates to test (default is from 0 to 100%).
        - step: float, the increment for discount rates within the range.
        - capacities: list, optional, power capacities to calculate and plot NPV curves for (default is empty list).
        - srec_incentive_duration: list, optional, SREC incentive durations to calculate and plot NPV PV curves for (default is empty list).
        """
        # Set font properties
        font_properties = {'fontname': 'Arial', 'fontsize': 25}

        # Helper function to calculate NPV with a given discount rate
        def npv_with_discount_rates(rate):
            
            # Select the correct rate
            if self.system_type in ('APV PV', 'conventional PV'):
                self.financial_rates.wacc = rate
            elif self.system_type in ('APV farm', 'conventional farm'):
                self.financial_rates.nominal_discount_rate = rate
            # Compute NPV with the current discount rate
            if self.system_type == 'APV PV':
                npv_value = self.get_npv()  # NPV for PV business
            elif self.system_type == 'APV farm':    
                npv_value = self.get_npv()
            return npv_value
        
        # If multiple srec_incentive_durations are provided, handle them
        if srec_incentives_duration:
            colors = plt.cm.coolwarm(np.linspace(0, 1, len(capacities)))  # Generate distinct colors for each capacity
            line_styles = ['-', '--', '-.', ':', '-']  # Generate distinct line styles for each duration
            plt.figure(figsize=(8, 8))

            # Store the original power capacity, srec duration and discount rate to be able to restore it after plotting
            original_power = self.conventional_economics.conventionalPVpower
            original_srec_duration = self.financial_rates.srec_duration
            original_wacc = self.financial_rates.wacc
            original_nominal_disc_rate = self.financial_rates.nominal_discount_rate

            for i, capacity in enumerate(capacities):
                for j, duration in enumerate(srec_incentives_duration):
                    # Temporarily update the power capacity and srec duration
                    self.pv_design.power = capacity
                    self.financial_rates.srec_duration = duration
                    # Recalculate all dependent variables for the updated capacity
                    self.pv_design.derived_variables(self.pv_design.rowspacing)
                    self.pv_design.get_capex_opex()
                    # Generate NPV values for each capacity and duration but only for PV business
                    npv_pv_values = []
                    discount_rates = np.arange(rate_range[0], rate_range[1] + step, step)
                    for rate in discount_rates:
                        npv_pv = npv_with_discount_rates(rate)
                        print(f"NPV for {capacity} kW and {duration} years at {rate * 100:.2f}% discount rate: {npv_pv:.2f}")
                        npv_pv_values.append(npv_pv)
                    # Plot NPV vs. Discount Rate for the current capacity and duration
                    plt.plot(discount_rates * 100, npv_pv_values, linestyle=line_styles[j], color=colors[i], linewidth=2, label=f'NPV PV - {capacity} kW, {duration} years')

            # Restore the original power capacity, srec duration and discount rate
            self.financial_rates.wacc = original_wacc
            self.financial_rates.nominal_discount_rate = original_nominal_disc_rate
            self.financial_rates.srec_duration = original_srec_duration
            self.pv_design.power = original_power
            # Recalculate all dependent variables for the restored capacity
            self.pv_design.derived_variables(self.conventional_economics.conventionalPVrowspacing) # Compute all derived variables based on the user inputs
            self.pv_design.adjust_row_spacing() # Adjust row spacing based on the user input
            self.pv_design.get_capex_opex() # Compute CAPEX and OPEX based on the user inputs
            
            plt.xlabel('Discount Rate (%)')
            plt.ylabel('Net Present Value ($M)')
            plt.ylim(-10000000, 40000000)
            plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '${:.1f}'.format(x / 1e6)))
            plt.title('NPV vs. Discount Rate for Multiple Capacities and SREC Durations')
            plt.legend()
            plt.grid(True)
            plt.show()
            return  # Exit after plotting for multiple capacities 
            

        
        # If multiple capacities are provided, handle them 
        if capacities:
            colors = plt.cm.viridis(np.linspace(0, 1, len(capacities)))  # Generate distinct colors for each capacity
            plt.figure(figsize=(8, 8))

            # Store the original power capacity and discount rate to be able to restore it after plotting
            #original_power = self.pv_design.power
            original_power = self.conventional_economics.conventionalPVpower
            original_wacc = self.financial_rates.wacc
            original_nominal_disc_rate = self.financial_rates.nominal_discount_rate

            for i, capacity in enumerate(capacities):
                print("Rowspasing before: ", self.pv_design.rowspacing)
                print("Ground CLearance before: ", self.pv_design.ground_clearance)
                print("Total arrays before: ", self.pv_design.totalarrays)
                print("area pv before:", self.pv_design.array_length * self.pv_design.array_width * self.pv_design.totalarrays * 0.0000229568)
                print("area farm before:", self.pv_design.get_area_farm())
                print("total area before: ", self.pv_design.totalarea)
                # Temporarily update the power capacity
                self.pv_design.power = capacity
                # Recalculate all dependent variables for the updated capacity
                self.pv_design.derived_variables(self.pv_design.rowspacing) # Compute all derived variables based on the user inputs
                # Do not readjust rowspacing: when we change the power capacity, we don't want to change the row spacing
                self.pv_design.get_capex_opex() # Compute CAPEX and OPEX based on the user inputs
                print("Rowspasing after: ", self.pv_design.rowspacing)
                print("Ground CLearance after: ", self.pv_design.ground_clearance)
                print("Total arrays after: ", self.pv_design.totalarrays)
                print("area pv after:", self.pv_design.array_length * self.pv_design.array_width * self.pv_design.totalarrays * 0.0000229568)
                print("area farm after:", self.pv_design.get_area_farm())
                print("total area after: ", self.pv_design.totalarea)

                # Generate NPV values for each capacity
                npv_pv_values = []
                npv_farm_values = []
                discount_rates = np.arange(rate_range[0], rate_range[1] + step, step)
                for rate in discount_rates:
                    npv_pv = npv_with_discount_rates(rate)
                    print(f"NPV for {capacity*0.001} MW at {rate * 100:.2f}% discount rate: {npv_pv:.2f}")
                    npv_farm = npv_with_discount_rates(rate)
                    npv_pv_values.append(npv_pv)
                    npv_farm_values.append(npv_farm)

                # Plot NPV vs. Discount Rate for the current capacity
                if self.system_type == 'APV PV':
                    plt.plot(discount_rates * 100, npv_pv_values, linestyle='-', color=colors[i], linewidth=2, label=f'PV - {capacity*0.001} MW')
                elif self.system_type == 'APV farm':
                    plt.plot(discount_rates * 100, npv_farm_values, linestyle='--', color=colors[i], linewidth=2, label=f'Farm - {capacity*0.001} MW')

                
            # Restore the original power capacity, discount rate and the default PV design
            self.financial_rates.wacc = original_wacc
            self.financial_rates.nominal_discount_rate = original_nominal_disc_rate
            self.pv_design.power = original_power
            # Recalculate all dependent variables for the restored capacity
            self.pv_design.derived_variables(self.conventional_economics.conventionalPVrowspacing) # Compute all derived variables based on the user inputs
            self.pv_design.adjust_row_spacing() # Adjust row spacing based on the user input
            self.pv_design.get_capex_opex() # Compute CAPEX and OPEX based on the user inputs
            print("Rowspasing final restored: ", self.pv_design.rowspacing)
            print("Ground CLearance final restored: ", self.pv_design.ground_clearance)
            print("Total arrays final restored: ", self.pv_design.totalarrays)
            print("area pv after restored:", self.pv_design.array_length * self.pv_design.array_width * self.pv_design.totalarrays * 0.0000229568)
            print("area farm after restored:", self.pv_design.get_area_farm())
            print("total area after restored: ", self.pv_design.totalarea)
            print("npv restored:", self.get_npv())

            # Define your interval 
            interval = step
            xticks = np.arange(rate_range[0]*100, (rate_range[1]+interval)*100, interval*100)  # Generate tick positions

            plt.xlabel('Discount Rate (%)', **font_properties)
            plt.ylabel('NPV ($M)', **font_properties)
            plt.ylim(-5000000, 30000000)
            plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '${:.1f}'.format(x / 1e6)))
            plt.legend(prop={'family': 'Arial', 'size': 15})
            plt.xticks(xticks, fontname='Arial', fontsize=25)
            plt.yticks(fontname='Arial', fontsize=25)
            plt.grid(True)
            plt.show()
            return  # Exit after plotting for multiple capacities           


        
        # If no capacities are provided, nor srec_incentive_durations, default case when using the model for a single capacity
        
        # Generate discount rates from the specified range
        npv_pv_values = []
        npv_farm_values = []
        discount_rates = np.arange(rate_range[0], rate_range[1] + step, step)
        original_wacc = self.financial_rates.wacc
        original_nominal_disc_rate = self.financial_rates.nominal_discount_rate  
        
        for rate in discount_rates:
            # Calculate NPV for each discount rate
            npv_pv = npv_with_discount_rates(rate)
            print(f"NPV for {rate * 100:.2f}% discount rate: {npv_pv:.2f}")
            npv_farm = npv_with_discount_rates(rate)
            npv_pv_values.append(npv_pv)
            npv_farm_values.append(npv_farm)

        # Restore the original discount rate
        self.financial_rates.wacc = original_wacc
        self.financial_rates.nominal_discount_rate = original_nominal_disc_rate

        # Plot NPV vs. Discount Rate
        plt.figure(figsize=(7, 7))

        # PV NPV plot
        if self.system_type == 'APV PV':
            plt.plot(discount_rates*100, npv_pv_values, color="skyblue", linewidth=2, label='PV')
            plt.fill_between(discount_rates, npv_pv_values, 0, where=(np.array(npv_pv_values) >= 0), color="skyblue", alpha=0.5)

        #farm NPV plot
        elif self.system_type == 'APV farm':
            plt.plot(discount_rates*100, npv_farm_values, color="orange", linewidth=2, label='Farm')
            plt.fill_between(discount_rates, npv_farm_values, 0, where=(np.array(npv_farm_values) >= 0), color="orange", alpha=0.5)

        plt.xlabel('Discount Rate (%)', **font_properties)
        plt.ylabel('NPV ($)', **font_properties)
        plt.ylim(-5000000, 30000000)
        plt.title('NPV vs. Discount Rate')
        plt.legend(prop={'family': 'Arial', 'size': 12})
        plt.xticks(fontname='Arial', fontsize=25)
        plt.yticks(fontname='Arial', fontsize=25)
        plt.grid(True)
        plt.show()

# Optimal farmer payment

# Partnership A: Extra Payment
    def get_optimal_extra_farmer_payment(self):
        """
        Compute the optimal complementary payment from the solar developer (SD) to the farmer .

        This method calculates the range [min_value, max_value] such that:
            1. NPV from the farm business is >= 0.
            2. Payment ($/W) < maintenance costs ($/W).

        Print the computed min and max values if applicable.
        """
        original_farmer_payment = self.partnership.extra_payment_sd_to_farmer #$/acre
        maintenance_costs = self.pv_design.maintenance_costs  # $/acre
        
        def npv_with_payment(payment):
            """Calculate NPV for farm business with a given farmer payment."""
            self.partnership.extra_payment_sd_to_farmer = payment
            npv_value = self.get_npv()
            return npv_value

        # Binary search for min farmer payment where NPV becomes non-negative
        low, high = 0, 4000
        tolerance = 1e-6
        max_iterations = 1000

        for _ in range(max_iterations):
            mid_payment = (low + high) / 2
            npv_mid = npv_with_payment(mid_payment)
            if abs(npv_mid) < tolerance:
                # Calculate npv PV to show how much when the npv farm = 0
                npv_value_PV = self.get_npv()
                # Restore the original Extra payment
                self.partnership.extra_payment_sd_to_farmer = original_farmer_payment
                # Print the results
                print(f"Breakeven Extra payment found: ${mid_payment:.4f}/acre with NPV (Farm) APV ~ 0.")
                print(f"Difference from the initial Extra payment: ${mid_payment - self.partnership.extra_payment_sd_to_farmer:.4f}/acre")
                print(f"NPV (PV) when NPV (Farm) is at breakeven: {npv_value_PV:.2f}")
                return mid_payment
            elif npv_mid > 0:
                high = mid_payment
            else:
                low = mid_payment

        self.partnership.extra_payment_sd_to_farmer = original_farmer_payment
        return "Breakeven Land lease payment calculation did not converge."

    def plot_npv_vs_extra_farmer_payment(self, payment_range=(0, 0.1), step=0.005):
        """
        Plots NPV for both the farm and PV businesses for a range of extra payments (from SD to farmer) in case Parternship A
        to visually show the impact on NPV for both businesses on the same graph.
        
        Parameters:
        - payment_range: tuple, the range of farmer payment rates to test (default is from 0 to 0.1 $/W).
        - step: float, the increment for farmer payment rates within the range.
        """
        
        # Store the original farmer payment rate to restore it after plotting
        original_payment = self.partnership.extra_payment_sd_to_farmer
        
        # Generate farmer payment rates within the specified range
        payment_rates = np.arange(payment_range[0], payment_range[1] + step, step)
        npv_farm_values = []
        npv_pv_values = []

        for payment in payment_rates:
            # Temporarily set the farmer payment rate
            self.partnership.extra_payment_sd_to_farmer = payment
            
            # Calculate NPV for both the farm and PV business with the current farmer payment
            npv_farm = self.get_npv()
            print(f"NPV Farm for farmer payment of ${payment:.4f}/acre: {npv_farm:.2f}")
            npv_pv = self.get_npv()
            print(f"NPV PV for farmer payment of ${payment:.4f}/acre: {npv_pv:.2f}")
            npv_farm_values.append(npv_farm)
            npv_pv_values.append(npv_pv)


        # Restore the original farmer payment rate
        self.partnership.extra_payment_sd_to_farmer = original_payment

        # Plot NPV vs Farmer Payment Rate for both businesses
        plt.figure(figsize=(8, 8))

        # Farm NPV plot
        if self.system_type == 'APV farm':
            plt.plot(payment_rates, npv_farm_values, color="orange", linewidth=2, label='NPV Farm')
            plt.fill_between(payment_rates, npv_farm_values, 0, where=(np.array(npv_farm_values) >= 0), color="orange", alpha=0.5)
        
        # PV NPV plot
        elif self.system_type == 'APV PV':
            plt.plot(payment_rates, npv_pv_values, color="skyblue", linewidth=2, label='NPV PV')
            plt.fill_between(payment_rates, npv_pv_values, 0, where=(np.array(npv_pv_values) >= 0), color="skyblue", alpha=0.5)

        # Formatting the plot
        plt.xlabel('Extra Payment (SD to farmer) ($/acre)')
        plt.ylabel('Net Present Value ($M)')
        plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '${:.1f}'.format(x / 1e6)))
        plt.title('NPV vs Extra Payment (PV & Farm) for Partnership A')
        plt.legend()
        plt.grid(True)
        plt.show()

# Partnership B: Land Lease Payment (from SD to farmer)

    def get_optimal_land_lease_payment(self):
        """
        Compute the optimal land lease payment from the SD to the farmer.

        This method calculates the range [min_value, max_value] such that:
            1. NPV from the farm business is >= 0.

        Print the computed min and max values if applicable.
        """
        original_land_lease_payment = self.partnership.land_lease_sd_to_farmer #$/acre
        
        def npv_with_payment(payment):
            """Calculate NPV for farm business with a given farmer payment."""
            self.partnership.land_lease_sd_to_farmer = payment
            npv_value = self.get_npv()
            return npv_value

        # Binary search for min farmer payment where NPV becomes non-negative
        low, high = 0, 3000
        tolerance = 1e-6
        max_iterations = 1000

        for _ in range(max_iterations):
            mid_payment = (low + high) / 2
            npv_mid = npv_with_payment(mid_payment)
            if abs(npv_mid) < tolerance:
                self.partnership.land_lease_sd_to_farmer = original_land_lease_payment
                print(f"Breakeven Land lease payment found: ${mid_payment:.4f}/acre with NPV Farm APV ~ 0.")
                print(f"Difference from the initial Land lease payment: ${mid_payment - self.partnership.land_lease_sd_to_farmer:.4f}/acre")
                return mid_payment
            elif npv_mid > 0:
                high = mid_payment
            else:
                low = mid_payment

        self.partnership.land_lease_sd_to_farmer = original_land_lease_payment
        return "Breakeven Land lease payment calculation did not converge."
    
    def plot_npv_vs_land_lease_payment(self, payment_range=(0, 4000), step=100):
        """
        Plots NPV for both the farm and PV businesses for a range of lease payment (from SD to farmer) in case Parternship B
        to visually show the impact on NPV for both businesses on the same graph.
        
        Parameters:
        - payment_range: tuple, the range of land lease payments to test (default is from 0 to 4000 $/acres).
        - step: float, the increment for land lease payments within the range.
        """

        # Set font properties
        font_properties = {'fontname': 'Arial', 'fontsize': 20}
        
        # Store the original land lease payment rate to restore it after plotting
        original_land_lease_payment = self.partnership.land_lease_sd_to_farmer
        
        # Generate farmer payment rates within the specified range
        payment_rates = np.arange(payment_range[0], payment_range[1] + step, step)
        npv_farm_values = []
        npv_pv_values = []

        for payment in payment_rates:
            # Temporarily set the land lease payment rate
            self.partnership.land_lease_sd_to_farmer = payment
            
            # Calculate NPV for both the farm and PV businesses with the current land lease payment
            if self.system_type == 'APV farm':
                npv_farm = self.get_npv()
                npv_farm_values.append(npv_farm)
                print(f"NPV for farmer payment of ${payment:.4f}/acre: {npv_farm:.2f}")
            elif self.system_type == 'APV PV':
                npv_pv = self.get_npv()
                npv_pv_values.append(npv_pv)
                print(f"NPV for farmer payment of ${payment:.4f}/acre: {npv_pv:.2f}")

        # Restore the original land lease payment rate
        self.partnership.land_lease_sd_to_farmer = original_land_lease_payment

        # Plot NPV vs Farmer Payment Rate for both businesses
        plt.figure(figsize=(8, 8))

        # Farm NPV plot
        if self.system_type == 'APV farm':
            plt.plot(payment_rates, npv_farm_values, linewidth=3, color="orange", label='Farm business')
        
        # PV NPV plot
        elif self.system_type == 'APV PV':
            plt.plot(payment_rates, npv_pv_values, linewidth=3, color="skyblue", label='PV business')

        # Control NPV horizontal line

        # Force the system type to be 'conventional PV' and 'conventional farm' to get the control NPV values
        original_system_type = self.system_type
        self.system_type = 'conventional PV'
        npv_control_pv = self.get_npv()
        self.system_type = 'conventional farm'
        npv_control_farm = self.get_npv()
        self.system_type = original_system_type

        plt.axhline(y=npv_control_pv, linewidth=3, color='skyblue', linestyle='--', label='Control PV business')
        plt.axhline(y=npv_control_farm, linewidth=3, color='orange', linestyle='--', label='Control farm business')

        # Formatting the plot
        interval = 1000
        xticks = np.arange(payment_range[0], payment_range[1]+interval, interval)


        plt.xlabel('Lease Payment (from PV to Farm) ($/acre)', **font_properties)
        plt.ylabel('NPV ($M)', **font_properties)
        plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '${:.1f}'.format(x / 1e6)))
        plt.legend(prop={'family': 'Arial', 'size': 15})
        plt.xticks(xticks, fontname='Arial', fontsize=22)
        plt.yticks(fontname='Arial', fontsize=22)
        plt.grid(True)
        plt.show()


# Breakeven calculations and plotting of NPV vs SREC incentives 
    def find_breakeven_srec_incentives(self):
        """
        Finds the minimum SREC incentives required to make NPV PV in APV case >= 0 if the current NPV is negative.
        
        Returns:
        - breakeven_srec_incentives (float): The SREC incentives ($/MWh) needed for a non-negative NPV.
        """
    
        # Set up initial variables for binary search
        low_incentives = 0
        high_incentives = self.financial_rates.srec * 5  # Set a high bound (e.g., 5 times the current incentives)
        tolerance = 1e-6  # Tolerance for NPV convergence
        max_iterations = 1000  # To prevent infinite loops

        # Define a function to calculate NPV with a given SREC incentives
        def npv_with_srec_incentives(incentives):
            self.financial_rates.srec = incentives
            npv_value = self.get_npv()
            return npv_value
        
        # Binary search for breakeven SREC incentives
        # Temporarily set the new SREC incentives
        original_incentives = self.financial_rates.srec
        for _ in range(max_iterations):
            mid_incentives = (low_incentives + high_incentives) / 2
            npv_mid = npv_with_srec_incentives(mid_incentives)

            if abs(npv_mid) < tolerance:
                # Restore the original SREC incentives after calculation
                self.financial_rates.srec = original_incentives
                print(f"Breakeven SREC incentives found: ${mid_incentives:.4f}/MWh with NPV PV APV ~ 0.")
                print(f"Difference from the initial SREC incentives: ${mid_incentives - self.financial_rates.srec:.4f}/MWh")
                return mid_incentives
            elif npv_mid > 0:
                high_incentives = mid_incentives
            else:
                low_incentives = mid_incentives

        # If no solution is found within max_iterations, raise an exception
        # Restore the original SREC incentives after calculation
        self.financial_rates.srec = original_incentives
        return "Breakeven SREC incentives calculation did not converge."
    
    def plot_npv_vs_srec_incentives(self, incentives_range=(0, 0.1), step=1, capacities=[]):
        """
        Plots NPV PV APV for a range of SREC incentives to visually show the point where NPV = 0 (breakeven).
        
        Parameters:
        - incentives_range: tuple, the range of SREC incentives to test (default is from 0 to 0.1 $/kWh).
        - step: float, the increment for SREC incentives within the range.
        - capacities: list, optional, power capacities to calculate and plot NPV curves for (default is empty list).
        """

        # Set font properties
        font_properties = {'fontname': 'Arial', 'fontsize': 25}
        
        # If multiple capacities are provided, handle them

        if capacities:
            colors = plt.cm.viridis(np.linspace(0, 1, len(capacities)))
            plt.figure(figsize=(8.7, 8.7))

            # Store the original power capacity and SREC incentives to be able to restore them after plotting
            #save the conventional power and not the apv power because you will restore the apv value by redoing the whole process once the breakeven is found
            original_power = self.conventional_economics.conventionalPVpower
            original_incentives = self.financial_rates.srec

            for i, capacity in enumerate(capacities):
                # Temporarily update the power capacity
                self.pv_design.power = capacity
                # Recalculate all dependent variables for the updated capacity
                self.pv_design.derived_variables(self.pv_design.rowspacing)
                self.pv_design.get_capex_opex()

                # Generate NPV values for each capacity
                npv_pv_values = []
                srec_incentives = np.arange(incentives_range[0], incentives_range[1]+ step, step)
                for incentives in srec_incentives:
                    # Temporarily set the SREC incentives
                    self.financial_rates.srec = incentives
                    # Calculate NPV with the current SREC incentives
                    npv_pv = self.get_npv()
                    print(f"NPV for {capacity*0.001} MW at {incentives:.2f} $/kWh SREC incentives: {npv_pv:.2f}")
                    npv_pv_values.append(npv_pv)

                # Plot NPV vs. SREC Incentives for the current capacity
                plt.plot(srec_incentives, npv_pv_values, color=colors[i], linewidth=2, label=f'{capacity*0.001} MW')

            # Restore the original power capacity and SREC incentives
            self.pv_design.power = original_power
            self.financial_rates.srec = original_incentives
            # Recalculate all dependent variables for the restored capacity
            self.pv_design.derived_variables(self.conventional_economics.conventionalPVrowspacing)
            self.pv_design.adjust_row_spacing()
            self.pv_design.get_capex_opex()

            # Define your interval 
            interval = step
            xticks = np.arange(incentives_range[0], incentives_range[1]+interval, interval)  # Generate tick positions

            plt.xlabel('Solar Renewable Energy Certificates ($/kWh)', **font_properties)
            plt.ylabel('NPV for PV Business ($M)', **font_properties)
            plt.ylim(-5000000, 30000000)
            plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '${:.1f}'.format(x / 1e6)))
            plt.legend(prop={'family': 'Arial', 'size': 20})
            plt.xticks(xticks, fontname='Arial', fontsize=25)
            plt.yticks(fontname='Arial', fontsize=25)
            plt.grid(True)
            plt.show()
            return  # Exit after plotting for multiple capacities
        
        # If no capacities are provided, default case when using the model for a single capacity

        # Generate SREC incentives from the specified range
        npv_pv_values = []
        srec_incentives = np.arange(incentives_range[0], incentives_range[1] + step, step)
        original_incentives = self.financial_rates.srec

        for incentives in srec_incentives:
            # Calculate NPV for each SREC incentives
            self.financial_rates.srec = incentives
            npv_pv = self.get_npv()
            print(f"NPV for {incentives:.2f} $/kWh SREC incentives: {npv_pv:.2f}")
            npv_pv_values.append(npv_pv)

        # Restore the original SREC incentives
        self.financial_rates.srec = original_incentives

        # Plot NPV vs. SREC Incentives
        plt.figure(figsize=(8.7, 8.7))
        plt.plot(srec_incentives, npv_pv_values, color='skyblue', linewidth=2)
        plt.fill_between(srec_incentives, npv_pv_values, 0, where=(np.array(npv_pv_values) >= 0), color='skyblue', alpha=0.5)
        plt.xlabel('SREC Incentives ($/kWh)', **font_properties)
        plt.ylabel('Net Present Value ($M)', **font_properties)
        plt.ylim(-5000000, 30000000)
        plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '${:.1f}'.format(x / 1e6)))
        plt.title('NPV PV vs Solar Renewable Energy Certificates')
        plt.legend(prop={'family': 'Arial', 'size': 15})
        plt.xticks(fontname='Arial', fontsize=25)
        plt.yticks(fontname='Arial', fontsize=25)
        plt.grid(True)
        plt.show()

# Breakeven calculations and plotting of NPV vs ITC

    def find_breakeven_itc(self):
        """
        Finds the minimum ITC required to make NPV PV in APV case >= 0 if the current NPV is negative.
        
        Returns:
        - breakeven_itc (float): The ITC (%) needed for a non-negative NPV.
        """

        # Set up initial variables for binary search
        low_itc = 0
        high_itc = self.financial_rates.itc_rate * 2  # Set a high bound (e.g., 2 times the current ITC)
        tolerance = 1e-6  # Tolerance for NPV convergence
        max_iterations = 1000  # To prevent infinite loops

        # Define a function to calculate NPV with a given ITC
        def npv_with_itc(itc):
            self.financial_rates.itc_rate = itc
            npv_value = self.get_npv()
            return npv_value

        # Binary search for breakeven ITC
        # Temporarily set the new ITC
        original_itc = self.financial_rates.itc_rate
        for _ in range(max_iterations):
            mid_itc = (low_itc + high_itc) / 2
            npv_mid = npv_with_itc(mid_itc)

            if abs(npv_mid) < tolerance:
                # Restore the original ITC after calculation
                self.financial_rates.itc_rate = original_itc
                print(f"Breakeven ITC found: {mid_itc:.4f}% with NPV PV APV ~ 0.")
                print(f"Difference from the initial ITC: {mid_itc - original_itc:.4f}%")
                return mid_itc  # Return the ITC when NPV is close to zero
            elif npv_mid > 0:
                high_itc = mid_itc  # Decrease the upper bound if NPV is positive
            else:
                low_itc = mid_itc  # Increase the lower bound if NPV is negative

        # If no solution is found within max_iterations, raise an exception
        self.financial_rates.itc_rate = original_itc
        return "Breakeven ITC calculation did not converge."


    def plot_npv_vs_itc(self, itc_range=(0, 0.3), step=0.05, capacities=[]):
        """
        Plots NPV PV APV for a range of ITC rates to visually show the point where NPV = 0 (breakeven).
        
        Parameters:
        - itc_range: tuple, the range of ITC rates to test (default is from 0 to 0.3).
        - step: float, the increment for ITC rates within the range.
        - capacities: list, optional, power capacities to calculate and plot NPV curves for (default is empty list).
        """

        # Set font properties
        font_properties = {'fontname': 'Arial', 'fontsize': 25}
        
        # If multiple capacities are provided, handle them

        if capacities:
            colors = plt.cm.viridis(np.linspace(0, 1, len(capacities)))
            plt.figure(figsize=(8, 8))

            # Store the original power capacity and ITC to be able to restore them after plotting
            original_power = self.conventional_economics.conventionalPVpower
            original_itc = self.financial_rates.itc_rate

            for i, capacity in enumerate(capacities):
                # Temporarily update the power capacity
                self.pv_design.power = capacity
                # Recalculate all dependent variables for the updated capacity
                self.pv_design.derived_variables(self.pv_design.rowspacing)
                self.pv_design.get_capex_opex()

                # Generate NPV values for each capacity
                npv_pv_values = []
                itc_rates = np.arange(itc_range[0], itc_range[1] + step, step)
                for itc in itc_rates:
                    # Temporarily set the ITC
                    self.financial_rates.itc_rate = itc
                    # Calculate NPV with the current ITC
                    npv_pv = self.get_npv()
                    print(f"NPV for {capacity*0.001} MW at {itc:.2f} ITC: {npv_pv:.2f}")
                    npv_pv_values.append(npv_pv)

                # Plot NPV vs. ITC for the current capacity
                plt.plot(itc_rates * 100, npv_pv_values, color=colors[i], linewidth=2, label=f'{capacity*0.001} MW')
            
            # Restore the original power capacity and ITC
            self.pv_design.power = original_power
            self.financial_rates.itc_rate = original_itc
            # Recalculate all dependent variables for the restored capacity
            self.pv_design.derived_variables(self.conventional_economics.conventionalPVrowspacing)
            self.pv_design.adjust_row_spacing()
            self.pv_design.get_capex_opex()
            # Define your interval
            interval = step
            xticks = np.arange(itc_range[0]*100, (itc_range[1] + interval)*100, interval*100)  # Generate tick positions

            plt.xlabel('Investment Tax Credit (ITC) (%)', **font_properties)
            plt.ylabel('NPV for PV Business ($M)', **font_properties)
            plt.ylim(0, 16000000)
            plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '${:.1f}'.format(x / 1e6)))
            plt.legend(prop={'family': 'Arial', 'size': 15})
            plt.xticks(xticks, fontname='Arial', fontsize=25)
            plt.yticks(fontname='Arial', fontsize=25)
            plt.grid(True)
            plt.show()
            return  # Exit after plotting for multiple capacities

        # If no capacities are provided, default case when using the model for a single capacity
        # Generate ITC rates from the specified range
        npv_pv_values = []
        itc_rates = np.arange(itc_range[0], itc_range[1] + step, step)
        original_itc = self.financial_rates.itc_rate

        for itc in itc_rates:
            # Temporarily set the ITC
            self.financial_rates.itc_rate = itc
            # Calculate NPV with the current ITC
            npv_pv = self.get_npv()
            print(f"NPV for {self.pv_design.power * 0.001} MW at {itc:.2f} ITC: {npv_pv:.2f}")
            npv_pv_values.append(npv_pv)

        # Restore the original ITC
        self.financial_rates.itc_rate = original_itc
        # Plot NPV vs. ITC
        plt.figure(figsize=(8, 8))
        plt.plot(itc_rates * 100, npv_pv_values, color='skyblue', linewidth=2)
        plt.fill_between(itc_rates * 100, npv_pv_values, 0, where=(np.array(npv_pv_values) >= 0), color='skyblue', alpha=0.5)
        plt.xlabel('Investment Tax Credit (ITC) (%)', **font_properties)
        plt.ylabel('Net Present Value ($M)', **font_properties)
        plt.ylim(-1000000, 16000000)
        plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '${:.1f}'.format(x / 1e6)))
        plt.title('NPV PV vs Investment Tax Credit (ITC)')
        plt.legend(prop={'family': 'Arial', 'size': 15})
        plt.xticks(fontname='Arial', fontsize=25)
        plt.yticks(fontname='Arial', fontsize=25)
        plt.grid(True)
        plt.show()  
        
# Breakeven calculations and plotting of NPV vs energy price or vs crop price

    def find_breakeven_energy_price(self):
        """
        Finds the minimum base energy price required to make NPV PV in APV case >= 0 if the current NPV is negative.
        
        Returns:
        - breakeven_energy_price (float): The energy price ($/kWh) needed for a non-negative NPV.
        """

        # Set up initial variables for binary search
        low_price = 0  # Set a reasonable lower bound for energy price
        high_price = self.pv_design.energyprice * 5  # Set a high bound (e.g., 5 times the current price)
        tolerance = 1e-6  # Tolerance for NPV convergence
        max_iterations = 1000  # To prevent infinite loops

        # Define a function to calculate NPV with a given energy price
        def npv_with_energy_price(energy_price):
            self.pv_design.energyprice = energy_price
            npv_value = self.get_npv()
            
            return npv_value

        # Binary search for breakeven energy price
        # Temporarily set the new energy price
        original_price = self.pv_design.energyprice
        for _ in range(max_iterations):
            mid_price = (low_price + high_price) / 2
            npv_mid = npv_with_energy_price(mid_price)

            if abs(npv_mid) < tolerance:
                # Restore the original energy price after calculation
                self.pv_design.energyprice = original_price
                print(f"Breakeven base energy price found: ${mid_price:.4f}/kWh with NPV PV APV ~ 0.")
                print(f"Difference from the initial base price: ${mid_price - self.pv_design.energyprice:.4f}/kWh")
                return mid_price  # Return the price when NPV is close to zero
            elif npv_mid > 0:
                high_price = mid_price  # Decrease the upper bound if NPV is positive
            else:
                low_price = mid_price  # Increase the lower bound if NPV is negative

        # If no solution is found within max_iterations, raise an exception
        # Restore the original energy price after calculation
        self.pv_design.energyprice = original_price
        return "Breakeven base energy price calculation did not converge."
    
    def plot_npv_vs_energy_price(self, price_range=(0, 1), step=0.01, capacities=[]):
        """
        Plots NPV PV APV for a range of base energy prices to visually show the point where NPV = 0 (breakeven).
        
        Parameters:
        - price_range: tuple, the range of energy prices to test (default is from 0 to 1 $/kWh).
        - step: float, the increment for energy prices within the range.
        - capacities: list, optional, power capacities to calculate and plot NPV curves for (default is empty list).
        """

        # Set font properties
        font_properties = {'fontname': 'Arial', 'fontsize': 25}
        
        # If multiple capacities are provided, handle them

        if capacities:
            colors = plt.cm.viridis(np.linspace(0, 1, len(capacities)))  # Generate distinct colors for each capacity
            plt.figure(figsize=(8, 8))

            # Store the original power capacity and energy price to be able to restore it after plotting
            #original_power = self.pv_design.power
            original_power = self.conventional_economics.conventionalPVpower
            original_price = self.pv_design.energyprice

            for i, capacity in enumerate(capacities):
                # Temporarily update the power capacity
                self.pv_design.power = capacity
                # Recalculate all dependent variables for the updated capacity
                self.pv_design.derived_variables(self.pv_design.rowspacing) # Compute all derived variables based on the user inputs
                self.pv_design.get_capex_opex() # Compute CAPEX and OPEX based on the user inputs

                # Generate NPV values for each capacity
                npv_pv_values = []
                energy_prices = np.arange(price_range[0], price_range[1], step)
                for price in energy_prices:
                    # Temporarily set the energy price
                    self.pv_design.energyprice = price
                    # Calculate NPV with the current energy price
                    npv_pv = self.get_npv()
                    print(f"NPV for {capacity*0.001} MW at {price:.2f} $/kWh base energy price: {npv_pv:.2f}")
                    npv_pv_values.append(npv_pv)
                
                # Plot NPV vs. Energy Price for the current capacity
                plt.plot(energy_prices, npv_pv_values, color=colors[i], linewidth=2)

            # Restore the original power capacity and energy price
            self.pv_design.power = original_power
            self.pv_design.energyprice = original_price
            # Recalculate all dependent variables for the restored capacity
            self.pv_design.derived_variables(self.conventional_economics.conventionalPVrowspacing) # Compute all derived variables based on the user inputs
            self.pv_design.adjust_row_spacing() # Adjust row spacing based on the user input
            self.pv_design.get_capex_opex() # Compute CAPEX and OPEX based on the user inputs

            interval = step
            xticks = np.arange(price_range[0], price_range[1], interval)  # Generate tick positions

            plt.xlabel('Power Purchase Agreement ($/kWh)', **font_properties)
            plt.ylabel('NPV for PV Business ($M)', **font_properties)
            plt.ylim(-5000000, 30000000)
            plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '${:.1f}'.format(x / 1e6)))
            plt.xticks(xticks, fontname='Arial', fontsize=25)
            plt.yticks(fontname='Arial', fontsize=25)
            plt.grid(True)
            plt.show()
            return  # Exit after plotting for multiple capacities
        
        # If no capacities are provided, default case when using the model for a single capacity
        # Generate energy prices from the specified range
        energy_prices = np.arange(price_range[0], price_range[1] + step, step)
        npv_values = []

        # Store the original energy price to restore it after plotting
        original_price = self.pv_design.energyprice

        for price in energy_prices:
            # Temporarily set the energy price
            self.pv_design.energyprice = price
            # Calculate NPV with the current energy price
            npv = self.get_npv()
            print(f"NPV for {price:.2f} $/kWh base energy price: {npv:.2f}")
            npv_values.append(npv)

        # Restore the original energy price
        self.pv_design.energyprice = original_price

        # Plot NPV vs. Energy Price
        plt.figure(figsize=(8, 8))
        plt.plot(energy_prices, npv_values, color="skyblue", linewidth=2)
        plt.fill_between(energy_prices, npv_values, 0, where=(np.array(npv_values) >= 0), color="skyblue", alpha=0.5)
        plt.xlabel('PPA Price ($/kWh)', **font_properties)
        plt.ylabel('Net Present Value ($M)', **font_properties)
        plt.ylim(-10000000, 40000000)
        plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '${:.1f}'.format(x / 1e6)))
        plt.title('NPV PV vs. PPA Price')
        plt.legend(prop={'family': 'Arial', 'size': 15})
        plt.xticks(fontname='Arial', fontsize=25)
        plt.yticks(fontname='Arial', fontsize=25)
        plt.grid(True)
        plt.show()
        

    def find_breakeven_crop_price(self, crop_name):
        """
        Finds the minimum edible crop price required to make NPV farm APV >= 0 for a specific crop if the current NPV is negative.
        
        Parameters:
        - crop_name (str): The name of the crop for which the breakeven price should be calculated.

        Returns:
        - breakeven_crop_price (float): The edible crop price ($/t) needed for a non-negative NPV for the specified crop.
        """
        # Set up initial variables for binary search
        low_price = 0  # Set a reasonable lower bound for crop price
        high_price = self.conventional_economics.conventionalcropprice[crop_name] * 100  # Set an upper bound (e.g., 100 times the current price)
        tolerance = 1e-6  # Tolerance for NPV convergence
        max_iterations = 1000  # To prevent infinite loops

        # Define a function to calculate NPV with a given crop price
        def npv_with_crop_price(crop_price):
            # Temporarily set the new crop price
            original_price = self.conventional_economics.conventionalcropprice[crop_name]
            self.conventional_economics.conventionalcropprice[crop_name] = crop_price
            npv_value = self.get_npv()
            # Restore the original crop price after calculation
            self.conventional_economics.conventionalcropprice[crop_name] = original_price
            return npv_value

        # Binary search for breakeven crop price
        for _ in range(max_iterations):
            mid_price = (low_price + high_price) / 2
            npv_mid = npv_with_crop_price(mid_price)

            if abs(npv_mid) < tolerance:
                print(f"Breakeven crop price for {crop_name} found: ${mid_price:.4f}/t with NPV ~ 0.")
                print(f"Difference from original crop price: ${mid_price - self.conventional_economics.conventionalcropprice[crop_name]:.4f}/t")
                return mid_price  # Return the price when NPV is close to zero
            elif npv_mid > 0:
                high_price = mid_price  # Decrease the upper bound if NPV is positive
            else:
                low_price = mid_price  # Increase the lower bound if NPV is negative

        # If no solution is found within max_iterations, raise an exception
        return "Breakeven crop price calculation did not converge."

    def plot_npv_vs_all_crop_prices(self, price_range=(0, 3000), step=100, capacities=[]):
        """
        Plots NPV variation for each crop price within a specified range to visualize breakeven points on the same graph.
        
        Parameters:
        - price_range: tuple, the range of crop prices to test (default is from 0 to 3000 $/t).
        - step: float, the increment for crop prices within the range.
        - capacities: list, optional, power capacities to calculate and plot NPV curves for (default is empty list).
        """

        # Set font properties
        font_properties = {'fontname': 'Arial', 'fontsize': 25}

        # Select the crops for which to plot NPV vs. crop price
        selected_crops = self.cropschedule.selected_crops
        
        if not selected_crops:
            print("No crops selected. Cannot plot NPV vs. crop prices.")
            return


        # If multiple capacities are provided, handle them
        if capacities:
            colors = plt.cm.viridis(np.linspace(0, 1, len(capacities)))
            line_styles = ['--', '-', ':', '-.']  # Unique style per crop
            plt.figure(figsize=(8, 8))

            # Store the original power capacity and crop prices to be able to restore them after plotting
            #original_power = self.pv_design.power
            original_power = self.conventional_economics.conventionalPVpower
            original_prices = {crop: self.conventional_economics.conventionalcropprice[crop] for crop in selected_crops}

            for i, capacity in enumerate(capacities):
                # Temporarily update the power capacity
                self.pv_design.power = capacity
                # Recalculate all dependent variables for the updated capacity
                self.pv_design.derived_variables(self.pv_design.rowspacing)
                self.pv_design.get_capex_opex()

                # Generate NPV values for each crop price
                for j, crop in enumerate(selected_crops):
                    crop_prices = np.arange(price_range[0], price_range[1] + step, step)
                    npv_values = []

                    for price in crop_prices:
                        # Temporarily set the crop price
                        self.conventional_economics.conventionalcropprice[crop] = price
                        # Calculate NPV with the current crop price
                        npv = self.get_npv()
                        print(f"NPV for {crop} at {price:.2f} $/t and {capacity*0.001} MW: {npv:.2f}")
                        npv_values.append(npv)

                    # Restore the original crop price
                    self.conventional_economics.conventionalcropprice[crop] = original_prices[crop]

                    # Plot NPV vs. Crop Price for the current crop and capacity
                    color = colors[i]
                    line_style = line_styles[j % len(line_styles)]
                    plt.plot(crop_prices, npv_values, color=color, linestyle=line_style, linewidth=2)

            # Restore the original power capacity and crop prices
            self.pv_design.power = original_power
            # Recalculate all dependent variables for the restored capacity
            self.pv_design.derived_variables(self.conventional_economics.conventionalPVrowspacing)
            self.pv_design.adjust_row_spacing()
            self.pv_design.get_capex_opex()


            interval = step
            xticks = np.arange(price_range[0], price_range[1]+interval, interval)  # Generate tick positions

            plt.xlabel('Crop Price ($/t)', **font_properties)
            plt.ylabel('NPV for Farm Business ($M)', **font_properties)
            plt.ylim(-10000000, 10000000)
            plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '${:.1f}'.format(x / 1e6)))
            plt.xticks(xticks, fontname='Arial', fontsize=25)
            plt.yticks(fontname='Arial', fontsize=25)
            plt.grid(True)
            plt.show()
            return  # Exit after plotting for multiple capacities
        
        # If no capacities are provided, default case when using the model for a single capacity
        # Store original crop prices
        original_prices = {crop: self.conventional_economics.conventionalcropprice[crop] for crop in selected_crops}

        # Define a list of distinct colors to use for each crop
        color_list = ['orange', 'yellow','red']
        
        plt.figure(figsize=(8, 8))  # Set the figure size for a single combined plot

        for i, crop in enumerate(selected_crops):
            crop_prices = np.arange(price_range[0], price_range[1] + step, step)
            npv_values = []

            for price in crop_prices:
                # Temporarily set the crop price
                self.conventional_economics.conventionalcropprice[crop] = price
                # Calculate NPV with the current crop price
                npv = self.get_npv()
                print(f"NPV for {crop} at {price:.2f} $/t: {npv:.2f}")
                npv_values.append(npv)

            # Restore the original crop price
            self.conventional_economics.conventionalcropprice[crop] = original_prices[crop]

            # Plot NPV vs. Crop Price for the current crop
            curve_color = color_list[i % len(color_list)]
            plt.plot(crop_prices, npv_values, label=f'{crop} NPV', color=curve_color, linewidth=2)
            plt.fill_between(crop_prices, npv_values, 0, where=(np.array(npv_values) >= 0), color=curve_color, alpha=0.2)
        
        # Plot formatting
        plt.xlabel('Crop Price ($/t)', **font_properties)
        plt.ylabel('NPV for Farm Business ($M)', **font_properties)
        plt.ylim(-10000000, 10000000)
        plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '${:.1f}'.format(x / 1e6)))
        plt.title('NPV Farm vs. Crop Price for all Selected Crops')
        plt.legend(prop={'family': 'Arial', 'size': 15})
        plt.xticks(fontname='Arial', fontsize=25)
        plt.yticks(fontname='Arial', fontsize=25)
        plt.grid(True)
        plt.show()

# Breakeven calculations and plotting of NPV vs crop yield

    def find_breakeven_crop_yield(self, crop_name):
        """
        Finds the minimum crop yield required to make NPV farm APV >= 0 for a specific crop if the current NPV is negative.
        
        Parameters:
        - crop_name (str): The name of the crop for which the breakeven yield should be calculated.

        Returns:
        - breakeven_crop_yield (float): The crop yield (t/ha) needed for a non-negative NPV for the specified crop.
        """
        # Set up initial variables for binary search
        low_yield = 0
        high_yield = self.crop_yield.get_APV_crop_yield(crop_name) * 10  # Set an upper bound (e.g., 10 times the current yield)
        tolerance = 1e-6  # Tolerance for NPV convergence
        max_iterations = 1000  # To prevent infinite loops

        # Binary search for breakeven crop yield
        for _ in range(max_iterations):
            mid_yield = (low_yield + high_yield) / 2

            # Temporarily set the new crop yield
            self.crop_yield.set_temporary_yield(crop_name, mid_yield)

            # Calculate NPV with the updated crop yield
            npv_mid = self.get_npv()

            # Debugging line
            print(f"Testing yield: {mid_yield:.4f} t/acre, NPV: {npv_mid:.4f}")

            # Check for convergence
            if abs(npv_mid) < tolerance:
                self.crop_yield.reset_yields()
                print(f"Breakeven crop yield for {crop_name} found: {mid_yield:.4f} t/acre with NPV ~ 0.")
                print(f"Difference from the initial yield: {mid_yield - self.crop_yield.get_APV_crop_yield(crop_name):.4f} t/acre")
                return mid_yield
            elif npv_mid > 0:
                high_yield = mid_yield  # Adjust upper bound
            else:
                low_yield = mid_yield  # Adjust lower bound

        # Reset the crop yields after the calculation
        self.crop_yield.reset_yields()

        # If no solution is found within max_iterations
        print("Breakeven crop yield calculation did not converge.")
        return None
    
    def plot_npv_vs_all_crop_yields(self, yield_range=(0, 20), step=0.5, capacities=[]):
        """
        Plots NPV variation for a specific crop yield within a specified range to visualize breakeven points.
        
        Parameters:
        - crop_yield_instance: An instance of the CropYield class to retrieve crop yields dynamically.
        - yield_range: tuple, the range of crop yields to test (default is from 0 to 20 t/acre).
        - step: float, the increment for crop yields within the range.
        - capacities: list, optional, power capacities to calculate and plot NPV curves for (default is empty list).
        """
        # Set font properties
        font_properties = {'fontname': 'Arial', 'fontsize': 25}

        # Select the crops for which to plot NPV vs. crop yield
        selected_crops = self.cropschedule.selected_crops
        
        if not selected_crops:
            print("No crops selected. Cannot plot NPV vs. crop yields.")
            return

        # If multiple capacities are provided, handle them
        if capacities:
            colors = plt.cm.viridis(np.linspace(0, 1, len(capacities)))
            line_styles = ['--', '-', ':', '-.']  # Unique style per crop
            plt.figure(figsize=(8, 8))

            # Store the original power capacity to be able to restore them after plotting
            original_power = self.conventional_economics.conventionalPVpower

            for i, capacity in enumerate(capacities):
                # Temporarily update the power capacity
                self.pv_design.power = capacity
                # Recalculate all dependent variables for the updated capacity
                self.pv_design.derived_variables(self.pv_design.rowspacing)
                self.pv_design.get_capex_opex()

                # Generate NPV values for each crop yield
                for j, crop in enumerate(selected_crops):
                    crop_yields = np.arange(yield_range[0], yield_range[1] + step, step)
                    npv_values = []

                    for new_yield in crop_yields:
                        # Temporarily update the crop yield for the NPV calculation
                        # Temporarily set the new yield
                        self.crop_yield.set_temporary_yield(crop, new_yield)
                        
                        # Calculate NPV with the updated crop yield
                        npv = self.get_npv()
                        print(f"NPV for {crop} at {new_yield:.2f} t/acre and {capacity*0.001} MW: {npv:.2f}")
                        npv_values.append(npv)

                    # Restore the original crop yield
                    self.crop_yield.reset_yields()

                    # Plot NPV vs. Crop Yield for the current crop and capacity
                    color = colors[i]
                    line_style = line_styles[j % len(line_styles)]
                    plt.plot(crop_yields, npv_values, color=color, linestyle=line_style, linewidth=2)

            # Restore the original power capacity
            self.pv_design.power = original_power
            # Recalculate all dependent variables for the restored capacity
            self.pv_design.derived_variables(self.conventional_economics.conventionalPVrowspacing)
            self.pv_design.adjust_row_spacing()
            self.pv_design.get_capex_opex()

            interval = step
            xticks = np.arange(yield_range[0], yield_range[1]+interval, interval)  # Generate tick positions

            plt.xlabel('Crop Yield (t/acre)', **font_properties)
            plt.ylabel('NPV for Farm Business ($M)', **font_properties)
            plt.ylim(-10000000, 10000000)
            plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '${:.1f}'.format(x / 1e6)))
            plt.xticks(xticks, fontname='Arial', fontsize=25)
            plt.yticks(fontname='Arial', fontsize=25)
            plt.grid(True)
            plt.show()
            return  # Exit after plotting for multiple capacities

        # If no capacities are provided, default case when using the model for a single capacity
        # Define a list of distinct colors to use for each crop
        color_list = ['orange', 'yellow','red']

        plt.figure(figsize=(8, 8))  # Set the figure size for a single combined plot

        for i, crop in enumerate(selected_crops):
                crop_yields = np.arange(yield_range[0], yield_range[1] + step, step)
                npv_values = []
                
                for new_yield in crop_yields:
                    # Temporarily update the crop yield for the NPV calculation
                    # Temporarily set the new yield
                    self.crop_yield.set_temporary_yield(crop, new_yield)
                    
                    # Calculate NPV with the updated crop yield
                    npv = self.get_npv()
                    print(f"NPV for {crop} at {new_yield:.2f} t/acre: {npv:.2f}")
                    npv_values.append(npv)

                # Restore the original crop yield
                self.crop_yield.reset_yields()
                

                # Plot NPV vs. Crop Yield for the current crop
                curve_color = color_list[i % len(color_list)]
                plt.plot(crop_yields, npv_values, label=f'{crop} NPV', color = curve_color, linewidth=2)
            
        # Plot formatting
        plt.xlabel('Crop Yield (t/acre)', **font_properties)
        plt.ylabel('Net Present Value ($M)', **font_properties)
        plt.ylim(-10000000, 10000000)
        plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '${:.1f}'.format(x / 1e6)))
        plt.title('NPV Farm vs. Crop Yield for all Selected Crops')
        plt.legend(prop={'family': 'Arial', 'size': 15})
        plt.xticks(fontname='Arial', fontsize=25)
        plt.yticks(fontname='Arial', fontsize=25)
        plt.grid(True)
        plt.show()
        
# Breakeven calculation and plotting of NPV vs Farm Costs Change (scaling factor for APV crop costs compared to conventional costs)

    def find_breakeven_farm_costs_change(self):
        """
        Finds the minimum scaling factor (farm_costs_change) required to make NPV APV farm >= 0 if the current NPV is negative.
        
        Returns:
        - breakeven_farm_costs_change (float): The scaling factor needed for a non-negative NPV.
        """
        # Set up initial variables for binary search
        low_farm_costs_change = 0
        high_farm_costs_change = 10  # Set an upper bound for farm_costs_change
        tolerance = 1e-6  # Tolerance for NPV convergence
        max_iterations = 1000  # To prevent infinite loops

        # Define a function to calculate NPV with a given farm_costs_change
        def npv_with_farm_costs_change(farm_costs_change):
            # Temporarily set the new farm_costs_change
            original_farm_costs_change = self.farm_costs_change
            self.farm_costs_change = farm_costs_change
            npv_value = self.get_npv()
            # Restore the original farm_costs_change after calculation
            self.farm_costs_change = original_farm_costs_change
            return npv_value
        
        # Binary search for breakeven farm_costs_change
        for _ in range(max_iterations):
            mid_farm_costs_change = (low_farm_costs_change + high_farm_costs_change) / 2
            npv_mid = npv_with_farm_costs_change(mid_farm_costs_change)

            if abs(npv_mid) < tolerance:
                print(f"Breakeven Farm Costs Change found: {mid_farm_costs_change:.4f} with NPV farm ~ 0.")
                print(f"Difference from the initial Farm Costs Change: {mid_farm_costs_change - self.farm_costs_change:.4f}")
                return mid_farm_costs_change
            elif npv_mid > 0:
                low_farm_costs_change = mid_farm_costs_change
            else:
                high_farm_costs_change = mid_farm_costs_change

        # If no solution is found within max_iterations, raise an exception
        return "Breakeven farm_costs_change calculation did not converge."
    
    def plot_npv_vs_farm_costs_change(self, farm_costs_change_range=(0, 10), step=0.1, capacities=[]):
        """
        Plots NPV for a range of scaling factors (farm_costs_change) to visually show the point where NPV = 0 (breakeven).
        
        Parameters:
        - farm_costs_change_range: tuple, the range of scaling factors to test (default is from 0 to 10).
        - step: float, the increment for farm_costs_change values within the range.
        - capacities: list, optional, power capacities to calculate and plot NPV curves for (default is empty list).
        """
        # Set font properties
        font_properties = {'fontname': 'Arial', 'fontsize': 25}

        # If multiple capacities are provided, handle them
        if capacities:
            colors = plt.cm.viridis(np.linspace(0, 1, len(capacities)))
            plt.figure(figsize=(8, 8))

            # Store the original power capacity and farm_costs_change to be able to restore them after plotting
            original_power = self.conventional_economics.conventionalPVpower
            original_farm_costs_change = self.farm_costs_change

            for i, capacity in enumerate(capacities):
                # Temporarily update the power capacity
                self.pv_design.power = capacity
                # Recalculate all dependent variables for the updated capacity
                self.pv_design.derived_variables(self.pv_design.rowspacing)
                self.pv_design.get_capex_opex()

                # Generate NPV values for each farm_costs_change value
                farm_costs_change_values = np.arange(farm_costs_change_range[0], farm_costs_change_range[1], step)
                npv_values = []

                for farm_costs_change in farm_costs_change_values:
                    # Temporarily set the farm_costs_change value
                    self.farm_costs_change = farm_costs_change
                    # Calculate NPV with the current farm_costs_change value
                    npv = self.get_npv()
                    print(f"NPV for {capacity*0.001} MW at Farm Costs Change {farm_costs_change:.2f}: {npv:.2f}")
                    npv_values.append(npv)

                # Restore the original farm_costs_change value
                self.farm_costs_change = original_farm_costs_change

                # Plot NPV vs. farm_costs_change for the current capacity
                plt.plot(farm_costs_change_values, npv_values, color=colors[i], linewidth=2, label=f'{capacity*0.001} MW', linestyle='--')

            # Restore the original power capacity 
            self.pv_design.power = original_power
            # Recalculate all dependent variables for the restored capacity
            self.pv_design.derived_variables(self.conventional_economics.conventionalPVrowspacing)
            self.pv_design.adjust_row_spacing()
            self.pv_design.get_capex_opex()

            interval = step
            xticks = np.arange(farm_costs_change_range[0], farm_costs_change_range[1], interval)  # Generate tick positions

            plt.xlabel('Farm Costs Scaling Factor', **font_properties)
            plt.ylabel('NPV for Farm Business ($M)', **font_properties)
            plt.ylim(-10000000, 10000000)
            # Format y-axis to display values in millions
            plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '${:.1f}'.format(x / 1e6)))
            plt.legend(prop={'family': 'Arial', 'size': 20})
            plt.xticks(xticks, fontname='Arial', fontsize=25)
            plt.yticks(fontname='Arial', fontsize=25)
            plt.grid(True)
            plt.show()
            return  # Exit after plotting for multiple capacities

        # If no capacities are provided, default case when using the model for a single capacity
        # Generate farm_costs_change values from the specified range
        farm_costs_change_values = np.arange(farm_costs_change_range[0], farm_costs_change_range[1] + step, step)
        npv_values = []


        # Store the original farm_costs_change value to restore it after plotting
        original_farm_costs_change = self.farm_costs_change

        for farm_costs_change in farm_costs_change_values:
            # Temporarily set the farm_costs_change value
            self.farm_costs_change = farm_costs_change
            # Calculate NPV with the current farm_costs_change value
            npv = self.get_npv()
            print(f"NPV for Farm Costs Change {farm_costs_change:.2f}: {npv:.2f}")
            npv_values.append(npv)

        # Restore the original farm_costs_change value
        self.farm_costs_change = original_farm_costs_change

        # Plot NPV vs. farm_costs_change
        plt.figure(figsize=(8, 8))
        plt.plot(farm_costs_change_values, npv_values, label='NPV Farm', color="orange", linewidth=2)
        plt.fill_between(farm_costs_change_values, npv_values, 0, where=(np.array(npv_values) >= 0), color="orange", alpha=0.5)
        plt.xlabel('Cost Change (-)', **font_properties)
        plt.ylabel('Net Present Value ($M)', **font_properties)
        plt.ylim(-10000000, 10000000)
        plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '${:.1f}'.format(x / 1e6)))
        plt.title('NPV Farm vs. Farm Costs Change (scaling factor)')
        plt.legend(prop={'family': 'Arial', 'size': 20}, title = 'NPV', title_fontproperties = {'family': 'Arial', 'size': 15})
        plt.xticks(fontname='Arial', fontsize=25)
        plt.yticks(fontname='Arial', fontsize=25)
        plt.grid(True)
        plt.show()

# Breakeven calculation and plotting of NPV vs change of row spacing


    def find_breakeven_rowspacing_change(self, change_range=(-30, 30), tolerance=0.01):
        """
        Calculates the breakeven row spacing change where NPV equals zero using binary search.

        Parameters:
        - change_range (tuple): Range of row spacing change values to test (default is -30 ft to 30 ft).
        - tolerance (float): The tolerance within which the NPV is considered zero (default is 0.01).

        Returns:
        - breakeven_change (float or None): The breakeven row spacing change value, or None if no breakeven is found.
        """
        
        original_rowspacing = self.pv_design.rowspacing
        original_row_spacing_change = self.pv_design.row_spacing_change
        original_power = self.conventional_economics.conventionalPVpower

        # Set up initial variables for binary search
        low, high = change_range
        tolerance = 1e-6  # Tolerance for NPV convergence
        max_iterations = 1000  # To prevent infinite loops

        breakeven_change = None

        for _ in range(max_iterations):
            mid = (low + high) / 2

            # Temporarily update row spacing change
            self.pv_design.row_spacing_change = mid
            self.pv_design.power = original_power
            self.pv_design.derived_variables(self.conventional_economics.conventionalPVrowspacing)
            self.pv_design.adjust_row_spacing()
            self.pv_design.get_capex_opex()

            # Calculate NPV for the specified business
            npv = self.get_npv()

            # Debugging information
            print(f"Row spacing change: {mid:.2f} ft, NPV ({self.system_type}): {npv:.2f}")

            # Check if NPV is approximately zero
            if abs(npv) <= tolerance:
                breakeven_change = mid
                # Restore
                self.pv_design.row_spacing_change = original_row_spacing_change
                self.pv_design.power = original_power
                self.pv_design.derived_variables(self.conventional_economics.conventionalPVrowspacing)
                self.pv_design.adjust_row_spacing()
                self.pv_design.get_capex_opex()
                return breakeven_change
            
            if self.system_type == 'APV PV':
                # Adjust the search range for a decreasing PV NPV with respect to change in rowspacing
                if npv > 0:
                    low = mid 
                else:
                    high = mid 
            elif self.system_type == 'APV Farm':
                # Adjust the search range for an increasing farm NPV with respect to change in rowspacing
                if npv > 0:
                    high = mid
                else:
                    low = mid

        # Restore original values
        self.pv_design.row_spacing_change = original_row_spacing_change
        self.pv_design.power = original_power
        self.pv_design.derived_variables(self.conventional_economics.conventionalPVrowspacing)
        self.pv_design.adjust_row_spacing()
        self.pv_design.get_capex_opex()

        return breakeven_change

    
    def plot_npv_vs_rowspacing_change(self, change_range=(-30, 30), step=1):
        """
        Plots NPV for PV and Farm businesses over a range of row spacing change values.

        Parameters:
        - change_range (tuple): Range of row spacing change values (default is -30 ft to 30 ft).
        - step (float): Increment for row spacing change values.
        """
        # Generate row spacing change values
        change_values = np.arange(change_range[0], change_range[1] + step, step)
        npv_pv_values = []      # NPV for PV business
        npv_farm_values = []    # NPV for farm business

        # Store the original row spacing, row spacing change and power to restore
        original_rowspacing = self.pv_design.rowspacing
        original_row_spacing_change = self.pv_design.row_spacing_change
        original_power = self.conventional_economics.conventionalPVpower
        

        for change in change_values:
            # Temporarily update row spacing change
            self.pv_design.row_spacing_change = change
            # Restore original power before calculating the fixed used and total areas 
            # Indeed in this SA, initial power must always be the conventional one since we want to check how any variation from row spacing affect
            # the power and thus by extension capex, opex and npv compared to conventional case.
            self.pv_design.power = original_power
            self.pv_design.derived_variables(self.conventional_economics.conventionalPVrowspacing)  # Update dependent variables to get the fixed used and total area
            self.pv_design.adjust_row_spacing()  # Adjust for the current row spacing change
            self.pv_design.get_capex_opex()  # Recalculate CAPEX and OPEX

            # Calculate NPV for both PV and Farm businesses
            npv_pv = self.get_npv()
            print(f"NPV PV for {change:.2f} ft row spacing change: {npv_pv:.2f}")
            npv_farm = self.get_npv()
            print(f"NPV Farm for {change:.2f} ft row spacing change: {npv_farm:.2f}")

            # Append NPVs to their respective lists
            npv_pv_values.append(npv_pv)
            npv_farm_values.append(npv_farm)


        # Restore the original row spacing and row spacing change
        self.pv_design.row_spacing_change = original_row_spacing_change
        self.pv_design.power = original_power
        self.pv_design.derived_variables(self.conventional_economics.conventionalPVrowspacing)
        self.pv_design.adjust_row_spacing()  # Adjust for the current row spacing change
        self.pv_design.get_capex_opex()

        # Plot NPVs
        plt.figure(figsize=(10, 6))
        plt.plot(change_values, npv_pv_values, label='NPV PV', color='skyblue', linewidth=2)
        plt.plot(change_values, npv_farm_values, label='NPV Farm', color='orange', linewidth=2)

        # Fill areas for positive NPVs
        plt.fill_between(change_values, npv_pv_values, 0, where=(np.array(npv_pv_values) >= 0), color='skyblue', alpha=0.2)
        plt.fill_between(change_values, npv_farm_values, 0, where=(np.array(npv_farm_values) >= 0), color='orange', alpha=0.2)

        # Labels and legend
        plt.xlabel('Row Spacing Change (ft)')
        plt.ylabel('Net Present Value ($)')
        plt.title('NPV PV and Farm vs. Row Spacing Change')
        plt.legend()
        plt.grid(True)

        # Show the plot
        plt.show()



# LCOE, PPR calculations (only for APV system)

    def get_lcoe(self):
        """
        Calculates the Levelized Cost of Energy (LCOE).
        
        Parameters:
        - capex: Capital expenditure at year 0 ($)
        - opex: Annual operational expenditure ($/year)
        - energy_yield_0: Initial energy yield at year 0 (kWh)
        - discount_factor: Discount factor for discounted calculations (the discount factor is computed in class Financial_Rates)
        - deg_rate: Degradation rate for energy yield per year (as a decimal, e.g., 0.01 for 1%)
        - years: Number of years for the project lifespan
        
        Returns:
        - LCOE: Levelized Cost of Energy ($/kWh)
        """
        
        years = self.lifetime.years
        discount_factors = self.get_discount_factors()

        total_disc_costs = 0
        total_energy_prod = 0

        if self.system_type == 'conventional PV':
    
            for t in range(years + 1):
                
                # CAPEX is only applied at year 0
                capex_t = self.get_total_initial_investment() if t == 0 else 0
                
                # OPEX is yearly
                opex_t = self.get_opex(t) if t > 0 else 0
                
                # Total discounted cost over the project lifetime
                total_disc_costs += (capex_t + opex_t) * discount_factors[t]
                
                # Total degradation-adjusted and discounted energy production over the project lifetime
                energy_prod_t = self.pv_design.get_energy_prod(t) 
                total_energy_prod += energy_prod_t * discount_factors[t]
            
            # LCOE is the ratio of CAPEX and OPEX discounted of costs to total energy production
            lcoe = total_disc_costs / total_energy_prod

            return lcoe

        elif self.system_type == 'APV PV':

            for t in range(years + 1):
                
                # CAPEX is only applied at year 0
                capex_t = self.get_total_initial_investment() if t == 0 else 0
                
                # OPEX is yearly
                opex_t = self.get_opex(t) if t > 0 else 0
                
                # Total discounted cost over the project lifetime
                total_disc_costs += (capex_t + opex_t) * discount_factors[t]
                
                # Total degradation-adjusted and discounted energy production over the project lifetime
                energy_prod_t = self.pv_design.get_energy_prod(t) 
                total_energy_prod += energy_prod_t * discount_factors[t]
            
            # LCOE is the ratio of CAPEX and OPEX discounted of costs to total energy production
            lcoe = total_disc_costs / total_energy_prod

            return lcoe
        else: 
            return "No LCOE for farm business"
        
        

    def get_ppr(self):
        """
        Calculates the Price Performance Ratio (PPR) only for APV system (no use for conventional system by definition).
        
        Parameters:
        - overall project extra price to convert into APV: costs APV - costs conventional ($)
        - overall project APV discounted farm Revenue ($) 
        
        Returns:
        - ppr in (-)
        """
        if self.system_type == 'APV':
            capex_APV = self.get_total_initial_investment()  # contains both capex of PV and farm under APV
            capex_pv_conventional = self.conventional_economics.conventionalPVcapex  # capex of PV under conventional system
            opex_pv_conventional = self.conventional_economics.conventionalPVopex    # opex of PV under conventional system
            energy_yield_APV = self.pv_design.energy_yield
            energy_yield_conventional = self.conventional_economics.conventionalPVyield
            years = self.lifetime.years
            discount_factors = self.get_discount_factors()
            
            # Initialize totals for discounted costs and farm revenues
            total_disc_costs = 0
            total_farm_revenues = 0

                
            for t in range(years + 1):
                
                # CAPEX difference is only applied at year 0
                capex_t = capex_APV - capex_pv_conventional if t == 0 else 0
                
                # OPEX difference is yearly
                opex_APV = self.get_opex(t) # contains both opex of PV and farm under APV
                opex_t = opex_APV - opex_pv_conventional if t > 0 else 0
                
                # Total discounted cost difference for year t
                total_disc_costs += (capex_t + opex_t) * discount_factors[t]

                # Sum discounted farm revenues for all crops for each year t
                if t > 0:
                    # Fetch establishment and production revenues for year t
                    crop_revenues = self.get_APV_crop_revenues(t)
                    
                    
                    # Add establishment and production revenues for all crops in year t
                    total_farm_revenues += crop_revenues * discount_factors[t]
                   

                else:
                    total_farm_revenues += 0  # No revenues in year 0

                
    
            # Calculate PPR as the ratio of total discounted cost difference to total discounted farm revenues
            if total_farm_revenues > 0:
                ppr = total_disc_costs * (energy_yield_conventional / energy_yield_APV) / total_farm_revenues
            else:
                ppr = "Insufficient farm revenues for calculating PPR"
            
        else:
            ppr = "No PPR for conventional systems"

        return ppr            

  
# Outputs method to get quickly some important results in a list
   
    def outputs(self):
        """
        Returns a list of results over the project's timeline, including EBT, cashflows, LCOE, NPV, IRR.
        """
        years = self.lifetime.years
        #lcoe = self.get_lcoe()  # LCOE is typically constant across years
        #ppr = self.get_ppr()

        if self.system_type == 'APV PV':
            
            cashflows = [self.get_cashflows(t) for t in range(years + 1)]
            ebt = [self.get_ebt(t) for t in range(years + 1)]
            npv = self.get_npv()
            irr = self.get_irr()
            lcoe = self.get_lcoe() 
        elif self.system_type == 'APV farm':
            
            cashflows = [self.get_cashflows(t) for t in range(years + 1)]
            ebt = [self.get_ebt(t) for t in range(years + 1)]
            npv = self.get_npv()
            irr = self.get_irr()
            lcoe = self.get_lcoe() 
        else:
            cashflows = [self.get_cashflows(t) for t in range(years + 1)]
            ebt = [self.get_ebt(t) for t in range(years + 1)]
            npv = self.get_npv()
            irr = self.get_irr()
            lcoe = self.get_lcoe()  

        

        return [ebt, cashflows, lcoe, npv, irr]

# B - Displaying Results

## 1 - Initializing the different modules

In [ ]:
# Location 
location_instance = Location()

In [ ]:
# Year 
lifetime_instance = Lifetime()

In [ ]:
# Partnership 
partnership_instance = Partnership()

In [ ]:
# Unit Capex and Opex 

# Conventional system 
print("Conventional system")
capex_opex_conventional = CapexOpex(system_type="conventional")

# APV system 
print("APV system")
capex_opex_apv = CapexOpex(system_type="APV")

In [ ]:
capex_opex_apv.compute_total_unit_capex_for_scales()

In [ ]:
capex_opex_apv.find_best_trendline()


In [ ]:
# Financial Rates
financial_rates = FinancialRates(lifetime_instance)

In [ ]:
# CropSchedule
cropschedule = CropSchedule(location_instance, lifetime_instance, crops_df)

In [ ]:
# ConventionalEconomics class with multiple crops
conventional_economics = ConventionalEconomics(
    location=location_instance,
    cropschedule=cropschedule,  # Pass the entire CropSchedule instance to access selected crops
    lifetime = lifetime_instance,
    crops_df=crops_df,
    financial_rates=financial_rates,
    capex_opex_conventional=capex_opex_conventional
)

In [ ]:
# PV Design
pv_design = PVDesign(financial_rates, lifetime_instance, conventional_economics, capex_opex_apv)

In [ ]:
# Initialize CropYield
crop_yield = CropYield(
    cropschedule=cropschedule,  # Pass the CropSchedule instance to access selected crops
    financial_rates=financial_rates,
    conventional_economics = conventional_economics
)

# Output yields for all selected crops
yields = crop_yield.outputs()

# Print the results for each crop
for crop, yield_value in zip(cropschedule.selected_crops, yields):
    print(f"Crop: {crop}, Yield: {yield_value} t/acre")



In [ ]:
#Financial Tool

#Financial Tool for APV Systems
financial_tool_APV_PV = FinancialTool(
    pv_design, 
    cropschedule, 
    crop_yield, 
    financial_rates, 
    partnership_instance,
    conventional_economics,
    lifetime_instance, 
    system_type='APV PV'
    
)

financial_tool_APV_farm = FinancialTool(
    pv_design, 
    cropschedule, 
    crop_yield, 
    financial_rates, 
    partnership_instance,
    conventional_economics,
    lifetime_instance, 
    system_type='APV farm'
    
)

#Financial Tool for Conventional system (both PV and farm businesses)
financial_tool_conventionalPV = FinancialTool(
    pv_design, 
    cropschedule, 
    crop_yield, 
    financial_rates, 
    partnership_instance,
    conventional_economics, 
    lifetime_instance, 
    system_type='conventional PV'
    
)

financial_tool_conventionalfarm = FinancialTool(
    pv_design, 
    cropschedule, 
    crop_yield,  
    financial_rates, 
    partnership_instance,
    conventional_economics, 
    lifetime_instance, 
    system_type='conventional farm'
    
)



# ==================================================

## 2 - Summarized Results (NPV, IRR, LCOE)

In [ ]:
#How to get a quick view of all results of the financial tool thanks to the outputs method I have defined in each class
print(financial_tool_APV_PV.outputs())
print(financial_tool_APV_farm.outputs())

In [ ]:
print(financial_tool_conventionalPV.outputs())
print(financial_tool_conventionalfarm.outputs())

In [ ]:
#print area farm in APV
print(pv_design.get_area_farm())

In [ ]:
print(financial_rates.get_wacc())
print(financial_rates.get_effective_tax_rate())
print(financial_rates.get_nominal_discount_rate())

# ==================================================

## 3 - Cumulated Discounted Cash Flows (all scenarios)

In [ ]:
financial_tool_APV_PV.plot_cumulated_discounted_cash_flows(color = 'skyblue', y_min=-2200000, y_max=2500000)
financial_tool_APV_farm.plot_cumulated_discounted_cash_flows(color = 'orange', y_min=-1100000, y_max=500000)



In [ ]:
financial_tool_conventionalPV.plot_cumulated_discounted_cash_flows(color='skyblue',y_min=-2200000, y_max=2500000)
financial_tool_conventionalfarm.plot_cumulated_discounted_cash_flows(color='orange', y_min=-1100000, y_max=500000)

# ==================================================

## 4 - Checking some results on carryover and remaining debt after the plots (to see if plots don't affect the results (normally they shouldn't))

In [ ]:
financial_tool_APV_PV.remaining_debt

In [ ]:
financial_tool_APV_PV.carryover

In [ ]:
financial_tool_conventionalPV.carryover

# ==================================================

## 5 - DCF Tables (all scenarios)

In [ ]:
financial_tool_APV_PV.print_dcf_table()

In [ ]:
financial_tool_APV_farm.print_dcf_table()

In [ ]:
financial_tool_conventionalPV.print_dcf_table()

In [ ]:
financial_tool_conventionalfarm.print_dcf_table()

# ==================================================

## 6 - Buget Tables (farm businesses)

In [ ]:
financial_tool_APV_farm.generate_farm_budget_table()

In [ ]:
financial_tool_conventionalfarm.generate_farm_budget_table()

# ==================================================

## 7 - Sensitivity Analysis and Breakeven point

# ==================================================

### 7.1 - NPV vs Discount rate (for PV business)

In [ ]:
financial_tool_APV_PV.plot_npv_vs_discount_rate(rate_range=(0, 0.15), step=0.01)

In [ ]:
financial_tool_APV_PV.plot_npv_vs_discount_rate(rate_range=(0.025, 0.075), step=0.01, capacities=[500, 1000, 3000, 10000, 20000])

In [ ]:
financial_tool_APV_PV.plot_npv_vs_discount_rate(rate_range=(0, 0.15), step=0.01, capacities=[3000, 10000], srec_incentives_duration=[15, 20, 25])

# ==================================================

### 7.2 - NPV vs Discount rate (for farm business)

In [ ]:
financial_tool_APV_farm.plot_npv_vs_discount_rate(rate_range=(0, 0.15), step=0.01)

In [ ]:
financial_tool_APV_farm.plot_npv_vs_discount_rate(rate_range=(0.025, 0.075), step=0.01, capacities=[500, 1000, 3000, 10000, 20000])

In [ ]:
financial_tool_APV_farm.plot_npv_vs_discount_rate(rate_range=(0, 0.15), step=0.01, capacities=[3000, 10000], srec_incentives_duration=[15, 20, 25])

# ==================================================

### 7.3 - NPV vs energy price SREC incentives (for PV business)

In [ ]:
financial_tool_APV_PV.plot_npv_vs_srec_incentives(incentives_range=(0, 0.3), step=0.01)

In [ ]:
financial_tool_APV_PV.plot_npv_vs_srec_incentives(incentives_range=(0.07,0.13), step=0.02, capacities=[500, 1000, 3000, 10000, 20000])

In [ ]:
breakeven_price = financial_tool_APV_PV.find_breakeven_srec_incentives()

# ==================================================

### 7.4 - NPV vs energy price (for PV business)

In [ ]:
financial_tool_APV_PV.plot_npv_vs_energy_price(price_range=(0.02, 0.06), step=0.001)

In [ ]:
financial_tool_APV_PV.plot_npv_vs_energy_price(price_range=(0.04, 0.075), step=0.0116, capacities=[500, 1000, 3000, 10000, 20000])

In [ ]:
breakeven_price = financial_tool_APV_PV.find_breakeven_energy_price()
#print(f"Breakeven Energy Price: ${breakeven_price:.4f} per kWh")

# ==================================================

### 7.5 - NPV vs ITC (for PV business)

In [ ]:
financial_tool_APV_PV.plot_npv_vs_itc(itc_range=(0, 0.3), step=0.01)

In [ ]:
financial_tool_APV_PV.plot_npv_vs_itc(itc_range=(0, 0.3), step=0.05, capacities=[500, 1000, 3000, 10000, 20000])

In [ ]:
breakeven_price = financial_tool_APV_PV.find_breakeven_itc()
#print(f"Breakeven ITC: {breakeven_price:.4f}")

# ==================================================

### 7.6 - NPV vs crop prices (for farm business)

In [ ]:
# Plot NPV variation for all crops in the selected range
financial_tool_APV_farm.plot_npv_vs_all_crop_prices(price_range=(0, 6000), step=5)

In [ ]:
financial_tool_APV_farm.plot_npv_vs_all_crop_prices(price_range=(2670, 4960), step=458, capacities=[500, 1000, 3000, 10000, 20000])

In [ ]:
# Find the breakeven price for "BLUEBERRIES, TAME"
financial_tool_APV_farm.find_breakeven_crop_price("BLUEBERRIES, TAME")

# ==================================================

### 7.7 - NPV vs APV crop costs (for farm business)

In [ ]:
financial_tool_APV_farm.plot_npv_vs_farm_costs_change(farm_costs_change_range=(1.05, 1.1), step=0.01)

In [ ]:
financial_tool_APV_farm.plot_npv_vs_farm_costs_change(farm_costs_change_range=(1.05, 1.1), step=0.01, capacities=[500, 1000, 3000, 10000, 20000])

In [ ]:
financial_tool_APV_farm.find_breakeven_farm_costs_change()

# ==================================================

### 7.8 - NPV vs crop yields (for farm business)

In [ ]:
financial_tool_APV_farm.plot_npv_vs_all_crop_yields(yield_range=(3.9, 7.25), step=0.1)

In [ ]:
financial_tool_APV_farm.plot_npv_vs_all_crop_yields(yield_range=(3.9, 7.3), step=0.85, capacities=[500, 1000, 3000, 10000, 20000])

In [ ]:
financial_tool_APV_farm.find_breakeven_crop_yield("BLUEBERRIES, TAME")
#financial_tool_APV.find_breakeven_crop_yield("CORN, GRAIN")
#financial_tool_APV.find_breakeven_crop_yield("SOYBEANS")

# ==================================================

### 7.9 - Solar Developer to Farmer Payments

#### Only when Partnership A (will crash the code if not Partnership A)

In [ ]:
financial_tool_APV_farm.plot_npv_vs_extra_farmer_payment(payment_range=(0, 1500), step=100)

In [ ]:
financial_tool_APV_farm.get_optimal_extra_farmer_payment()

#### Only when Partnership B (will crash the code if not Partnership B)

In [ ]:
financial_tool_APV_PV.plot_npv_vs_land_lease_payment(payment_range=(0, 6000), step=100)

In [ ]:
financial_tool_APV_PV.get_optimal_land_lease_payment()

In [ ]:
financial_tool_APV_farm.plot_npv_vs_land_lease_payment(payment_range=(0, 6000), step=100)

In [ ]:
financial_tool_APV_farm.get_optimal_land_lease_payment()

# ==================================================

### 7.10 - NPV vs change in rowspacing (for both PV and farm businesses)

# NOT WORKING (DONOT USE)

In [ ]:
financial_tool_APV_PV.plot_npv_vs_rowspacing_change(change_range=(-5, 30), step=1)

In [ ]:
financial_tool_APV_PV.find_breakeven_rowspacing_change(change_range=(-10, 60), tolerance=0.01)

In [ ]:
financial_tool_APV_farm.plot_npv_vs_rowspacing_change(change_range=(-5, 30), step=1)

In [ ]:
financial_tool_APV_farm.find_breakeven_rowspacing_change(change_range=(-10, 30), tolerance=0.01)

# ==================================================

# C - Export Tables to Excel

In [ ]:
# Export DCF tables to Excel files

### BE CAREFUL TO CHANGE THE NAME OF THE FILES SO IT CORRESPONDS TO THE CORRECT PARTNERSHIP AND CROP ###

# DCF tables
financial_tool_APV_PV.export_dcf_to_excel('dcf_table_rotation_PartnershipA.xlsx', table='cashflows')
financial_tool_APV_farm.export_dcf_to_excel('dcf_table_rotation_PartnershipA.xlsx', table='cashflows')
financial_tool_conventionalPV.export_dcf_to_excel('dcf_table_rotation_PartnershipA.xlsx', table='cashflows')
financial_tool_conventionalfarm.export_dcf_to_excel('dcf_table_rotation_PartnershipA.xlsx', table='cashflows')

# Farm budget tables
financial_tool_APV_farm.export_dcf_to_excel('farm_budgets_rotation_PartnershipA.xlsx', table='budgets')
financial_tool_conventionalfarm.export_dcf_to_excel('farm_budgets_rotation_PartnershipA.xlsx', table='budgets')


# ==================================================

# D - Extra Plots (Not automated yet)

In this part are plots made manually from the same data results that are used in the paper "Integrating farm and solar business practices for agrivoltaics financial modeling" but the data where fetched from an excel file instead of being calculated in the code.

A good way would be to introduce a way to select if user wants to have the data calculated with or without the model features (e.g., with or without establishment years or rotation or tax credits).

### Energy Production over lifetime

In [ ]:
# Collect energy production values
years = range(financial_tool_APV_PV.lifetime.years + 1)
energy_productions = [pv_design.get_energy_prod(t)*10**(-6) for t in years]

# Plot energy production
plt.plot(years, energy_productions, marker='o', color='skyblue')
plt.xlabel('Time (years)')
plt.ylabel('Energy Production (TWh)')
plt.title('Energy Production Over Project Lifetime')
plt.grid(True)
plt.show()

# Calculate and print percent decrease
start_prod = energy_productions[0]
end_prod = energy_productions[-1]
percent_decrease = 100 * (start_prod - end_prod) / start_prod
print(f"Total energy production decrease over project lifetime: {percent_decrease:.1f}%")


### NPV Energy vs Farm vs Capacity

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Data
capacity = np.array([0.5, 1, 3, 10, 20])
npv_energy = np.array([129243, 379486, 1640201, 7015276, 15649221])  
npv_farm = np.array([8951, 17902, 53705, 179016, 358033])


# Plot
plt.figure(figsize=(10, 6))
plt.scatter(capacity, npv_energy, color='blue', label='NPV Energy Data', zorder=5)
plt.plot(capacity, npv_energy, color='skyblue', linestyle='-', label='Energy Trend')

plt.scatter(capacity, npv_farm, color='red', label='NPV Farm Data', zorder=5)
plt.plot(capacity, npv_farm, color='orange', linestyle='-', label='Farm Trend')

# Add labels, title, and legend
plt.xlabel('Capacity (MW)', fontsize=12)
plt.ylabel('NPV ($)', fontsize=12)
plt.title('NPV Energy vs Farm vs Capacity', fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, linestyle='--', alpha=0.7)

# Show plot
plt.show()


### Unitary PV Capex vs System Capacity for Agrivoltaics

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
slope = -0.089524266135645
exp_intercept = 1.8574496354585432

# Function to compute y
def compute_y(x, slope, exp_intercept):
    return exp_intercept * x**slope

# Generate x values and compute y
x_values = np.linspace(0.1, 20, 500)  # Avoid x = 0 to prevent division by zero
y_values = compute_y(x_values, slope, exp_intercept)

# Plot
plt.figure(figsize=(8, 6))
plt.plot(x_values, y_values, label=r"$y = 1.8574496354585432 \cdot x^{-0.089524266135645}$", color='skyblue')
plt.title("Unitary PV Capex vs System Capacity for Agrivoltaics", fontsize=14)
plt.xlabel("System Capacity [MW]", fontsize=12)
plt.ylabel("Unitary PV Capex [$/W]", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(fontsize=12)
plt.show()


### Establishment vs no establishment years

In [ ]:
# Data
x = np.arange(26)  # X-axis positions
xticks = np.arange(0, 26, 5)
esta = [-150928.4268, -378390.9525, -441350.4356, -477965.4957, -478840.4714, -452853.773, -391946.5334,
        -355625.5245, -321189.0517, -288539.3346, -257583.6664, -228234.1505, -200407.4507, -174024.5549,
        -149010.5505, -125294.4118, -102808.7983, -81489.86352, -61277.07365, -42113.03574, -23943.33473,
        -6716.378877, 9616.746681, 25102.41883, 39784.60817, 53705.00385]

no_esta = [-150928.4268, -100924.5544, -53514.03735, -8562.376132, 34057.95308, 74467.86036, 112781.985,
           149109.0208, 183552.0243, 216208.7073, 247171.7138, 276528.8831, 304363.499, 330754.5257,
           355776.8322, 379501.4046, 401995.5474, 423323.0745, 443544.4901, 462717.1607, 480895.4773,
           498131.0103, 514472.6553, 529966.7721, 544657.3161, 558585.9632]

# Bar width
bar_width = 0.4  

# Plot
plt.figure(figsize=(8, 8))
plt.bar(x - bar_width/2, esta, width=bar_width, label="Establishment & production years",  color="darkcyan", alpha=0.7)
plt.bar(x + bar_width/2, no_esta, width=bar_width, label="Only production years", color="coral", alpha=0.7)

# Set font properties
font_properties = {'fontname': 'Arial', 'fontsize': 25}

# Labels and title
plt.xlabel("Years", **font_properties)
plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:.1f}'.format(x / 1e6)))
plt.ylabel("NPV Farm Business ($M)", **font_properties)
plt.ylim(-500000, 700000)
#plt.title("Adding a distinction between Establishment and Production Years")
plt.xticks(xticks, fontname='Arial', fontsize=25)  # Ensure x-axis labels match data points
plt.yticks(fontname='Arial', fontsize=25)
plt.legend(prop={'family': 'Arial', 'size': 16.5})
plt.grid(axis="y", linestyle="--", alpha=0.7)

# Show plot
plt.show()

### Rotation vs no rotation

In [ ]:
# Data
x = np.arange(26)  # X-axis positions
xticks = np.arange(0, 26, 5)

rotation = [-350000, -281712.892, -288993.2531, -227608.5386, -234153.0079, -178972.9998, -184855.9607,
            -135253.4933, -140541.8103, -95953.11145, -100706.8906, -60625.1734, -64898.4452, -28868.14848,
            -32709.48218, -321.0923133, -3774.147796, 25340.45558, 22236.43154, 48408.15952, 45617.88717,
            69144.2027, 66635.96833, 87784.26709, 85529.56257, 104540.2122]

no_rotation = [-350000, -281711.2733, -216964.211, -155575.1313, -97369.87894, -42183.33075, 10141.07258,
               59751.77072, 106789.5048, 151387.7167, 193672.9276, 233765.0968, 271777.9622, 307819.3631,
               341991.5455, 374391.4531, 405111.0013, 434237.3388, 461853.0946, 488036.6119, 512862.1711,
               536400.2002, 558717.4743, 579877.3057, 599939.7229, 618961.6411]

# Bar width
bar_width = 0.4  

# Plot
plt.figure(figsize=(8, 8))
plt.bar(x - bar_width/2, rotation, width=bar_width, label="Crop rotation",  color="darkcyan", alpha=0.7)
plt.bar(x + bar_width/2, no_rotation, width=bar_width, label="No crop rotation", color="coral", alpha=0.7)

# Set font properties
font_properties = {'fontname': 'Arial', 'fontsize': 25}

# Labels and title
plt.xlabel("Years", **font_properties)
plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:.1f}'.format(x / 1e6)))
plt.ylabel("NPV Farm Business ($M)", **font_properties)
plt.ylim(-500000, 700000)
#plt.title("Adding a distinction between Establishment and Production Years")
plt.xticks(xticks, fontname='Arial', fontsize=25)  # Ensure x-axis labels match data points
plt.yticks(fontname='Arial', fontsize=25)
plt.legend(prop={'family': 'Arial', 'size': 16.5})
plt.grid(axis="y", linestyle="--", alpha=0.7)

# Show plot
plt.show()

### ITC 30% vs ITC 0%

In [ ]:
# Data
x = np.arange(26)  # X-axis positions
xticks = np.arange(0, 26, 5)

# EP_S
#itc_30 = [-2034351.9,-1773502.7,-1526047.0,-1291306.7,-1068638.8,-857432.9,-657110.3,-467121.8,-286946.2,-116089.0,45918.9,199523.0,345145.9,483188.8,614032.3,738038.1,855549.1,966891.3,1072373.9,1172290.5,1266920.1,1356527.5,1441364.1,1521668.8,1596122.3,1640201.9]
#itc_0 = [-2034352,-1773502.8,-1526047.1,-1291306.8,-1068638.8,-857433,-665885.5,-527902.5,-397705.5,-274903.8,-159126.9,-50023.1,52741.5,149483.64,240504.06,326088.22,406507.11,482017.99,552865.11,619280.39,681484.03,739685.14,794082.31,844864.14,892209.75,936289.28]

# CR_S

itc_30 = [-11364053.7,-10542577.3,-9755247.0,-9000706.3,-8277654.5,-7584844.4,-6921079.8,-6347716.9,-5799502.5,-5275449.3,-4774609.9,-4296075.1,-3838971.9,-3402462.5,-2985742.4,-2588038.9,-2208610.0,-1846743.2,-1501753.9,-1172984.3,-859802.5,-561601.3,-277797.2,-7829.2,248841.7,492733.7]
itc_0 = [-11364053.7,-10542577.3,-9755247.0,-9000706.3,-8277654.5,-7584844.4,-6921079.8,-6536526.7,-6172142.1,-5827147.4,-5500795.3,-5192369.0,-4901180.3,-4626568.5,-4367899.4,-4124564.0,-3895977.6,-3681578.4,-3480827.2,-3293205.6,-3118215.8,-2955379.6,-2804237.3,-2664347.3,-2535285.1,-2416642.7]


# Bar width
bar_width = 0.4  

# Plot
plt.figure(figsize=(8, 8))
plt.bar(x - bar_width/2, itc_30, width=bar_width, label="ITC 30%",  color="darkcyan", alpha=0.7)
plt.bar(x + bar_width/2, itc_0, width=bar_width, label="ITC 0%", color="coral", alpha=0.7)

# Set font properties
font_properties = {'fontname': 'Arial', 'fontsize': 25}

# Labels and title
plt.xlabel("Years", **font_properties)
plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:.1f}'.format(x / 1e6)))
plt.ylabel("NPV PV Business ($M)", **font_properties)
plt.ylim(-13500000, 1000000)
#plt.title("Adding a distinction between Establishment and Production Years")

plt.xticks(xticks, fontname='Arial', fontsize=25)  # Ensure x-axis labels match data points
plt.yticks(fontname='Arial', fontsize=25)
plt.legend(prop={'family': 'Arial', 'size': 16.5})
plt.grid(axis="y", linestyle="--", alpha=0.7)

# Show plot
plt.show()

### SREC vs no SREC

In [ ]:
# Data
x = np.arange(26)  # X-axis positions
xticks = np.arange(0, 26, 5)


# EP_S
#srec = [-2034351.9,-1773502.7,-1526047.0,-1291306.7,-1068638.8,-857432.9,-657110.3,-467121.8,-286946.2,-116089.0,45918.9,199523.0,345145.9,483188.8,614032.3,738038.1,855549.1,966891.3,1072373.9,1172290.5,1266920.1,1356527.5,1441364.1,1521668.8,1596122.3,1640201.9]
#no_srec = [-2034351.9, -2128517.1, -2217155.3, -2300596.9, -2379153.3, -2453118.0, -2522767.9, -2588363.7, -2650151.1, -2708361.5, -2763212.9, -2814910.7, -2863648.2, -2909607.3, -2952959.3, -2993865.3, -3032477.1, -3068937.3, -3103379.9, -3135931.1, -3166709.5, -3195826.6, -3223387.0, -3249489.0, -3274225.2, -3297682.1]

# CR_S
srec = [-11364053.7,-10542577.3,-9755247.0,-9000706.3,-8277654.5,-7584844.4,-6921079.8,-6347716.9,-5799502.5,-5275449.3,-4774609.9,-4296075.1,-3838971.9,-3402462.5,-2985742.4,-2588038.9,-2208610.0,-1846743.2,-1501753.9,-1172984.3,-859802.5,-561601.3,-277797.2,-7829.2,248841.7,492733.7]
no_srec =[-11364053.7,-11543260.4,-11704810.2,-11850028.4,-11980157.1,-12096360.4,-12199728.4,-12306503.2,-12403217.4,-12490706.3,-12569751.0,-12641081.0,-12705377.5,-12763276.1,-12815369.6,-12862210.4,-12904312.9,-12942155.7,-12976184.0,-13006811.2,-13034421.0,-13059369.5,-13081986.3,-13102576.6,-13121422.5,-13138784.4]

# Bar width
bar_width = 0.4  

# Plot
plt.figure(figsize=(8, 8))
plt.bar(x - bar_width/2, srec, width=bar_width, label="SREC (100$/MWh)",  color="darkcyan", alpha=0.7)
plt.bar(x + bar_width/2, no_srec, width=bar_width, label="No SREC", color="coral", alpha=0.7)

# Set font properties
font_properties = {'fontname': 'Arial', 'fontsize': 25}

# Labels and title
plt.xlabel("Years", **font_properties)
plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:.1f}'.format(x / 1e6)))
plt.ylabel("NPV PV Business ($M)", **font_properties)
plt.ylim(-13500000, 1000000)
#plt.title("Adding a distinction between Establishment and Production Years")

plt.xticks(xticks, fontname='Arial', fontsize=25)  # Ensure x-axis labels match data points
plt.yticks(fontname='Arial', fontsize=25)
plt.legend(prop={'family': 'Arial', 'size': 16.5})
plt.grid(axis="y", linestyle="--", alpha=0.7)

# Show plot
plt.show()

### Carryover and no tax if EBT<0 vs no carryover and tax if EBT<0

In [ ]:
# Data
x = np.arange(26)  # X-axis positions
xticks = np.arange(0, 26, 5)
carryfwd = [-2034352, -1773502.8, -1526047.1, -1291306.8, -1068638.8, -857433, -657110.4, -467121.9,
            -286946.3, -116089.1, 45918.9, 199523, 345145.9, 483188.7, 614032.3, 738038, 855549.1,
            966891.3, 1072373.9, 1172290.54, 1266920.13, 1356527.49, 1441364.11, 1521668.85, 1596122.35, 1640201.88]

no_carryfwd = [-2034352, -217411.7472, 228573.0693, 551699.0179, 800935.2958, 1036853.219, 1221564.113,
               1359519.523, 1489686.839, 1612456.998, 1728200.873, 1837270.486, 1939999.911, 2036706.203,
               2127690.318, 2213237.866, 2293620.074, 2369094.359, 2439905.093, 2506284.393, 2568452.554,
               2626618.805, 2680981.849, 2731730.383, 2779043.602, 2823091.711]

# Bar width
bar_width = 0.4  

# Plot
plt.figure(figsize=(8, 8))
plt.bar(x - bar_width/2, carryfwd, width=bar_width, label="Tax credits carried forward",  color="darkcyan", alpha=0.7)
plt.bar(x + bar_width/2, no_carryfwd, width=bar_width, label="Tax credits accounted during Year 1", color="coral", alpha=0.7)

# Set font properties
font_properties = {'fontname': 'Arial', 'fontsize': 25}

# Labels and title
plt.xlabel("Years", **font_properties)
plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:.1f}'.format(x / 1e6)))
plt.ylabel("NPV PV Business ($M)", **font_properties)
#plt.title("Adding a distinction between Establishment and Production Years")

plt.xticks(xticks, fontname='Arial', fontsize=25)  # Ensure x-axis labels match data points
plt.yticks(fontname='Arial', fontsize=25)
plt.legend(prop={'family': 'Arial', 'size': 16.5})
plt.grid(axis="y", linestyle="--", alpha=0.7)

# Show plot
plt.show()